Le code suivant permet de créer un fichier de données (au format csv) contenant les informations sur toutes les communes du terrritoire SRU, il y a des données des bilans annuels et des données des bilans triannaux. Le fichier produit est similaire au fichier produit par le code de donnees_2, cependant des informations supplémentaires sont présentes dans le fichiers produit par le code ci-dessous. Ce sont des informations quantitatives concernant le nombre total pour chaque année entre 2004 et 2021 et le nombre de construction de logements sociaux pour chaque triennal.

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# I. Bilans triennaux
## 1. Bilan triennal 2002--2004

In [2]:
# Chargement des données du bilan triennal 2002-2004
bt_02_04 = pd.read_csv("../donnees_initiales/bilans_triennaux/bilan_triennal_2002_2004.csv",
                       sep = ';')
bt_02_04.head()

,Dép,Unnamed: 1,numcom,commune,Taux LS 2004,Bilan 2002-2004,Objectif 2002-2004,Taux de réalisation 2002-2004,Carence
0,1,1043,1043,Beynost,"4,70%",53.0,37.0,"143,2%",0
1,1,1142,1142,Dagneux,"9,12%",56.0,23.0,"243,5%",0
2,1,1313,1313,Prévessin-Moëns,"11,65%",46.0,18.0,"255,6%",0
3,1,1322,1322,Reyrieux,"6,03%",18.0,28.0,"64,3%",0
4,1,1344,1344,Saint-Denis-lès-Bourg,"3,47%",61.0,47.0,"129,8%",0


La colonne Unnamed: 1 est une redite de numcol, on va donc la supprimer.\
Les numéros de département ne sont pas écrit sous le bon format, tout comme les numéro des communes.\
Les taux LS 2004 et Taux de réalisation 2002-2004 sont écrit avec le signe "%".

In [3]:
bt_02_04.drop('Unnamed: 1', axis = 1, inplace = True) # suppression de la colonne 'Unnamed: 1'

In [4]:
bt_02_04.dtypes # Pour avoir des informations sur les variables

Dép                               object
numcom                            object
commune                           object
Taux LS 2004                      object
Bilan 2002-2004                  float64
Objectif 2002-2004               float64
Taux de réalisation 2002-2004     object
Carence                           object
dtype: object

In [5]:
departements = [] # Le numéro de département est une chaîne de caractère avec au moins deux caractères
                  # (on va ajouter 1 caractère au début du numéro de département quand il est manquant)
code_commune = [] # Le numéro de la commune est une chaîne de caractères contenant 5 caractères, parfois
                  # le premier '0' est manquant, on va l'ajouter
taux_ls = [] # taux de logements sociaux
taux_real = [] # taux de logements sociaux construits durant les 3 ans sur l'objectif à atteindre

for i in range(len(bt_02_04)) : # Pour chaque commune
    departements.append(bt_02_04.Dép[i].zfill(2)) # zfill(2) permet d'ajouter des "0" à gauche pour qu'il y est au moins 2 caractères
                                                  # on ajoute le numéro du département à la liste départements
    code_commune.append(bt_02_04.numcom[i].zfill(5)) # idem pour le code de la commune avec 5 caractères que l'on ajoute à la liste code_commune
    taux_ls.append(float(bt_02_04["Taux LS 2004"][i].replace(',', '.')[:-1])/100)
        # Le taux de logements sociaux est comme chaine de caractères
        # de la forme n% avec n/100 le taux de logements sociaux de la commune
        # pour le transfromer en nombre réel entre 0 et 1, on remplace "," par "." (.replace(',', '.')) ;
        # on enlève le dernier caractère ([:-1]) ;
        # c'est maintenant un nombre donc on transforme la chaine de caractères en nombre (float) ;
        # et on divise ce nombre par 100 pour avoir le taux.
        # on ajoute ce taux à la liste taux_ls
    temp = str(bt_02_04["Taux de réalisation 2002-2004"][i]).replace(',', '.')[:-1]
        # le taux de réalisation est stocké sous la même forme que le taux de logements sociaux
        # on réalise donc des opérations similaires
        # sauf pour certaines communes où le taux de réalisation n'est pas indiqué
    if temp == 'na' : # lorsque la valeur n'est pas indiquée
        taux_real.append(np.nan) # on ajoute une valeur manquante à la liste taux_real
    else : # sinon
        taux_real.append(float(temp)/100) # on ajoute la valeur du taux de réalisation
        
# On remplace les colonnes du DataFrame du bilan triennal 2002-2004 par les colonnes que l'on a créées
bt_02_04.Dép = departements
bt_02_04.numcom = code_commune
bt_02_04["Taux LS 2004"] = taux_ls
bt_02_04["Taux de réalisation 2002-2004"] = taux_real

In [6]:
bt_02_04.head() # affichage des 5 premières lignes du DataFrame

,Dép,numcom,commune,Taux LS 2004,Bilan 2002-2004,Objectif 2002-2004,Taux de réalisation 2002-2004,Carence
0,01,01043,Beynost,0.0470,53.0,37.0,1.432,0
1,01,01142,Dagneux,0.0912,56.0,23.0,2.435,0
2,01,01313,Prévessin-Moëns,0.1165,46.0,18.0,2.556,0
3,01,01322,Reyrieux,0.0603,18.0,28.0,0.643,0
4,01,01344,Saint-Denis-lès-Bourg,0.0347,61.0,47.0,1.298,0


In [7]:
# Création du DataFrame pour le bilan 2002-2004

temp = [] # Variable temporaire pour stocker les données

for i in range(len(bt_02_04)) : # Pour chaque commune du bilan
    # liste contient les données pour une commune
    liste = [bt_02_04.numcom[i], bt_02_04['Carence  '][i]] # le numéro de la commune, carence de la commune (amende ou non)
    
    # dans la suite on ajoute la situation de la commune dans liste
    if bt_02_04["Taux LS 2004"][i] > 0.2 : # En 2004 le taux à atteindre était de 0,2
        liste.append("Soumise taux atteint en 2004") 
    elif (bt_02_04["Taux de réalisation 2002-2004"][i] >= 1) :
        liste.append("Soumise objectif triennal 2002-2004 atteint")
    elif bt_02_04[bt_02_04.columns[-1]][i] == '1' :
        liste.append("Soumise objectif triennal 2002-2004 non atteint avec amende")
    else  :
        liste.append("Soumise objectif triennal 2002-2004 non atteint sans amende")
    
    # on ajoute le nombre de constructions durant le triennal dans liste
    liste.append(bt_02_04["Bilan 2002-2004"][i])
    
    # objectif de construction du triennal
    liste.append(bt_02_04["Objectif 2002-2004"][i])
        
    temp.append(liste) # Les données de la commune sont ajoutées aux données des autres communes

# création du DataFrame df_02_04 avec les données contenues dans temp réparties dans les colonnes code_commune, carence_02_04, situation_02_04
df_02_04 = pd.DataFrame(temp, columns = ['code_commune', 'carence_02_04', 'situation_02_04', 'construction_02_04', 'objectif_02_04'], index = bt_02_04.numcom)
df_02_04.replace(' -', False, inplace = True)
df_02_04.replace('0', False, inplace = True)
df_02_04.replace('1', True, inplace = True)
#df_02_04.carence_02_04 = df_02_04.carence_02_04.astype(bool)
df_02_04.head() # affichage des 5 premières lignes du DataFrame

,code_commune,carence_02_04,situation_02_04,construction_02_04,objectif_02_04
numcom,,,,,
01043,01043,False,Soumise objectif triennal 2002-2004 atteint,53.0,37.0
01142,01142,False,Soumise objectif triennal 2002-2004 atteint,56.0,23.0
01313,01313,False,Soumise objectif triennal 2002-2004 atteint,46.0,18.0
01322,01322,False,Soumise objectif triennal 2002-2004 non attein...,18.0,28.0
01344,01344,False,Soumise objectif triennal 2002-2004 atteint,61.0,47.0


## 2. Bilan triennal 2005--2007

In [8]:
# Chargement des données du bilan triennal 2005-2007
bt_05_07 = pd.read_csv("../donnees_initiales/bilans_triennaux/bilan_triennal_2005_2007.csv",
                       sep = ';')
bt_05_07.head() # affichage des premières lignes

,Region,Dept,departement,Unnamed: 3,Code INSEE,Commune,Taux 2007,Objectif 2005-2007,Bilan \n2005-2007\n(a)-(b)+( c)+(d) pr IDF,Taux de réalisation,Arrêté de carence,Taux de majoration,gfd
0,Alsace,67,Bas Rhin,67046,67046,Bischwiller,"15,10%",36,149,414%,0,NaN,Bischwiller
1,Alsace,67,Bas Rhin,67118,67118,Eckbolsheim,"5,31%",69,13,19%,0,NaN,Eckbolsheim
2,Alsace,67,Bas Rhin,67152,67152,Geispolsheim,"6,12%",55,43,78%,1,"21,82",Geisposheim
3,Alsace,67,Bas Rhin,67180,67180,Haguenau,"11,17%",217,134,62%,0,NaN,Haguenau
4,Alsace,67,Bas Rhin,67204,67204,Hoenheim,"12,20%",58,115,198%,0,NaN,Hoenheim


In [9]:
bt_05_07.dtypes # Affichage des noms des colonnes avec leur type (entier, chaine de caractères, ...)

Region                                         object
Dept                                           object
departement                                    object
Unnamed: 3                                     object
Code INSEE                                     object
Commune                                        object
Taux 2007                                      object
Objectif 2005-2007                             object
Bilan \n2005-2007\n(a)-(b)+( c)+(d) pr  IDF    object
Taux de réalisation                            object
Arrêté de carence                               int64
Taux de majoration                             object
gfd                                            object
dtype: object

Pour certaines communes il y a des valeurs manquantes pour les valeurs de la colonne "Taux 2007" dont on a besoin pour trouver la situation de la commune. Pour ces communes, j'ai supposé que les valeurs n'étaient pas atteintes. Trois communes sont concernées : 65047 (Aureilhan), 65100 (Bordères-sur-l'Echez) et 65417 (Séméac).

In [10]:
a = 0 # variable pour vérifier que les données de toutes les communes ont été récupérées
      # si elle reste à 0, toutes les communes apparaissent dans la dataframe
temp = [] # variable temporaire pour stocker les données

for i in range(len(bt_05_07)) : # pour chaque commune
    # on met les informations de la commune dans liste
    liste = [bt_05_07["Code INSEE"][i],  bt_05_07["Arrêté de carence"][i]] # le code de la commune, carence de la commune
    t1 = bt_05_07["Taux 2007"][i].replace(',', '.')[:-1] # le taux de logements sociaux
    t2 = bt_05_07["Taux de réalisation"][i].replace(',', '.')[:-1] # le taux de réalisation
    
    if t1 == 'n' : # si le taux de logements sociaux est manquant, on suppose que le taux de 0,2 n'est pas atteint
                   # dans ce cas, seules les autres situations sont possibles
        #print(bt_05_07["Code INSEE"][i], bt_05_07.Commune[i])
        if (float(t2)/100 >= 1) :
            liste.append("Soumise objectif triennal 2005-2007 atteint")
        elif bt_05_07["Arrêté de carence"][i] == 0 :
            liste.append("Soumise objectif triennal 2005-2007 non atteint sans amende")
        elif bt_05_07["Arrêté de carence"][i] == 1 :
            liste.append("Soumise objectif triennal 2005-2007 non atteint avec amende")
        else :
            a = a + 1
            #print(bt_05_07.loc[i])
    elif float(t1)/100 > 0.2 :
        liste.append("Soumise taux atteint en 2007")
    elif (float(t2)/100 >= 1) :
        liste.append("Soumise objectif triennal 2005-2007 atteint")
    elif bt_05_07["Arrêté de carence"][i] == 0 :
        liste.append("Soumise objectif triennal 2005-2007 non atteint sans amende")
    elif bt_05_07["Arrêté de carence"][i] == 1 :
        liste.append("Soumise objectif triennal 2005-2007 non atteint avec amende")
    else :
        a = a + 1
        #print(bt_05_07.loc[i])
    
    # nombre de construction durant le triennal
    liste.append(int(bt_05_07["Bilan \n2005-2007\n(a)-(b)+( c)+(d) pr  IDF"][i].replace('\xa0', '')))
    
    # objectif du triennal
    liste.append(int(bt_05_07["Objectif 2005-2007"][i].replace('\xa0', '')))
        
    temp.append(liste) # on ajoute les données de la communes dans la variable temp
    
# création du DataFrame avec les données que l'on a collectées
df_05_07 = pd.DataFrame(temp, columns = ['code_commune', 'carence_05_07', 'situation_05_07', 'construction_05_07', 'objectif_05_07'], index = bt_05_07["Code INSEE"])
#print(a)
df_05_07.carence_05_07 = df_05_07.carence_05_07.astype(bool)
df_05_07.head() # affichage des premières lignes

,code_commune,carence_05_07,situation_05_07,construction_05_07,objectif_05_07
Code INSEE,,,,,
67046,67046,False,Soumise objectif triennal 2005-2007 atteint,149,36
67118,67118,False,Soumise objectif triennal 2005-2007 non attein...,13,69
67152,67152,True,Soumise objectif triennal 2005-2007 non attein...,43,55
67180,67180,False,Soumise objectif triennal 2005-2007 non attein...,134,217
67204,67204,False,Soumise objectif triennal 2005-2007 atteint,115,58


Les 3 communes concernées par le manque de données n'ont pas atteint leur objectif triennal (Soumise objectif triennal 2005-2007 non atteint sans amende) donc on peut penser que notre hypothèse est juste.

## 3. Bilan triennal 2008--2010

In [11]:
# chargement des données du bilan triennal 200_-2010
bt_08_10 = pd.read_csv("../donnees_initiales/bilans_triennaux/bilan_triennal_2008_2010.csv",
                       sep = ';')
bt_08_10.head()

,Numreg,Region,Dép,Unnamed: 3,Code,Commune,Structure d'application,Population municipale 2008,Taux LS 2010,Variation Inventaire 2010-2007,...,LLS financés en 2009,LLS financés en 2010,Bilan 2008-2010,Objectif 2008-2010,Taux de réal,Objectif non atteint,Arrêté de carence,Taux de majoration,observations,ggg
0,1,Ile-de-France,75,75056,75056,Paris,agglo,2211297,"0,162485092",10894,...,5386,6109,11196,7989,"1,40142696207285",0,NaN,NaN,NaN,75056
1,1,Ile-de-France,77,77040,77040,Boissise-le-Roi,agglo,3616,"0,032721468",0,...,0,13,31,31,1,0,NaN,NaN,NaN,77040
2,1,Ile-de-France,77,77058,77058,Bussy-Saint-Georges,agglo,21108,"0,098727907",44,...,40,320,729,103,"7,07766990291262",0,NaN,NaN,NaN,77058
3,1,Ile-de-France,77,77067,77067,Cesson,agglo,8012,"0,064146266",130,...,138,0,207,73,"2,83561643835616",0,NaN,NaN,NaN,77067
4,1,Ile-de-France,77,77085,77085,Chanteloup-en-Brie,agglo,1948,"0,100877193",23,...,100,60,218,11,"19,8181818181818",0,NaN,NaN,NaN,77085


In [12]:
bt_08_10.dtypes

Numreg                                                                                   int64
Region                                                                                  object
Dép                                                                                     object
Unnamed: 3                                                                              object
Code                                                                                    object
Commune                                                                                 object
Structure d'application                                                                 object
Population municipale 2008                                                               int64
Taux LS 2010                                                                            object
Variation Inventaire 2010-2007                                                           int64
LLS financés et comptabilisés aux précédents bilan

Pour ce bilan triennal, il manque également certaines valeurs de taux de logements sociaux. Cinq communes sont concernées : 59092 (Bouchain), 59107 (Bray-Dunes), 59206 (Escaudœuvres), 59248 (Fort-Mardyck) et 59302 (Hérin). Comme pour le bilan précédent, on suppose que le taux de 20% de logements sociaux n'est pas atteint.

In [13]:
temp = [] # variable temporaire pour stocker les données
a = 0 # variable permettant de vérifier que les données ont été récupérées pour toutes les communes

for i in range(len(bt_08_10)) : # pour chaque commune
    
    # liste contient les données pour la commune
    liste = [bt_08_10["Code"][i]] # le code de la commune
    
    t1 = bt_08_10["Taux LS 2010"][i].replace(',', '.') # le taux de logement sociaux
    t2 = bt_08_10["Taux de réal"][i].replace(',', '.') # le taux de réalisation de l'objectif
    
    if t1 == 'nd' : # si le taux de logement sociaux est une valeur manquante
        #print(bt_08_10["Code"][i], bt_08_10.Commune[i])
        if (float(t2) >= 1) :
            liste.append(False) # carence, l'objectif est atteint la commune n'est pas carencée
            liste.append("Soumise objectif triennal 2008-2010 atteint") # situation
        elif bt_08_10["Objectif non atteint"][i] == 1 :
            if bt_08_10["Arrêté de carence"][i] == 0 :
                liste.append(False) # carence
                liste.append("Soumise objectif triennal 2008-2010 non atteint sans amende") # situation
            elif bt_08_10["Arrêté de carence"][i] == 1 :
                liste.append(True) # carence
                liste.append("Soumise objectif triennal 2008-2010 non atteint avec amende") # situation
            else :
                liste.append(np.nan) # carence
                liste.append("Soumise objectif triennal 2008-2010 non atteint (aucune info amende)") # situation
        else :
            #print(bt_08_10.loc[i])
            a = a + 1
            
    elif float(t1) > 0.2 :
        liste.append(False) # carence, l'objectif est atteint la commune n'est pas carencée
        liste.append("Soumise taux atteint en 2010") # situation
    elif (float(t2) >= 1) :
        liste.append(False) # carence
        liste.append("Soumise objectif triennal 2008-2010 atteint") # situation
    elif bt_08_10["Objectif non atteint"][i] == 1 :
        if bt_08_10["Arrêté de carence"][i] == 0 :
            liste.append(False) # carence
            liste.append("Soumise objectif triennal 2008-2010 non atteint sans amende") # situation
        elif bt_08_10["Arrêté de carence"][i] == 1 :
            liste.append(True) # carence
            liste.append("Soumise objectif triennal 2008-2010 non atteint avec amende") # situation
        else :
            liste.append(np.nan) # carence
            liste.append("Soumise objectif triennal 2008-2010 non atteint (aucune info amende)") # situation
    else :
        #print(bt_08_10.loc[i])
        a = a + 1
        
    # nombre de constructions pendant le triennal
    liste.append(bt_08_10["Bilan 2008-2010 "][i])
    
    # objectif du triennal
    liste.append(bt_08_10["Objectif 2008-2010"][i])
        
    temp.append(liste) # les données de la commune sont ajoutées dans la variables temp
    
#print(temp)
#print(a)
# création du DataFrame avec les données de toutes les communes
df_08_10 = pd.DataFrame(temp, columns = ['code_commune', 'carence_08_10', 'situation_08_10', 'construction_08_10', 'objectif_08_10'], index = bt_08_10["Code"])
df_08_10.carence_08_10 = df_08_10.carence_08_10.astype(bool)
df_08_10.head()

,code_commune,carence_08_10,situation_08_10,construction_08_10,objectif_08_10
Code,,,,,
75056,75056,False,Soumise objectif triennal 2008-2010 atteint,11196,7989
77040,77040,False,Soumise objectif triennal 2008-2010 atteint,31,31
77058,77058,False,Soumise objectif triennal 2008-2010 atteint,729,103
77067,77067,False,Soumise objectif triennal 2008-2010 atteint,207,73
77085,77085,False,Soumise objectif triennal 2008-2010 atteint,218,11


## 4. Bilan triennal 2011 -- 2013

In [14]:
# chargement des données du bilan 2011-2013
bt_11_13 = pd.read_csv("../donnees_initiales/bilans_triennaux/bilan_triennal_2011_2013.csv",
                       sep = ';')
bt_11_13.head()

,code région,Unnamed: 1,région,Dept,Unnamed: 4,Code INSEE,Commune,Taux LLS au 1/01/2013,Variation du nombre de LS entre le 1er janvier 2010 (2011 ou 2012) et le 1er janvier 2013 (a),LLS financés comptabilisés aux bilans précédents et figurant à l'inventaire 2013 (b),...,date arrêté carence initial,date arrêté carence modificatif,arrêté de levée de carence,date CRHH,date CD,obsrvations,procédure terminée,Objectif minimum réglementaire SRU (h) dep 37,Taux de réalisation par rapport à l'objectif minimum réglementaire SRU (f)/(h) dep 37,ooo
0,41,grand est,Lorraine,57,57012,57012,ALGRANGE,"15,39%",69,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,oui,NaN,NaN,57012
1,41,grand est,Lorraine,57,57143,57143,CLOUANGE,"14,36%",8,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,oui,NaN,NaN,57143
2,41,grand est,Lorraine,57,57306,57306,HAYANGE,"17,39%",193,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,oui,NaN,NaN,57306
3,41,grand est,Lorraine,57,57412,57412,LONGEVILLE LES METZ,"12,30%",47,47.0,...,NaN,NaN,NaN,NaN,NaN,NaN,oui,NaN,NaN,57412
4,41,grand est,Lorraine,57,57447,57447,MARLY,"15,55%",34,39.0,...,NaN,NaN,NaN,NaN,NaN,NaN,oui,NaN,NaN,57447


In [15]:
bt_11_13["carence"].unique() # les différentes valeurs contenues dans la colonne carence

array([nan, 'non', '1', ' ', 'Non'], dtype=object)

Dans les données du bilan triennal 2011-2013, deux communes sont codés avec 6012 (ce code correspond à 06012).

In [16]:
bt_11_13.loc[bt_11_13['Code INSEE'] == '6012'] 

,code région,Unnamed: 1,région,Dept,Unnamed: 4,Code INSEE,Commune,Taux LLS au 1/01/2013,Variation du nombre de LS entre le 1er janvier 2010 (2011 ou 2012) et le 1er janvier 2013 (a),LLS financés comptabilisés aux bilans précédents et figurant à l'inventaire 2013 (b),...,date arrêté carence initial,date arrêté carence modificatif,arrêté de levée de carence,date CRHH,date CD,obsrvations,procédure terminée,Objectif minimum réglementaire SRU (h) dep 37,Taux de réalisation par rapport à l'objectif minimum réglementaire SRU (f)/(h) dep 37,ooo
700,93,PACA,PACA,6,6012,6012,BEAULIEU-SUR-MER,"7,85%",44,0.0,...,NaN,NaN,NaN,03/07/14,NaN,NaN,non,NaN,NaN,6012
729,93,PACA,PACA,6,6012,6012,BEAUSOLEIL,"7,85%",-5,14.0,...,06/08/14,sans objet,NaN,03/07/14,NaN,NaN,non,NaN,NaN,6012


Après vérification, le code 06012 est celui de la commune nommée Beausoleil et la commune nommée Beaulieu-sur-Mer à pour code 06011.

In [17]:
bt_11_13['Code INSEE'][700] = '06011' # changement pour le vrai code pour la commune Beaulieu-sur-Mer

In [18]:
bt_11_13.dtypes # noms des colonnes et leur type

code région                                                                                                   int64
Unnamed: 1                                                                                                   object
région                                                                                                       object
Dept                                                                                                         object
Unnamed: 4                                                                                                   object
Code INSEE                                                                                                   object
Commune                                                                                                      object
Taux LLS au 1/01/2013                                                                                        object
Variation du nombre de LS entre le 1er janvier 2010 (2011 ou 2012) et le

Il manque des données dans les taux de réalisation et/ou les taux de logements sociaux pour les communes suivantes :
- 62139 Blendecques
- 62757 St Martin au Laert
- nan BATENHEIM
- 29039 Concarneau
- 29241 Rosporden
- 29293 Trégunc
- 44130 PONT-SAINT-MARTIN
- 44410 LA CHAPELLE DES MARAIS
- 44210 TRIGNAC
- 14 AUBIERE
- 32 BEAUMONT
- 42 BLANZAT
- 63 CEBAZAT
- 69 LE CENDRE
- 70 CEYRAT
- 75 CHAMALIERES
- 284 PONT DU CHÂTEAU
- 307 ROMAGNAT
- 308 ROYAT
- 87201 Verneuil sur Vienne
- 27562 Saint-Marcel
- 45004 Amilly
- 45338 Villemandeur
- 97611 Mamoudzou\
Les valeurs des codes communes ne sont pas correctes pour plusieurs de ces communes non plus.

In [19]:
bt_11_13['Code INSEE'].loc[bt_11_13.Commune == 'BATENHEIM'] = '68022'
bt_11_13['Code INSEE'].loc[bt_11_13.Commune == 'AUBIERE'] = '63014'
bt_11_13['Code INSEE'].loc[bt_11_13.Commune == 'BEAUMONT'] = '63032'
bt_11_13['Code INSEE'].loc[bt_11_13.Commune == 'BLANZAT'] = '63042'
bt_11_13['Code INSEE'].loc[bt_11_13.Commune == 'CEBAZAT'] = '63063'
bt_11_13['Code INSEE'].loc[bt_11_13.Commune == 'LE CENDRE'] = '63069'
bt_11_13['Code INSEE'].loc[bt_11_13.Commune == 'CEYRAT'] = '63070'
bt_11_13['Code INSEE'].loc[bt_11_13.Commune == 'CHAMALIERES'] = '63075'
bt_11_13['Code INSEE'].loc[bt_11_13.Commune == 'PONT DU CHÂTEAU'] = '63284'
bt_11_13['Code INSEE'].loc[bt_11_13.Commune == 'ROMAGNAT'] = '63307'
bt_11_13['Code INSEE'].loc[bt_11_13.Commune == 'ROYAT'] = '63308'

Lorsqu'il y a un manque de valeurs on suppose que l'objectif n'est pas atteint. Lorsqu'il manque la valeur du taux de réalisation mais pas celle du taux de logements sociaux le taux de logements sociaux de 0,2 n'est atteint pour aucune commune dans cette situation.

In [20]:
temp = [] # variable temporaire pour stocké les données
a = 0 # variable de vérification

for i in range(len(bt_11_13)) : # pour chaque commune
    # liste contient les données pour la commune
    liste = [str(bt_11_13["Code INSEE"][i]).zfill(5)] # le code INSEE de la commune avec 5 caractères
    
    t1 = bt_11_13["Taux LLS au 1/01/2013"][i].replace(',', '.')[:-1] # taux de logements sociaux en pourcentage ([:-1] permet d'enlever le signe '%')
    t2 = str(bt_11_13["Taux de réalisation (f)/(g) (sans décompte des LS financés en 2011, 2012 ou 2013)"][i]).replace(',', '.')[:-1] # taux de réalisation de l'objectif
    
    if t1 == '#N/' or t2 == 'na' or t2 == '' or t2 == '#DIV/0 ': # s'il y a une valeur manquante
        #print(bt_11_13["Code INSEE"][i], bt_11_13.Commune[i])
        
        if bt_11_13["obj triennal non atteint"][i] == '1' : # si l'objectif triennal est atteint
            if bt_11_13["carence"][i] == 'non' or bt_11_13["carence"][i] == 'Non' :
                liste.append(False) # carence
                liste.append("Soumise objectif triennal 2011-2013 non atteint sans amende") # situation
            else :
                liste.append(np.nan) # carence
                liste.append("Soumise objectif triennal 2011-2013 non atteint (aucune info amende)") # situation
        elif bt_11_13["Objectif de réalisation           (g)"][i] == '0' : # si l'objectif est fixé à 0
            liste.append(np.nan) # carence
            liste.append("Soumise objectif triennal 2011-2013 atteint") # situation
        elif t2 != 'na' and t2 != '#DIV/0 ' :
            if float(t2) >= 100 :
                liste.append(False) # carence
                liste.append("Soumise objectif triennal 2011-2013 atteint") # situation
            else : 
                a = a + 1
        else :
            liste.append(np.nan) # carence
            liste.append(np.nan) # situation           
            
            
    elif float(t1)/100 > 0.2 :
        liste.append(False) # carence
        liste.append("Soumise taux atteint en 2013") # situation
    elif (float(t2)/100 >= 1) :
        liste.append(False) # carence 
        liste.append("Soumise objectif triennal 2011-2013 atteint") # situation
    elif bt_11_13["obj triennal non atteint"][i] == '1' :
        if bt_11_13["carence"][i] == 'non' or bt_11_13["carence"][i] == 'Non' :
            liste.append(False) # carence
            liste.append("Soumise objectif triennal 2011-2013 non atteint sans amende") # situation
        elif bt_11_13["carence"][i] == '1' :
            liste.append(True) # carence
            liste.append("Soumise objectif triennal 2011-2013 non atteint avec amende") # situation
        else :
            liste.append(np.nan) # carence
            liste.append("Soumise objectif triennal 2011-2013 non atteint (aucune info amende)") # situation
    else :
        #print(bt_11_13.loc[i])
        a = a + 1
        
    # nombre de constructions
    liste.append(bt_11_13["Bilan triennal            (a)-(b)+(c)+(d)+(e) =(f) (sans décompte de LS financés en 2011, 2012 ou 2013)"][i])
    
    # objectif triennal
    liste.append(float(str(bt_11_13["Objectif de réalisation           (g)"][i]).replace(',', '.')))
    
    temp.append(liste) # les données de la commune sont ajoutées à temp
    #print(liste)

#print(a)
# création du DataFrame du bilan triennal 2011-2013
df_11_13 = pd.DataFrame(temp,  index = bt_11_13["Code INSEE"],
                        columns = ['code_commune', 'carence_11_13', 'situation_11_13', 'construction_11_13', 'objectif_11_13'])
df_11_13.head()

,code_commune,carence_11_13,situation_11_13,construction_11_13,objectif_11_13
Code INSEE,,,,,
57012,57012,False,Soumise objectif triennal 2011-2013 atteint,69.0,26.0
57143,57143,False,Soumise objectif triennal 2011-2013 atteint,10.0,10.0
57306,57306,False,Soumise objectif triennal 2011-2013 atteint,193.0,35.0
57412,57412,False,Soumise objectif triennal 2011-2013 atteint,17.0,9.0
57447,57447,False,Soumise objectif triennal 2011-2013 atteint,69.0,9.0


## 5. Bilan triennal 2014 -- 2016

In [21]:
# chargement des données des bilans triennaux 2014-2016 et 2017-2019
bt_14_19 = pd.read_csv("../donnees_initiales/bilans_triennaux/bilan_triennal_2017_2019.csv",
                       sep = ';')
bt_14_19.head()

,Région,Département,Unnamed: 2,Code INSEE,Commune,Population municipale 2017,Agglomération d'appartenance,taux de tension moyen sur 3 ans_agglo,EPCI d'appartenance,taux de tension moyen sur 3 ans_epci,...,Nombre de bilans triennaux réalisés entre 2002 et 2016,Nb de fois où la commune n’a pas atteint son objectif quantitatif,Nb de fois où la commune a été carencée,Carence supplémentaire demandée (oui=1/non=0),Renforcement du taux de majoration demandé (oui=1/non=0) ou proposition de majoration,Demande la reprise des PC (oui=1/non=0),Demande d’information /alerte,Remarques de la commission sur la proposition de carence,Demande d’information complémentaire,Réponse de la DDT/Arguments préfet
0,Auvergne Rhône Alpes,Ain,1043,1043,Beynost,4613,Lyon,"4,4947",NaN,NaN,...,5,2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Auvergne Rhône Alpes,Ain,1142,1142,Dagneux,4717,Lyon,"4,4947",NaN,NaN,...,5,1,1,NaN,NaN,NaN,NaN,RAS,NaN,NaN
2,Auvergne Rhône Alpes,Ain,1262,1262,Montluel,6964,Lyon,"4,4947",NaN,NaN,...,0,NC,NC,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Auvergne Rhône Alpes,Ain,1281,1281,Ornex,4410,Genève (SUI) - Annemasse (partie française),"4,6745",NaN,NaN,...,1,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Auvergne Rhône Alpes,Ain,1313,1313,Prévessin-Moëns,8233,Genève (SUI) - Annemasse (partie française),"4,6745",NaN,NaN,...,5,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
# affichage du nom des colonnes
for j in range(len(bt_14_19.columns)) :
    print(bt_14_19.columns[j], bt_14_19.dtypes[bt_14_19.columns[j]])

Région object
Département object
Unnamed: 2 object
Code INSEE object
Commune object
Population municipale 2017 int64
Agglomération d'appartenance object
taux de tension moyen sur 3 ans_agglo object
EPCI d'appartenance object
taux de tension moyen sur 3 ans_epci object
1ère année de soumission de la commune object
Taux de logements sociaux (LLS) à l’inventaire de la 1ère année de soumission object
Taux de logements sociaux (LLS) au 01/01/2016 object
Taux de logements sociaux (LLS) au 01/01/2019 object
Nombre de LLS au 01/01/2019 float64
Nombre de RP en 2019 int64
Taux légal à atteindre d'ici 2025 (en %) - décret 2017 object
Taux légal à atteindre d'ici 2025 (en %) - décret 2020 object
PC-
logts commencés 2014-2018 
(5 ans) float64
PC – logts commencés moyenne par an entre 2014 et 2018 float64
Période triennale 2017-2019 partiel (oui/non) object
Objectif quantitatif pour la période 2017-2019 object
Etat quantitatif des réalisations  2017-2019 int64
dont report du précédent triennal float

Il manque certaines valeurs de taux de logements sociaux et de réalisation de l'objectif. Cela concerne 82 communes. Comme précedemment, on suppose que l'objectif n'est pas atteint si la valeur est manquante.

In [23]:
temp = [] # variable temporaire pour le stockage des données
a = 0 # variable de vérification
b = 0

for i in range(len(bt_14_19)) : # pour chaque commune
    # liste contient les informations de la commune
    liste = [bt_14_19["Code INSEE"][i].zfill(5)] # le code INSEE de la commune avec 5 caractères
    
    t1 = str(bt_14_19['Taux de logements sociaux (LLS) au 01/01/2016'][i]).replace(',', '.')[:-1] # le taux de logements sociaux
    t2 = str(bt_14_19["taux de réalisation"][i]).replace(',', '.')[:-1] # le taux de réalisation de l'objectif
    
    if t1 == 'na' or t1 == 'N' or t2[0] == 'N' :
        b = b+1
        if t2[0] != 'N' : # si t2 n'est pas une valeur manquante (donc t1 manquante)
            if (float(t2)/100 >= 1) :
                liste.append(False) # carence
                liste.append("Soumise objectif triennal 2014-2016 atteint") # situation
        elif t1 != 'na' and t1 != 'N' : # si t1 n'est pas une valeur manquante (donc t2 manquante)
            if float(t1)/100 > 0.25 :
                liste.append(False) # carence
                liste.append("Soumise taux atteint en 2016") # situation
            elif bt_14_19['Carence (oui=1\xa0; non=0) '][i] == '0' :
                liste.append(False) # carence
                liste.append("Soumise objectif triennal 2014-2016 non atteint sans amende") # situation
            else :
                a = a + 1
        elif bt_14_19['Carence (oui=1\xa0; non=0) '][i] == '0' :
            liste.append(False) # carence
            liste.append(np.nan) # situation
        else :
            liste.append(np.nan) # carence
            liste.append(np.nan) # situation
        
        
    elif float(t1)/100 > 0.25 :
        liste.append(False) # carence
        liste.append("Soumise taux atteint en 2016") # situation
    elif (float(t2)/100 >= 1) :
        liste.append(False) # carence
        liste.append("Soumise objectif triennal 2014-2016 atteint") # situation
    elif bt_14_19['Carence (oui=1\xa0; non=0) '][i] == '0' :
        liste.append(False) # carence
        liste.append("Soumise objectif triennal 2014-2016 non atteint sans amende") # situation
    elif bt_14_19['Carence (oui=1\xa0; non=0) '][i] == '1' :
        liste.append(True) # carence
        liste.append("Soumise objectif triennal 2014-2016 non atteint avec amende") # situation
    else :
        #print(bt_14_19.loc[i])
        a = a + 1
    
    # nombre de constructions triennal
    liste.append(bt_14_19["réalisation"][i])
    
    # objectif de construction triennal
    liste.append(bt_14_19["objectifs\nQuantitatifs"][i])
    
    temp.append(liste) # les données de la communes sont mises dans la variables temp
    #print(liste)

# print(a)
# création du DataFrame du bilan triennal 2014-2016
df_14_16 = pd.DataFrame(temp, columns = ['code_commune', 'carence_14_16', 'situation_14_16', 'construction_14_16', 'objectif_14_16'])
df_14_16.head()

,code_commune,carence_14_16,situation_14_16,construction_14_16,objectif_14_16
0,01043,True,Soumise objectif triennal 2014-2016 non attein...,18,84
1,01142,True,Soumise objectif triennal 2014-2016 non attein...,32,53
2,01262,False,Soumise objectif triennal 2014-2016 non attein...,NC,NC
3,01281,False,Soumise objectif triennal 2014-2016 atteint,135,18
4,01313,False,Soumise objectif triennal 2014-2016 atteint,64,52


In [24]:
df_14_16.replace(['NC', 'NR'], '', inplace = True)
df_14_16.replace(['NR-fusion', 'NC fusion'], '', inplace = True)
df_14_16.head()

,code_commune,carence_14_16,situation_14_16,construction_14_16,objectif_14_16
0,01043,True,Soumise objectif triennal 2014-2016 non attein...,18,84
1,01142,True,Soumise objectif triennal 2014-2016 non attein...,32,53
2,01262,False,Soumise objectif triennal 2014-2016 non attein...,,
3,01281,False,Soumise objectif triennal 2014-2016 atteint,135,18
4,01313,False,Soumise objectif triennal 2014-2016 atteint,64,52


## 6. Bilan triennal 2017 -- 2019

Il y a 8 communes pour lesquelles il manque la valeur du taux de logements sociaux. Il s'agit de :
- 76103 Bonsecours
- 76108 Bois-Guillaume
- 76116 Boos
- 76429 Le Mesnil-Esnard
- 76475 Franqueville-Saint-Pierre
- 76481 Octeville-sur-Mer
- 76552 Sainte-Adresse
- 2A004 Ajaccio\
On suppose que l'objectif seuil de 0,25 n'est pas atteint lorsque la valeur est manquante.

In [25]:
temp = [] # variable temporaire pour stocker les données
a = 0 # variable de vérification

for i in range(len(bt_14_19)) : # pour chaque commune
    # liste contient les informations de la commune
    liste = [bt_14_19["Code INSEE"][i].zfill(5)] # le code de la commune avec 5 caractères
    
    t1 = str(bt_14_19['Taux de logements sociaux (LLS) au 01/01/2019'][i]).replace(',', '.')[:-1] # le taux de logements sociaux
    t2 = str(bt_14_19["Taux de réalisation de l'objectif quantitatif 2017-2019"][i]).replace(',', '.')[:-1] # taux de réalisation de l'objectif
    
    if t1 == 'na' : # si la valeur du taux de logements sociaux est manquante
        # print(bt_14_19["Code INSEE"][i], bt_14_19.Commune[i])
        if (float(t2)/100 >= 1) :
            liste.append(False) # carence
            liste.append("Soumise objectif triennal 2017-2019 atteint") # situation
        else :
            liste.append(False) # carence
            liste.append("Soumise objectif triennal 2017-2019 non atteint sans amende") # situation
        
        
    elif float(t1)/100 > 0.25 :
        liste.append(False) # carence
        liste.append("Soumise taux atteint en 2019") # situation
    elif (float(t2)/100 >= 1) :
        liste.append(False) # carence
        liste.append("Soumise objectif triennal 2017-2019 atteint") # situation
    elif bt_14_19['Carence envisagée (1 si oui, 0 si non)'][i] == 0 :
        liste.append(False) # carence
        liste.append("Soumise objectif triennal 2017-2019 non atteint sans amende") # situation
    elif bt_14_19['Carence envisagée (1 si oui, 0 si non)'][i] == 1 :
        liste.append(True) # carence
        liste.append("Soumise objectif triennal 2017-2019 non atteint avec amende") # situation
    else :
        #print(bt_14_19.loc[i])
        a = a + 1
        
    # nombre de constructions de logements sociaux durant la période 2017-2019
    liste.append(int(bt_14_19["Etat quantitatif des réalisations  2017-2019"][i]))
    
    # objectif de construction de logements sociaux durant la période 2017-2019
    liste.append(float(bt_14_19["Objectif quantitatif pour la période 2017-2019"][i].replace(',', '.')))
        
    temp.append(liste) # les informations de la communes sont ajoutées à temp
    #print(liste)

#print(a)
# création du DataFrame du bilan triennal 2017-2019
df_17_19 = pd.DataFrame(temp, columns = ['code_commune', 'carence_17_19', 'situation_17_19', 'construction_17_19', 'objectif_17_19'])
df_17_19.head()

,code_commune,carence_17_19,situation_17_19,construction_17_19,objectif_17_19
0,01043,False,Soumise objectif triennal 2017-2019 atteint,107,101.0
1,01142,True,Soumise objectif triennal 2017-2019 non attein...,46,68.0
2,01262,False,Soumise objectif triennal 2017-2019 atteint,41,5.0
3,01281,False,Soumise objectif triennal 2017-2019 atteint,105,1.0
4,01313,False,Soumise objectif triennal 2017-2019 atteint,103,64.0


# II. Bilans annuels
## 1. Bilan 2004

In [26]:
# chargement des données du bilan 2004 
b_04 = pd.read_csv("../donnees_initiales/bilans_annuels/ART_55_Bilan_SRU_2004_donnees2020.csv",
                   sep = ';')
b_04.head()

,Code,Code Agglo,Dép,Commune,POP99,NB_RP,pop < seuil,Situation de la commune par rapport au L.302-5 du CCH\n,decroiss,DSU et +15%,...,Dépenses déductibles EPCI,Dépenses déductibles,Dépenses indûment déduites,Reliquat disponible,Bénéficiaires locaux,Prélèvement net hors majoration à destination des bénéfici,Majoration net à destination du Fonds National,Prélèvement net total,Reliquat reportable,Obs
0,1030,553,1,Beauregard,817,324,1,Non soumise sous seuil pop,0,0,...,NC,NC,NC,NC,NC,NC,NC,NC,NC,NaN
1,1043,757,1,Beynost,3530,1403,0,Soumise prélèvement nul,0,0,...,NC,0,NR,"126472,77",NR,0,NC,0,0,NaN
2,1049,757,1,La Boisse,2709,986,1,Non soumise sous seuil pop,0,0,...,NC,NC,NC,NC,NC,NC,NC,NC,NC,NaN
3,1053,1501,1,Bourg-en-Bresse,40666,19443,0,Non soumise atteint taux légal,0,0,...,NC,NC,NC,NC,NC,NC,NC,NC,NC,NaN
4,1142,757,1,Dagneux,3757,1339,0,Soumise prélèvement nul,0,0,...,NC,0,NR,"28989,45",NR,0,NC,0,0,NaN


In [27]:
b_04.dtypes # noms et types des colonnes

Code                                                           object
Code Agglo                                                     object
Dép                                                            object
Commune                                                        object
POP99                                                           int64
NB_RP                                                           int64
pop < seuil                                                     int64
Situation de la commune par rapport au L.302-5 du CCH\n        object
decroiss                                                        int64
DSU et +15%                                                     int64
Serv/PEB                                                        int64
Plus de 20%                                                     int64
Total LLS                                                     float64
Taux LLS                                                       object
5/100 dép f°        

In [28]:
b_04['Situation de la commune par rapport au L.302-5 du CCH\n'].unique()

array(['Non soumise sous seuil pop', 'Soumise prélèvement nul',
       'Non soumise atteint taux légal', 'Soumise prélevée',
       'Soumise non prélevée', 'Non soumise exemptée', 'Soumise exonérée',
       'Soumise'], dtype=object)

In [29]:
data = [] # variable temporaire pour stocker les données

for i in range(len(b_04)) : # pour chaque commune
    data.append(b_04['Code'][i].zfill(5)) # code de la commune
    data.append(b_04['Code Agglo'][i].zfill(5)) # code de l'agglomération
    if float(b_04['Taux LLS'][i].replace(',', '.')[:-1])/100 > 0 :
        data.append(float(b_04['Taux LLS'][i].replace(',', '.')[:-1])/100) # taux de logements sociaux
    elif b_04['Situation de la commune par rapport au L.302-5 du CCH\n'][i] == 'Non soumise sous seuil pop' :
        data.append(float(b_04['Taux LLS'][i].replace(',', '.')[:-1])/100) # taux de logements sociaux
    else :
        data.append(np.nan) # taux de logements sociaux
    data.append(b_04['Situation de la commune par rapport au L.302-5 du CCH\n'][i]) # situation
    data.append(b_04["Total LLS"][i]) # nombre de logements sociaux
    
data = np.array(data) # on change le type de data
colonnes = ['code_commune', 'UU_99', 'taux_ls_04', 'situation_04', 'LLS_04'] # noms des colonnes
data = np.reshape(data, (len(b_04), len(colonnes))) # changer ses dimensions (8804 -> 2201*5)
df_04 = pd.DataFrame(data, columns = colonnes) # création du DataFrame avec les données de 2004
df_04.taux_ls_04 = pd.to_numeric(df_04.taux_ls_04, errors = 'coerce')
df_04.head()

,code_commune,UU_99,taux_ls_04,situation_04,LLS_04
0,01030,00553,0.0000,Non soumise sous seuil pop,0.0
1,01043,00757,0.0128,Soumise prélèvement nul,18.0
2,01049,00757,0.0000,Non soumise sous seuil pop,nan
3,01053,01501,NaN,Non soumise atteint taux légal,0.0
4,01142,00757,0.0934,Soumise prélèvement nul,125.0


In [30]:
# vérification valeurs aberrantes des taux de logements sociaux
b_04[df_04.taux_ls_04 > 0.7].iloc[:,:14]

,Code,Code Agglo,Dép,Commune,POP99,NB_RP,pop < seuil,Situation de la commune par rapport au L.302-5 du CCH\n,decroiss,DSU et +15%,Serv/PEB,Plus de 20%,Total LLS,Taux LLS
1152,62194,752,62,Calonne-Ricouart,5989,2280,0,Non soumise atteint taux légal,0,0,0,1,2460.0,"107,89%"
1187,62386,755,62,Grenay,6395,2337,0,Non soumise atteint taux légal,0,0,0,1,1827.0,"78,18%"
1214,62555,752,62,Marles-les-Mines,6088,2553,0,Non soumise atteint taux légal,0,0,0,1,1837.0,"71,95%"
1242,62771,755,62,Sallaumines,10677,3810,0,Non soumise atteint taux légal,0,0,0,1,2764.0,"72,55%"
1784,78621,851,78,Trappes,28812,10061,0,Non soumise atteint taux légal,0,0,0,1,7308.0,"72,64%"
2044,93030,851,93,Dugny,8641,3689,0,Non soumise atteint taux légal,0,0,0,1,2646.0,"71,73%"
2072,93079,851,93,Villetaneuse,11376,3848,0,Non soumise atteint taux légal,0,0,0,1,2761.0,"71,75%"


Les communes dans le tableau précédents ont des valeurs qui peuvent sembler abérrantes comme nombre de logements sociaux, notamment pour la commune 62194 qui aurait plus de logements sociaux que de résidences principales. Les communes 62386, 62555 et 62771 ont également des valeurs très élevées qui ne sont pas cohérentes avec les chiffres de l'INSEE.\
Pour les communes 78621, 93030 et 93079, leur nombre de logements sociaux semble cohérant.

## 2. Bilan 2005

In [31]:
# chargement des données du bilan de 2005
b_05 = pd.read_csv("../donnees_initiales/bilans_annuels/ART_55_Bilan_SRU_2005_donnees2020.csv",
                   sep = ';')
b_05.head()

,Code,Dép,Commune,POP99,Nb RP,pop < seuil,Situation de la commune par rapport au L.302-5 du CCH,décrois Démo,DSU et > 15%,SERV ou PEB,...,Dépenses déductibles EPCI,Dépenses déductibles,Dépenses indûment déduites,Reliquat disponible,Bénéficiaires locaux,Prélèvement net hors majoration à destination des bénéficiaires locaux,Majoration net à destination du Fonds National,Prélèvement net total,Reliquat reportable,Obs
0,01030,01,Beauregard,817,355,1,Non soumise sous seuil pop,0,0,0,...,NC,NC,NC,NC,NC,NC,NC,NC,NC,NaN
1,01043,01,Beynost,3530,1512,0,Soumise prélevée,0,0,0,...,NC,"0,00","0,00","0,00",EPF,"64 125,00",NC,"64 125,00","0,00",NaN
2,01049,01,La Boisse,2709,996,1,Non soumise sous seuil pop,0,0,0,...,NC,NC,NC,NC,NC,NC,NC,NC,NC,NaN
3,01053,01,Bourg-en-Bresse,40666,19165,0,Non soumise atteint taux légal,0,0,0,...,NC,NC,NC,NC,NC,NC,NC,NC,NC,NaN
4,01142,01,Dagneux,3757,1370,0,Soumise prélevée,0,0,0,...,NC,"0,00","0,00","0,00",EPF,"22 715,00",NC,"22 715,00","0,00",NaN


In [32]:
b_05.dtypes # noms et types des colonnes

Code                                                                      object
Dép                                                                       object
Commune                                                                   object
POP99                                                                      int64
Nb RP                                                                      int64
pop < seuil                                                                int64
Situation de la commune par rapport au L.302-5 du CCH                     object
décrois Démo                                                               int64
DSU et > 15%                                                               int64
SERV ou PEB                                                                int64
> 20%                                                                      int64
TOTAL                                                                      int64
Taux LLS                    

In [33]:
b_05['Situation de la commune par rapport au L.302-5 du CCH'].unique()

array(['Non soumise sous seuil pop', 'Soumise prélevée',
       'Non soumise atteint taux légal', 'Soumise non prélevée',
       'Non soumise exemptée', 'Soumise prélèvement nul',
       'Soumise exonérée', 'Soumise'], dtype=object)

In [34]:
data = [] # variable temporaire pour stocker les données

for i in range(len(b_05)) : # pour chaque commune
    data.append(b_05['Code'][i]) # code de la commune
    if float(b_05['Taux LLS'][i].replace(',', '.')[:-1]) > 0 or b_05['Situation de la commune par rapport au L.302-5 du CCH'][i] == 'Non soumise sous seuil pop' :
        data.append(float(b_05['Taux LLS'][i].replace(',', '.')[:-1])/100) # taux de logements sociaux
    else :
        data.append(np.nan) # tau de logements sociaux
    data.append(b_05['Situation de la commune par rapport au L.302-5 du CCH'][i]) # situation
    data.append(b_05["TOTAL"][i]) # nombre de logements sociaux
    
data = np.array(data) 
colonnes = ['code_commune', 'taux_ls_05', 'situation_05', 'LLS_05']
data = np.reshape(data, (len(b_05), len(colonnes)))
df_05 = pd.DataFrame(data, columns = colonnes) # création du DataFrame des données du bilan 2005
df_05.taux_ls_05 = pd.to_numeric(df_05.taux_ls_05, errors = 'coerce')
df_05.head()

,code_commune,taux_ls_05,situation_05,LLS_05
0,01030,0.0000,Non soumise sous seuil pop,0
1,01043,0.0470,Soumise prélevée,71
2,01049,0.0000,Non soumise sous seuil pop,0
3,01053,NaN,Non soumise atteint taux légal,0
4,01142,0.0912,Soumise prélevée,125


In [35]:
# vérification des valeurs abérrantes
df_05[df_05.taux_ls_05 > 0.7]

,code_commune,taux_ls_05,situation_05,LLS_05
1783,78621,0.7176,Non soumise atteint taux légal,7208


Il ne semble pas y avoir de nombre de logements soicaux surévalué dans le bilan de 2005.

## 3. Bilan 2006

In [36]:
# Chargement des données du bilan de 2006
b_06 = pd.read_csv("../donnees_initiales/bilans_annuels/ART_55_Bilan_SRU_2006_donnees2020.csv",
                   sep = ';')
b_06.head()

,Code,Commune,POP99,pop < seuil,Nb RP,Situation de la commune par rapport au L.302-5 du CCH\n,Exod,DSU et > 15%,Exor,> 20%,...,Dépenses déductibles EPCI,Dépenses déductibles,Dépenses indûment déduites,Reliquat disponible,Bénéficiaires locaux,Prélèvement net hors majoration à destination des bénéficiaires locaux,Majoration net à destination du Fonds National,Prélèvement net total,Reliquat reportable,Observation
0,01030,Beauregard,817,1,362,Non soumise sous seuil de pop,0,0,0,0,...,NC,NC,NC,NC,NC,NC,NC,NC,NC,NaN
1,01043,Beynost,3530,0,1568,Soumise prélevée,0,0,0,0,...,NC,"0,00",NR,"0,00",EPF,"70 671,00","0,00","70 671,00","0,00",NaN
2,01049,La Boisse,2709,1,1014,Non soumise sous seuil de pop,0,0,0,0,...,NC,NC,NC,NC,NC,NC,NC,NC,NC,NaN
3,01053,Bourg-en-Bresse,40666,0,19064,Non soumise atteint taux légal,0,0,0,2,...,NC,NC,NC,NC,NC,NC,NC,NC,NC,NaN
4,01142,Dagneux,3757,0,1389,Soumise prélèvement nul,0,0,0,0,...,NC,"30 489,00",NR,"0,00",Fond national/FRAFU,"0,00","0,00","0,00","6 859,00",NaN


In [37]:
b_06.dtypes # noms et types des colonnes

Code                                                                      object
Commune                                                                   object
POP99                                                                      int64
pop < seuil                                                                int64
Nb RP                                                                      int64
Situation de la commune par rapport au L.302-5 du CCH\n                   object
Exod                                                                       int64
DSU et > 15%                                                               int64
Exor                                                                       int64
> 20%                                                                      int64
TOTAL                                                                      int64
taux LLS                                                                  object
5/100 f°                    

In [38]:
b_06['Situation de la commune par rapport au L.302-5 du CCH\n'].unique()

array(['Non soumise sous seuil de pop', 'Soumise prélevée',
       'Non soumise atteint taux légal', 'Soumise prélèvement nul',
       'Non soumise exemptée', 'Soumise non prélevée', 'Soumise exonérée',
       'Soumise'], dtype=object)

Certaines situations ne sont pas écrites exactement comme les années précédentes, on va donc faire des changements.

In [39]:
b_06.replace(to_replace = 'Non soumise sous seuil de pop', value = 'Non soumise sous seuil pop', inplace = True)

In [40]:
data = [] # variable temporaire pour stocker les données

for i in range(len(b_06)) : # pour chaque commune
    data.append(b_06['Code'][i]) # code de la commune
    if float(b_06['taux LLS'][i].replace(',', '.')[:-1]) > 0 or b_06['Situation de la commune par rapport au L.302-5 du CCH\n'][i] == 'Non soumise sous seuil pop' :
        data.append(float(b_06['taux LLS'][i].replace(',', '.')[:-1])/100) # taux de logements sociaux
    else :
        data.append(np.nan)
    data.append(b_06['Situation de la commune par rapport au L.302-5 du CCH\n'][i]) # situation
    data.append(b_06['TOTAL'][i]) # nombre de logements sociaux
    
data = np.array(data)
colonnes = ['code_commune', 'taux_ls_06', 'situation_06', 'LLS_06']
data = np.reshape(data, (len(b_06), len(colonnes)))
df_06 = pd.DataFrame(data, columns = colonnes) # création du DataFrame des données de 2006
df_06.taux_ls_06 = pd.to_numeric(df_06.taux_ls_06, errors = 'coerce')
df_06.head()

,code_commune,taux_ls_06,situation_06,LLS_06
0,01030,0.0000,Non soumise sous seuil pop,0
1,01043,0.0453,Soumise prélevée,71
2,01049,0.0000,Non soumise sous seuil pop,0
3,01053,0.3839,Non soumise atteint taux légal,7318
4,01142,0.0878,Soumise prélèvement nul,122


In [41]:
df_06[df_06.taux_ls_06 > 0.7]

,code_commune,taux_ls_06,situation_06,LLS_06
830,57058,0.8393,Non soumise atteint taux légal,2769
1187,62386,0.7826,Non soumise atteint taux légal,1897
1214,62555,0.7098,Non soumise atteint taux légal,1795
1242,62771,0.7258,Non soumise atteint taux légal,2793
1784,78621,0.7116,Non soumise atteint taux légal,7208
2044,93030,0.7110,Non soumise atteint taux légal,2647


## 4. Bilan 2007

Le fichiers des données du bilan de 2007 est nettement plus petit que les fichiers des autres années. Il ne contient qu'une seule feuille sur laquelle il n'y a les données que des communes soumises.

In [42]:
# Chargement des données de 2007
b_07 = pd.read_csv("../donnees_initiales/bilans_annuels/ART_55_Bilan_SRU_2007.csv",
                   sep = ';')
b_07.head()

,Unnamed: 0,Code INSEE,Commune,POP99,LLS 2006,RP 2006,Taux LLS 2006,Situation de la commune,DSU et > 15%,Commune carencée,Date arrêté de carence,Taux de majoration,Prélèvement brut,Majoration,Total prélèvement brut avant plafonnement,Dépenses déductibles 2005,Reliquat disponible (repris dans bilan annuel 2006),Prélèvement net 2007,Affectation du prélèvement,Reliquat reportable
0,1043,01043,Beynost,3 530,71,1 561,"4,55%",Soumise prélevée,non,non,NaN,NaN,"75 261,41","0,00","75 261,41",0,"0,00","75 261,41",FAU,"0,00"
1,1142,01142,Dagneux,3 757,138,1 427,"9,67%",Soumise prélevée,non,non,NaN,NaN,"26 079,27","0,00","26 079,27",0,"6 859,00","19 219,73",FAU,"0,00"
2,1313,01313,Prévessin-Moëns,4 261,203,1 842,"11,02%",Soumise prélevée,non,non,NaN,NaN,"18 559,20","0,00","18 559,20",0,"0,00","18 558,87",EPCI,"0,00"
3,1322,01322,Reyrieux,3 683,78,1 330,"5,86%",Soumise prélevée,non,non,NaN,NaN,"44 687,60","0,00","44 687,60",0,"0,00","44 687,60",EPF,"0,00"
4,1344,01344,Saint-Denis-lès-Bourg,4 921,92,2 024,"4,55%",Soumise prélèvement nul,non,non,NaN,NaN,"41 726,88","0,00","41 726,88",78 816,"304,00","0,00",EPCI,"37 394,62"


In [43]:
b_07.dtypes # noms et types des colonnes

Unnamed: 0                                             object
Code INSEE                                             object
Commune                                                object
POP99                                                  object
LLS 2006                                               object
RP 2006                                                object
Taux LLS 2006                                          object
Situation de la commune                                object
DSU et > 15%                                           object
Commune carencée                                       object
Date arrêté de carence                                 object
Taux de majoration                                     object
Prélèvement brut                                       object
Majoration                                             object
Total prélèvement brut avant plafonnement              object
Dépenses déductibles 2005                              object
Reliquat

In [44]:
b_07['Situation de la commune'].unique()

array(['Soumise prélevée', 'Soumise prélèvement nul', 'Soumise exonérée',
       'Soumise non prélevée', 'Soumise', 'soumise exonérée'],
      dtype=object)

On change 'soumise exonérée' pour mettre une majuscule comme les autres.

In [45]:
b_07.replace('soumise exonérée', 'Soumise exonérée', inplace = True)

In [46]:
data = [] # variable temporaire pour stocker les données

for i in range(len(b_07)) : # pour chaque commune
    data.append(b_07['Code INSEE'][i]) # code de la commune
    if float(b_07['Taux LLS 2006'][i].replace(',', '.')[:-1]) > 0 :
        data.append(float(b_07['Taux LLS 2006'][i].replace(',', '.')[:-1])/100) # taux de logements sociaux
            # (on remplace ',' par '.', on enlève le signe '%' et on divise par 100)
    else :
        data.append(np.nan)
    data.append(b_07['Situation de la commune'][i]) # situation
    data.append(int(b_07["LLS 2006"][i].replace('\xa0',''))) # nombre de logements sociaux
    
data = np.array(data)
colonnes = ['code_commune', 'taux_ls_07', 'situation_07', 'LLS_07']
data = np.reshape(data, (len(b_07), len(colonnes)))
df_07 = pd.DataFrame(data, columns = colonnes) # création du DataFrame du bilan 2007
df_07.taux_ls_07 = pd.to_numeric(df_07.taux_ls_07, errors = 'coerce')
df_07.head()

,code_commune,taux_ls_07,situation_07,LLS_07
0,01043,0.0455,Soumise prélevée,71
1,01142,0.0967,Soumise prélevée,138
2,01313,0.1102,Soumise prélevée,203
3,01322,0.0586,Soumise prélevée,78
4,01344,0.0455,Soumise prélèvement nul,92


In [47]:
# vérification valeurs aberrantes
df_07[df_07.taux_ls_07 > .2]

,code_commune,taux_ls_07,situation_07,LLS_07


Dans le bilan de 2007, il n'y a pas les communes non soumises donc il n'y a pas les communes avec des grand taux de logements sociaux qui atteignent le seuil de 20% donc il n'y a pas de valeurs abérrantes en nombre de logements sociaux.

## 5. Bilan 2008

In [48]:
# chargement des données du bilan de 2008
b_08 = pd.read_csv("../donnees_initiales/bilans_annuels/ART_55_Bilan_SRU_2008_donnees2020.csv",
                   sep = ';')
b_08.head()

,Dép.,Code,Commune,POP99,pop < seuil,Situation de la commune par rapport au L.302-5 du CCH\n,décrois. Démo.,DSU et > 15%,SERV. ou PEB,> 20%,...,Date de l'A.P.,Taux MAJ,Dép. déduc.,Prélèv. arrêté,Reliquat N,Reliquat N+1,Dest. .prélèv.,Dép. agglo.,Vu,Obs.
0,01,01030,Beauregard,817,1.0,Non soumise sous seuil pop,0,0,0,0,...,NaN,0,0,0,0,0,0.0,69,1.0,NaN
1,01,01043,Beynost,3 530,0.0,Soumise prélevée,0,0,0,0,...,NaN,"0,00","0,00","67 298,62","0,00","0,00",3.0,69,1.0,NaN
2,01,01049,La Boisse,2 709,1.0,Non soumise sous seuil pop,0,0,0,0,...,NaN,0,0,0,0,0,0.0,69,1.0,NaN
3,01,01053,Bourg-en-Bresse,40 666,0.0,Non soumise atteint taux légal,0,1,0,2,...,NaN,0,0,0,0,0,0.0,1,1.0,NaN
4,01,01142,Dagneux,3 757,0.0,Soumise prélevée,0,0,0,0,...,NaN,"0,00","0,00","27 539,40","0,00","0,00",3.0,69,1.0,NaN


In [49]:
b_08.dtypes # noms et types des données des colonnes

Dép.                                                        object
Code                                                        object
Commune                                                     object
POP99                                                       object
pop < seuil                                                float64
Situation de la commune par rapport au L.302-5 du CCH\n     object
décrois. Démo.                                               int64
DSU et > 15%                                                 int64
SERV. ou PEB                                                 int64
> 20%                                                        int64
TOTAL                                                       object
Nb RP                                                       object
Taux LS 2007                                                object
Catég. 1                                                    object
Catég. 2                                                    ob

In [50]:
b_08['Situation de la commune par rapport au L.302-5 du CCH\n'].unique()

array(['Non soumise sous seuil pop', 'Soumise prélevée',
       'Non soumise atteint taux légal', 'Soumise prélèvement nul',
       'Soumise exonérée', 'Non soumise exemptée', 'Soumise non prélevée',
       'Soumise'], dtype=object)

In [51]:
data = [] # variable temporaire pour stocker les données

for i in range(len(b_08)) : # pour chaque commune
    data.append(b_08['Code'][i]) # code de la commune
    if float(b_08['Taux LS 2007'][i].replace(',', '.')[:-1]) > 0 or b_08['Situation de la commune par rapport au L.302-5 du CCH\n'][i] == 'Non soumise sous seuil pop' :
        data.append(float(b_08['Taux LS 2007'][i].replace(',', '.')[:-1])/100) # taux de logements sociaux
    else :
        data.append(np.nan) # taux de logements sociaux
    data.append(b_08['Situation de la commune par rapport au L.302-5 du CCH\n'][i]) # situation
    data.append(float(str(b_08["TOTAL"][i]).replace('\xa0', ''))) # nombre de logements sociaux
    
data = np.array(data)
colonnes = ['code_commune', 'taux_ls_08', 'situation_08', 'LLS_08']
data = np.reshape(data, (len(b_08), len(colonnes)))
df_08 = pd.DataFrame(data, columns = colonnes) # création du DataFrame du bilan 2008
df_08.taux_ls_08 = pd.to_numeric(df_08.taux_ls_08, errors = 'coerce')
df_08.head()

,code_commune,taux_ls_08,situation_08,LLS_08
0,01030,0.000,Non soumise sous seuil pop,0.0
1,01043,0.045,Soumise prélevée,71.0
2,01049,0.000,Non soumise sous seuil pop,0.0
3,01053,0.383,Non soumise atteint taux légal,7295.0
4,01142,0.096,Soumise prélevée,138.0


In [52]:
# vérification valeurs abérrantes
df_08[df_08.taux_ls_08 > 0.7]

,code_commune,taux_ls_08,situation_08,LLS_08
417,27701,0.797,Non soumise atteint taux légal,3167.0
1364,62386,0.805,Non soumise atteint taux légal,2016.0
1391,62555,0.716,Non soumise atteint taux légal,1794.0
1420,62771,0.735,Non soumise atteint taux légal,2853.0
2424,97120,0.705,Non soumise atteint taux légal,6188.0


## 6. Bilan 2009

In [53]:
# chargement des données du bilan de 2009
b_09 = pd.read_csv("../donnees_initiales/bilans_annuels/ART_55_Bilan_SRU_2009_toutes_communes.csv",
                   sep = ';')
b_09.head()

,Code,Code Agglo,Dép.,Commune,POP2006,agglo/epci,pop < seuil,Situation de la commune par rapport au L.302-5 du CCH\n,décrois. Démo.,DSU et > 15%,...,Date de l'A.P.,Taux MAJ,Dép. déduc.,Prélèv. arrêté,Reliquat N,Reliquat N+1,Dest. .prélèv.,Vu,Obs.,Unnamed: 37
0,1030,553,01,Beauregard,817,agglo,1,Non soumise sous seuil pop,0,0,...,NaN,0,0,0,0,0,0,1,NaN,1.0
1,1043,757,01,Beynost,3530,agglo,0,Soumise prélevée,0,0,...,NaN,0,"0,00","71 366,96","0,00","0,00",3,1,NaN,1.0
2,1049,757,01,La Boisse,2709,agglo,1,Non soumise sous seuil pop,0,0,...,NaN,0,0,0,0,0,0,1,NaN,1.0
3,1053,1501,01,Bourg-en-Bresse,40666,agglo,0,Non soumise atteinte taux légal,0,0,...,NaN,0,0,0,0,0,0,1,NaN,NaN
4,1142,757,01,Dagneux,3757,agglo,0,Soumise prélevée,0,0,...,NaN,0,"0,00","30 429,42","0,00","0,00",3,1,NaN,1.0


In [54]:
b_09.dtypes # noms et types des colonnes

Code                                                        object
Code Agglo                                                  object
Dép.                                                        object
Commune                                                     object
POP2006                                                      int64
agglo/epci                                                  object
pop < seuil                                                  int64
Situation de la commune par rapport au L.302-5 du CCH\n     object
décrois. Démo.                                               int64
DSU et > 15%                                                 int64
SERV. ou PEB                                                 int64
> 20%                                                        int64
LS 2008                                                      int64
RP 2008                                                      int64
Taux LS 2008                                                ob

In [55]:
b_09['Situation de la commune par rapport au L.302-5 du CCH\n'].unique()

array(['Non soumise sous seuil pop', 'Soumise prélevée',
       'Non soumise atteinte taux légal', 'Soumise prélèvement nul',
       'Non soumise exemptée', 'Soumise exonérée', 'Soumise non prélevée',
       'Soumise'], dtype=object)

Pour avoir les mêmes situations que les autres années, on enlève le "e" de "Non soumise atteinte taux légal".

In [56]:
b_09.replace('Non soumise atteinte taux légal', 'Non soumise atteint taux légal', inplace = True)

In [57]:
# affichage des différentes valeurs dans la variable agglo/epci indiquant dans quelle structure est la commune
b_09['agglo/epci'].unique()

array(['agglo', 'epci'], dtype=object)

In [58]:
data = [] # variable temporaire pour stocker les informations

for i in range(len(b_09)) : # pour chaque commune
    
    data.append(b_09['Code'][i].zfill(5)) # code de la commune
    
    if b_09['agglo/epci'][i] == 'agglo' :
        data.append(b_09['Code Agglo'][i].zfill(5)) # code unité urbaine
        data.append('NA') # code EPCI
    elif b_09['agglo/epci'][i] == 'epci' :
        data.append('NA') # code de l'unité urbaine
        data.append(str(b_09['Code Agglo'][i])) # code de l'EPCI
        
    if float(b_09['Taux LS 2008'][i].replace(',', '.').replace('%', '')) > 0 or b_09['Situation de la commune par rapport au L.302-5 du CCH\n'][i] == 'Non soumise sous seuil pop' :
        data.append(float(b_09['Taux LS 2008'][i].replace(',', '.').replace('%', ''))/100) # taux de logements sociaux
    else :
        data.append(np.nan) # taux de logements sociaux
        
    data.append(b_09['Situation de la commune par rapport au L.302-5 du CCH\n'][i]) # situation
    
    data.append(b_09['LS 2008'][i]) # nombre de logements sociaux
    
data = np.array(data)
colonnes = ['code_commune', 'code_agglo_09', 'code_epci_09', 'taux_ls_09', 'situation_09', 'LLS_09']
data = np.reshape(data, (len(b_09), len(colonnes)))
df_09 = pd.DataFrame(data, columns = colonnes) # création du DataFrame du bilan 2009
df_09.taux_ls_09 = pd.to_numeric(df_09.taux_ls_09, errors = 'coerce')
df_09.head()

,code_commune,code_agglo_09,code_epci_09,taux_ls_09,situation_09,LLS_09
0,01030,00553,NA,0.0000,Non soumise sous seuil pop,0
1,01043,00757,NA,0.0444,Soumise prélevée,71
2,01049,00757,NA,0.0000,Non soumise sous seuil pop,0
3,01053,01501,NA,0.3886,Non soumise atteint taux légal,7322
4,01142,00757,NA,0.0928,Soumise prélevée,138


In [59]:
# vérification des valeurs abérrantes
df_09[df_09.taux_ls_09 > 0.7]

,code_commune,code_agglo_09,code_epci_09,taux_ls_09,situation_09,LLS_09
1364,62386,00755,NA,0.8100,Non soumise atteint taux légal,2051
1391,62555,00752,NA,0.7034,Non soumise atteint taux légal,1755
1401,62628,00755,NA,0.7233,Non soumise atteint taux légal,2185
1420,62771,00755,NA,0.7676,Non soumise atteint taux légal,2999
2001,78621,00851,NA,0.7040,Non soumise atteint taux légal,7208


## 7. Bilan 2010

In [60]:
# chargement des données du bilan de 2010
b_10 = pd.read_csv("../donnees_initiales/bilans_annuels/ART_55_Bilan_SRU_2010_toutes_communes.csv",
                   sep = ';')
b_10.head()

,Dép,Code,Code Appart,Commune,Structure d'application,POP06,pop < seuil,Situation de la commune par rapport au L.302-5 du CCH\n,décrois Démo,DSU et > 15%,...,Taux MAJ,Reliquat N,Dép déduc,Dép indues,Prélèv arrêté,Reliquat N+1,Dest prélèv,Dép agglo,Vu,Obs
0,01,1043,00757,Beynost,agglo,4174,0,Soumise prélevée,0,0,...,0,0,0,0,76 029,0,2,69,1,NaN
1,01,1049,00757,La Boisse,agglo,2753,1,Nous soumise sous seuil pop,0,0,...,0,0,0,0,0,0,3,69,1,NaN
2,01,1053,01501,Bourg-en-Bresse,agglo,40156,0,Non soumise atteint taux légal,1,1,...,0,0,0,0,0,0,1,01,1,NaN
3,01,1065,240100628,Buellas,epci,1650,1,Nous soumise sous seuil pop,0,0,...,0,0,0,0,0,0,1,01,1,NaN
4,01,1142,00757,Dagneux,agglo,3906,0,Soumise prélevée,0,0,...,0,0,0,0,28 084,0,3,69,1,NaN


In [61]:
b_09.loc[b_09.Code == '35288']

,Code,Code Agglo,Dép.,Commune,POP2006,agglo/epci,pop < seuil,Situation de la commune par rapport au L.302-5 du CCH\n,décrois. Démo.,DSU et > 15%,...,Date de l'A.P.,Taux MAJ,Dép. déduc.,Prélèv. arrêté,Reliquat N,Reliquat N+1,Dest. .prélèv.,Vu,Obs.,Unnamed: 37
662,35288,35501,35,Saint-Malo,50675,agglo,0,Non soumise atteint taux légal,0,0,...,NaN,0,0,0,0,0,0,1,NaN,NaN


La commune 35288 apparait deux fois dans les données, une fois avec un EPCI et une fois avec une agglomération. Il y a d'autres différences entre les deux lignes, celle que l'on va conserver sera la deuxième, c'est-à-dire celle avec l'EPCI, la colonne "Vu" est à la valeur 1 pour cette ligne et 0 pour l'autre.

In [62]:
b_10.loc[b_10.Code == '35288']

,Dép,Code,Code Appart,Commune,Structure d'application,POP06,pop < seuil,Situation de la commune par rapport au L.302-5 du CCH\n,décrois Démo,DSU et > 15%,...,Taux MAJ,Reliquat N,Dép déduc,Dép indues,Prélèv arrêté,Reliquat N+1,Dest prélèv,Dép agglo,Vu,Obs
1364,35,35288,35501,Saint-Malo,agglo,50675,0,Non soumise atteint taux légal,0,0,...,0,0,0,0,0,0,0,35,0,NaN
1365,35,35288,243500782,Saint-Malo,epci,49661,0,Non soumise atteint taux légal,0,0,...,0,0,0,0,0,0,1,35,1,NaN


In [63]:
b_10.dtypes

Dép                                                        object
Code                                                       object
Code Appart                                                object
Commune                                                    object
Structure d'application                                    object
POP06                                                       int64
pop < seuil                                                 int64
Situation de la commune par rapport au L.302-5 du CCH\n    object
décrois Démo                                                int64
DSU et > 15%                                                int64
SERV ou PEB                                                 int64
LS 2009                                                     int64
RP 2009                                                     int64
Taux LS 2009                                               object
> 20%                                                       int64
Catég 1   

In [64]:
b_10['Situation de la commune par rapport au L.302-5 du CCH\n'].unique()

array(['Soumise prélevée', 'Nous soumise sous seuil pop',
       'Non soumise atteint taux légal', 'Soumise prélèvement nul',
       'Non soumise exemptée', 'Soumise non prélevée', 'Soumise exonérée',
       'Soumise'], dtype=object)

In [65]:
b_10.replace('Nous soumise sous seuil pop', 'Non soumise sous seuil pop', inplace = True)

In [66]:
# affichage valeurs variables Structure d'application
b_10["Structure d'application"].unique()

array(['agglo', 'epci'], dtype=object)

In [67]:
data = [] # varibale temporaire pour stocker les données

for i in range(len(b_10)) : # pour chaque commune
    
    data.append(b_10['Code'][i].zfill(5)) # code de la commune
    
    if b_10["Structure d'application"][i] == 'agglo' :
        data.append(b_10['Code Appart'][i]) # code agglomération
        data.append('NA') # code EPCI
    elif b_10["Structure d'application"][i] == 'epci' :
        data.append('NA') # code agglomération
        data.append(str(b_10['Code Appart'][i])) # code epci
    
    if float(b_10['Taux LS 2009'][i].replace(',', '.')[:-1]) > 0 or b_10['Situation de la commune par rapport au L.302-5 du CCH\n'][i] == "Non soumise sous seuil pop" :
        data.append(float(b_10['Taux LS 2009'][i].replace(',', '.')[:-1])/100) # taux de logements sociaux
    else :
        data.append(np.nan) # taux de logements sociaux
        
    data.append(b_10['Situation de la commune par rapport au L.302-5 du CCH\n'][i]) # situation
    
    data.append(b_10["LS 2009"][i]) # nombre de logements sociaux
    
data = np.array(data)
colonnes = ['code_commune', 'code_agglo_10', 'code_epci_10', 'taux_ls_10', 'situation_10', 'LLS_10']
data = np.reshape(data, (len(b_10), len(colonnes)))
df_10 = pd.DataFrame(data, columns = colonnes) # création du DataFrame du bilan de 2010
df_10.taux_ls_10 = pd.to_numeric(df_10.taux_ls_10, errors = 'coerce')
df_10.head()

,code_commune,code_agglo_10,code_epci_10,taux_ls_10,situation_10,LLS_10
0,01043,00757,NA,0.044,Soumise prélevée,71
1,01049,00757,NA,0.016,Non soumise sous seuil pop,17
2,01053,01501,NA,0.387,Non soumise atteint taux légal,7257
3,01065,NA,240100628,0.008,Non soumise sous seuil pop,5
4,01142,00757,NA,0.101,Soumise prélevée,152


In [68]:
# vérification des valeurs aberrantes
df_10[df_10.taux_ls_10 > .7]

,code_commune,code_agglo_10,code_epci_10,taux_ls_10,situation_10,LLS_10
1552,39308,NA,200010650,1.160,Non soumise sous seuil pop,87
2081,57058,57501,NA,0.850,Non soumise atteint taux légal,2661
2627,62188,00752,NA,2.443,Non soumise sous seuil pop,2900
2682,62386,00755,NA,0.750,Non soumise atteint taux légal,1927
2757,62771,00755,NA,0.717,Non soumise atteint taux légal,2809
3545,78621,00851,NA,0.701,Non soumise atteint taux légal,7206
3552,78644,00851,NA,0.727,Non soumise atteint taux légal,1490


In [69]:
b_10[df_10.taux_ls_10 > .7].iloc[:,:14]

,Dép,Code,Code Appart,Commune,Structure d'application,POP06,pop < seuil,Situation de la commune par rapport au L.302-5 du CCH\n,décrois Démo,DSU et > 15%,SERV ou PEB,LS 2009,RP 2009,Taux LS 2009
1552,39,39308,200010650,Malange,epci,261,1,Non soumise sous seuil pop,0,0,0,87,75,"116,0%"
2081,57,57058,57501,Behren-lès-Forbach,agglo,9146,0,Non soumise atteint taux légal,0,1,0,2661,3132,"85,0%"
2627,62,62188,00752,Burbure,agglo,2900,1,Non soumise sous seuil pop,0,0,0,2900,1187,"244,3%"
2682,62,62386,00755,Grenay,agglo,6532,0,Non soumise atteint taux légal,0,0,0,1927,2569,"75,0%"
2757,62,62771,00755,Sallaumines,agglo,10413,0,Non soumise atteint taux légal,0,0,0,2809,3919,"71,7%"
3545,78,78621,00851,Trappes,agglo,29529,0,Non soumise atteint taux légal,0,1,0,7206,10286,"70,1%"
3552,78,78644,00851,La Verrière,agglo,6057,0,Non soumise atteint taux légal,0,1,0,1490,2050,"72,7%"


## 8. Bilan 2011

In [70]:
# chargement des données de 2011
b_11 = pd.read_csv("../donnees_initiales/bilans_annuels/ART_55_Bilan_SRU_2011_donnees2020.csv",
                   sep = ';')
b_11.head()

,Code Insee,Dép.,Commune,Structure d'application,POP07,pop < seuil,Situation de la commune par rapport au L.302-5 du CCH\n,décrois. Démo.,DSU et > 15%,SERV. ou PEB,...,Dépenses déductibles EPCI,Dépenses déductibles,Dépenses indûment déduites.1,Reliquat disponible,Bénéficiaires locaux,Prélèvement net hors majoration à destination des bénéficiaires locaux,Majoration net à destination du Fonds National,Prélèvement net total,Reliquat reportable,Obs.
0,01030,01,Beauregard,agglo,894,1,Non soumise sous seuil pop,0,0,0,...,NC,NC,NC,NC,NC,NC,NC,NC,NC,La commune compte moins de 3500 habitants
1,01043,01,Beynost,agglo,4 240,0,Soumise prélevée,0,0,0,...,NC,"0,00","0,00","10 217,00",EPF,"61 991,00",NC,"61 991,00","0,00",En raison d'un trop perçu l'an dernier (10217....
2,01049,01,La Boisse,agglo,2804,1,Non soumise sous seuil pop,0,0,0,...,NC,NC,NC,NC,NC,NC,NC,NC,NC,La commune compte moins de 3500 habitants.
3,01053,01,Bourg-en-Bresse,agglo,40 506,0,Non soumise atteint taux légal,0,1,0,...,NC,NC,NC,NC,NC,NC,NC,NC,NC,NaN
4,01065,01,Buellas,epci,1633,1,Non soumise sous seuil pop,0,0,0,...,NC,NC,NC,NC,NC,NC,NC,NC,NC,La commune compte moins de 3500 habitants.


In [71]:
b_11.dtypes

Code Insee                                                                object
Dép.                                                                      object
Commune                                                                   object
Structure d'application                                                   object
POP07                                                                     object
pop < seuil                                                                int64
Situation de la commune par rapport au L.302-5 du CCH\n                   object
décrois. Démo.                                                             int64
DSU et > 15%                                                               int64
SERV. ou PEB                                                               int64
> 20%                                                                      int64
TOTAL                                                                     object
Nb RP                       

In [72]:
b_11['Situation de la commune par rapport au L.302-5 du CCH\n'].unique()

array(['Non soumise sous seuil pop', 'Soumise prélevée',
       'Non soumise atteint taux légal', 'Soumise prélèvement nul',
       'Non soumise exemptée', 'Soumise exonérée', 'Soumise non prélevée',
       'Soumise'], dtype=object)

In [73]:
data = [] # variable temporaire pour stocker les données

for i in range(len(b_11)) : # pour chaque commune
    data.append(b_11['Code Insee'][i]) # code de la commune
    
    taux = b_11['Taux LS 2010'][i].replace(',', '.')
    if taux[-1] == '%' :
        taux = float(taux[:-1])/100
    if float(taux) == 0 or b_11['Situation de la commune par rapport au L.302-5 du CCH\n'][i] == 'Non soumise sous seuil pop' :
        data.append(float(taux)) # taux de logements sociaux
    else :
        data.append(np.nan) # taux de logements sociaux
        
    data.append(b_11['Situation de la commune par rapport au L.302-5 du CCH\n'][i]) # situation
    
    data.append(int(b_11["TOTAL"][i].replace('\xa0', ''))) # nombre de logements sociaux
    
data = np.array(data)
colonnes = ['code_commune', 'taux_ls_11', 'situation_11', 'LLS_11']
data = np.reshape(data, (len(b_11), len(colonnes)))
df_11 = pd.DataFrame(data, columns = colonnes) # création du DataFrame du bilan 2011
df_11.taux_ls_11 = pd.to_numeric(df_11.taux_ls_11, errors = 'coerce')
df_11.head()

,code_commune,taux_ls_11,situation_11,LLS_11
0,01030,0.239437,Non soumise sous seuil pop,85
1,01043,NaN,Soumise prélevée,105
2,01049,0.019608,Non soumise sous seuil pop,21
3,01053,NaN,Non soumise atteint taux légal,7185
4,01065,0.109422,Non soumise sous seuil pop,72


In [74]:
# vérification des valeurs aberrantes
df_11[df_11.taux_ls_11 > 0.7]

,code_commune,taux_ls_11,situation_11,LLS_11
1505,38305,0.809129,Non soumise sous seuil pop,390
1586,39308,1.088608,Non soumise sous seuil pop,86


## 9. Bilan 2012

In [75]:
# chargement de données du bilan de 2012
b_12 = pd.read_csv("../donnees_initiales/bilans_annuels/ART_55_Bilan_SRU_2012_donnees2020.csv",
                   sep = ';')
b_12.head()

,Code,Dép.,Commune,Structure d'application,POP08,RP 2011,pop < seuil,Situation de la commune par rapport au L.302-5 du CCH\nÉtablie le 6/11/18,décrois. Démo.,DSU et > 15%,...,Dépenses déductibles EPCI,Dépenses déductibles,Dépenses indûment déduites.1,Reliquat disponible,Bénéficiaires locaux,Prélèvement net hors majoration à destination des bénéficiaires locaux,Majoration net à destination du Fonds National,Prélèvement net total,Reliquat reportable,Obs.
0,1423,01,Toussieux,agglo,745,280,1,Non soumise sous seuil pop,0,0,...,NC,NC,NC,NC,EPF,NC,NC,NC,NC,commune de moins de 3500 habitants
1,2257,02,Dallon,agglo,376,158,1,Non soumise sous seuil pop,0,0,...,NC,NC,NC,NC,NC,NC,NC,NC,NC,NaN
2,2303,02,Fayet,agglo,534,277,1,Non soumise sous seuil pop,0,0,...,NC,NC,NC,NC,NC,NC,NC,NC,NC,NaN
3,2340,02,Gauchy,agglo,5 593,2 310,0,Non soumise atteint taux légal,0,0,...,NC,NC,NC,NC,Fond national/FRAFU,NC,NC,NC,NC,NaN
4,2359,02,Grugies,agglo,956,460,1,Non soumise sous seuil pop,0,0,...,NC,NC,NC,NC,NC,NC,NC,NC,NC,NaN


In [76]:
b_12.dtypes # noms et types des colonnes

Code                                                                          object
Dép.                                                                          object
Commune                                                                       object
Structure d'application                                                       object
POP08                                                                         object
RP 2011                                                                       object
pop < seuil                                                                    int64
Situation de la commune par rapport au L.302-5 du CCH\nÉtablie le 6/11/18     object
décrois. Démo.                                                                 int64
DSU et > 15%                                                                   int64
SERV. ou PEB                                                                   int64
> 20%                                                            

In [77]:
b_12['Situation de la commune par rapport au L.302-5 du CCH\nÉtablie le 6/11/18'].unique()

array(['Non soumise sous seuil pop', 'Non soumise atteint taux légal',
       'Soumise prélèvement nul', 'Non soumise exemptée',
       'Soumise exonérée', 'soumise', 'Soumise prélevée',
       'Soumise non prélevée'], dtype=object)

In [78]:
b_12.replace('soumise', 'Soumise', inplace = True)

In [79]:
data = [] # variable temporaire pour stocker les données

for i in range(len(b_12)) : # pour chaque commune
    data.append(b_12['Code'][i].zfill(5)) # code de la commune
    
    if float(b_12['Taux LS 2011'][i].replace(',', '.')[:-1]) > 0 or b_12['Situation de la commune par rapport au L.302-5 du CCH\nÉtablie le 6/11/18'][i] == 'Non soumise sous seuil pop' :
        data.append(float(b_12['Taux LS 2011'][i].replace(',', '.')[:-1])/100) # taux de logements sociaux
    else :
        data.append(np.nan) # taux de logements sociaux
        
    data.append(b_12['Situation de la commune par rapport au L.302-5 du CCH\nÉtablie le 6/11/18'][i]) # situation
    
    data.append(b_12["LS 2011"][i]) # nombre de logements sociaux
    
data = np.array(data)
colonnes = ['code_commune', 'taux_ls_12', 'situation_12', 'LLS_12']
data = np.reshape(data, (len(b_12), len(colonnes)))
df_12 = pd.DataFrame(data, columns = colonnes) # création du DataFrame du bilan de 2012
df_12.taux_ls_12 = pd.to_numeric(df_12.taux_ls_12, errors = 'coerce')
df_12.head()

,code_commune,taux_ls_12,situation_12,LLS_12
0,01423,0.0,Non soumise sous seuil pop,0
1,02257,0.0,Non soumise sous seuil pop,0
2,02303,0.0,Non soumise sous seuil pop,0
3,02340,NaN,Non soumise atteint taux légal,0
4,02359,0.0,Non soumise sous seuil pop,0


In [80]:
# vérification des valeurs aberrantes
df_12[df_12.taux_ls_12 > .7]

,code_commune,taux_ls_12,situation_12,LLS_12
4413,93079,0.7014,Non soumise atteint taux légal,2913
4414,94011,0.7016,Non soumise atteint taux légal,4272
4415,93030,0.7060,Non soumise atteint taux légal,2646
4416,78644,0.7195,Non soumise atteint taux légal,1490
4417,62771,0.7203,Non soumise atteint taux légal,2809
4418,25547,0.7215,Non soumise atteint taux légal,1430
4419,62386,0.7532,Non soumise atteint taux légal,1929
4420,91235,0.8157,Non soumise atteint taux légal,1372
4421,57058,0.9807,Non soumise atteint taux légal,2947
4422,39308,1.1795,Non soumise sous seuil pop,92


## 10. Bilan 2013

In [81]:
# chargement des données du bilan de 2013
b_13 = pd.read_csv("../donnees_initiales/bilans_annuels/ART_55_Bilan_SRU_2013_donnees2020.csv",
                   sep = ';', header = 1)[:4775]
b_13.head()

,Struct appli,UU2010 / Siren EPCI,Nom agglomération / Nom EPCI,Dep,Code Insee,Commune,Population 2009,Nb RP 2012,"Exempt°, exonérat°, autres",fait le 30/10/18,...,Majoration net à destination du Fonds National,Prélèvement net total,Reliquat reportable,Obs.,Tx LS 2011,Tx LS (RPLS 2012) tous logts,Tx LS (RPLS 2012) logts conventionnés,redressé PH2,commentaires,en attente
0,agglo,9F501,Mamoudzou,976,97611,Mamoudzou,53022,NaN,NON,Soumise exonérée,...,NC,NC,NC,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,agglo,00758,Lyon,01,1030,Beauregard,869,347,OUI - Seuil de pop. - plus de 20% LLS,Non soumise sous seuil pop,...,NC,NC,NC,NaN,"31,12 %","29,39 %","29,39 %",NaN,NaN,NaN
2,agglo,00758,Lyon,01,1043,Beynost,4371,1749,NON,Soumise prélevée,...,"38 583,82","115 751,47","0,00","A ce jour, 1 arrêté (2/05/12) de délégation de...","6,14 %","5,37 %","5,37 %",NaN,NaN,NaN
3,agglo,00758,Lyon,01,1049,La Boisse,2 904,1 075,OUI - Seuil de pop.,Non soumise sous seuil pop,...,NC,NC,NC,commune de moins de 3500 habitants,"1,95 %","1,86 %","1,86 %",NaN,NaN,NaN
4,agglo,01501,Bourg-en-Bresse,01,1053,Bourg-en-Bresse,39586,18946,NON,Non soumise atteint taux légal,...,NC,NC,NC,NaN,"39,42 %","35,96 %","35,55 %",NaN,NaN,NaN


In [82]:
# affichage du nom des colonnes
for j in range(62) :
    print(b_13.columns[j])

Struct appli
UU2010 / Siren EPCI
Nom agglomération / Nom EPCI
Dep
Code Insee
Commune
Population 2009
Nb RP 2012
Exempt°, exonérat°, autres
fait le 30/10/18
Unnamed: 10
Pop < seuil
décrois. Démo.
DSU et > 15%
SERV. ou PEB ou PPR
> 20%
Taux LS 2012
LS 2012
Inv. notif.
5/100 f°
Prélèv. brut
Prélèv. brut maj
Total prélèvement
Prélev. Brut redressé
Majoration redressé
Total prélèv. brut maj redressé
Carence
Date de l'A.P.
Taux MAJ
Bénéficiaire convention
Dép. déduc.
Dépenses indument déduites
Prélèvt Bénéficiaires locaux
Reliquat N
Prélèvt FNDOLLTS
Reliquat N+1
Dest. Prélev. Redressé
% DRF
Plafond
CARENCE
Date de l'arrêté de carence
Taux de majoration
Prélèvement brut
Majoration
Total prélèvement brut avant plafonnement
Total prélèvement brut plafonné
Dépenses déductibles EPCI
Dépenses déductibles
Dépenses indûment déduites
Reliquat disponible
Bénéficiaires locaux
Prélèvement net hors majoration à destination des bénéficiaires locaux
Majoration net à destination du Fonds National
Prélèvemen

In [83]:
b_13['fait le 30/10/18'].unique()

array(['Soumise exonérée', 'Non soumise sous seuil pop',
       'Soumise prélevée', 'Non soumise atteint taux légal',
       'Soumise prélèvement nul', 'Soumise non prélevée',
       'Non soumise exemption', 'Exercice non réalisé', 'Soumise',
       'soumise'], dtype=object)

In [84]:
b_13.replace('soumise', 'Soumise', inplace = True)

In [85]:
# affichage des différentes valeurs de la variable Struct appli
b_13['Struct appli'].unique()

array(['agglo', 'epci'], dtype=object)

In [86]:
data = [] # variable temporaire pour stocker les données

for i in range(4775) : # pour chaque commune
    data.append(str(b_13['Code Insee'][i]).zfill(5)) # code de la commune
    
    if b_13['Struct appli'][i] == 'agglo' :
        data.append(b_13['UU2010 / Siren EPCI'][i]) # code agglomération
        data.append('NA') # code EPCI
    elif b_13['Struct appli'][i] == 'epci' :
        data.append('NA') # code agglomération
        data.append(str(b_13['UU2010 / Siren EPCI'][i])) # code EPCI
    
    # taux de logements sociaux
    if type(b_13['Taux LS 2012'][i]) == float :
        data.append(b_13['Taux LS 2012'][i])
    elif b_13['Taux LS 2012'][i][0] == '#' or b_13['Taux LS 2012'][i][0] == 'n' :
        data.append(np.nan)
    elif float(b_13['Taux LS 2012'][i].replace(',', '.')[:-1]) > 0 or b_13['fait le 30/10/18'][i] == 'Non soumise sous seuil pop' :
        data.append(float(b_13['Taux LS 2012'][i].replace(',', '.')[:-1])/100)
    else :
        data.append(np.nan)
        
    data.append(b_13['fait le 30/10/18'][i]) # situation
    
    data.append(float(str(b_13["LS 2012"][i]).replace('nc', 'nan'))) # nombre de logements sociaux
    
data = np.array(data)
colonnes = ['code_commune', 'code_agglo_13', 'code_epci_13', 'taux_ls_13', 'situation_13', 'LLS_13']
data = np.reshape(data, (len(b_13), len(colonnes)))
df_13 = pd.DataFrame(data, columns = colonnes) # création du DataFrame du bilan 2013
df_13.taux_ls_13 = pd.to_numeric(df_13.taux_ls_13, errors = 'coerce')
df_13.head()

,code_commune,code_agglo_13,code_epci_13,taux_ls_13,situation_13,LLS_13
0,97611,9F501,NA,NaN,Soumise exonérée,nan
1,01030,00758,NA,0.3141,Non soumise sous seuil pop,109.0
2,01043,00758,NA,0.0595,Soumise prélevée,104.0
3,01049,00758,NA,0.0195,Non soumise sous seuil pop,21.0
4,01053,01501,NA,0.4003,Non soumise atteint taux légal,7584.0


In [87]:
# vérification des valeurs aberrantes
df_13[df_13.taux_ls_13 > .7]

,code_commune,code_agglo_13,code_epci_13,taux_ls_13,situation_13,LLS_13
1051,27701,NA,242700409,0.7262,Non soumise atteint taux légal,3071.0
2431,57058,57501,NA,0.9858,Non soumise atteint taux légal,2842.0
3049,62386,00756,NA,0.7342,Non soumise atteint taux légal,1945.0
3135,62771,00756,NA,0.7223,Non soumise atteint taux légal,2811.0
3801,76322,00755,NA,0.7206,Non soumise atteint taux légal,8491.0
4065,78644,00851,NA,0.7222,Non soumise atteint taux légal,1490.0
4132,80725,80601,NA,0.7995,Non soumise atteint taux légal,1029.0
4442,91235,00851,NA,0.8075,Non soumise atteint taux légal,1372.0
4546,93030,00851,NA,0.7408,Non soumise atteint taux légal,2773.0
4579,94011,00851,NA,0.7371,Non soumise atteint taux légal,4432.0


## 11. Bilan 2014

Il y a **153** communes sans aucun logement sociaux d'après les données mais qui sont "non soumise atteint taux légal" donc il y a plus de 25 % (ou 20%) de logements sociaux.

In [88]:
# chargement des données du bilan de 2014
b_14 = pd.read_csv("../donnees_initiales/bilans_annuels/ART_55_Bilan_SRU_2014_donnees2020.csv",
                   sep = ';')#[:5157]
b_14.head()

,struct appli,Dep,Code INSEE,Communes,Population commune,Population >= seuil,Taux LLS à atteindre en 2013 pour les communes concernées par L302-5,Nom Agglo,Nom EPCI,RP 2013,...,Dépenses déductibles EPCI,Dépenses déductibles,Dépenses indûment déduites.1,Reliquat disponible.1,Bénéficiaires locaux.1,Prélèvement net hors majoration à destination des bénéficiaires locaux,Majoration net à destination du Fonds National,Prélèvement net total,Reliquat reportable,Obs
0,agglo,1,1030,Beauregard,870,0,25,Lyon,NaN,356,...,NC,NC,NC,NC,NC,NC,NC,NC,NC,NaN
1,agglo,1,1043,Beynost,4475,1,25,Lyon,NaN,1761,...,"1101425,06",0,0,0,EPF local,0,0,0,"939499,94",NaN
2,agglo,1,1049,La Boisse,2928,0,25,Lyon,NaN,1086,...,NC,NC,NC,NC,NC,NC,NC,NC,NC,NaN
3,agglo/epci,1,1053,Bourg-en-Bresse,40088,1,20,Bourg-en-Bresse,CA de Bourg en Bresse,19314,...,NC,NC,NC,NC,NC,NC,NC,NC,NC,NaN
4,epci,1,1065,Buellas,1669,0,20,NaN,CA de Bourg en Bresse,705,...,NC,NC,NC,NC,NC,NC,NC,NC,NC,NaN


In [89]:
b_14.dtypes # noms et types des colonnes

struct appli                                                               object
Dep                                                                        object
Code INSEE                                                                 object
Communes                                                                   object
Population commune                                                          int64
Population >= seuil                                                         int64
Taux LLS à atteindre en 2013 pour les communes concernées par L302-5        int64
Nom Agglo                                                                  object
Nom EPCI                                                                   object
RP 2013                                                                     int64
PFH 2013                                                                   object
exemption 2014                                                            float64
Décrois Démo    

In [90]:
b_14['fait le 30/10/18'].unique()

array(['Non soumise sous seuil pop', 'soumise prélèvement nul',
       'Non soumise atteint taux légal', 'Soumise prélevée',
       'Non soumise exemption', 'Soumise non prélevée',
       'Soumise exonérée', 'Soumise\xa0prélèvement nul', 'Soumise\xa0',
       'Soumise\xa0non prélevée'], dtype=object)

In [91]:
b_14.replace('soumise prélèvement nul', 'Soumise prélèvement nul', inplace = True)
b_14.replace('Soumise\xa0prélèvement nul', 'Soumise prélevement nul', inplace = True)
b_14.replace('Soumise\xa0', 'Soumise', inplace = True)
b_14.replace('Soumise\xa0non prélevée', 'Soumise non prélevée', inplace = True)
b_14.replace('Non soumise exemption', 'Non soumise exemptée', inplace = True)

In [92]:
data = [] # variable temporaire pour stocker les données

for i in range(len(b_14)) : # pour chaque commune
    # code commune
    data.append(b_14['Code INSEE'][i].zfill(5))
    
    # taux ls
    if float(b_14.total_lls_new[i]) > 0 or b_14['fait le 30/10/18'][i] == 'Non soumise sous seuil pop' :
        data.append(float(b_14.total_lls_new[i])/b_14['RP 2013'][i])
    else :
        data.append(np.nan)
        
    # situation
    data.append(b_14['fait le 30/10/18'][i])
    
    # nombre de logements sociaux
    data.append(b_14["Total LS 2013"][i])
    
data = np.array(data)
colonnes = ['code_commune', 'taux_ls_14', 'situation_14', 'LLS_14']
data = np.reshape(data, (len(b_14), len(colonnes)))
df_14 = pd.DataFrame(data, columns = colonnes) # création du DataFrame du bilan 2014
df_14.taux_ls_14 = pd.to_numeric(df_14.taux_ls_14, errors = 'coerce')
df_14.head()

,code_commune,taux_ls_14,situation_14,LLS_14
0,01030,0.306180,Non soumise sous seuil pop,109.0
1,01043,0.059057,Soumise prélèvement nul,104.0
2,01049,0.019337,Non soumise sous seuil pop,21.0
3,01053,0.410635,Non soumise atteint taux légal,7931.0
4,01065,0.131915,Non soumise sous seuil pop,93.0


In [93]:
# vérification des valeurs aberrantes
df_14[df_14.taux_ls_14 > 0.7]

,code_commune,taux_ls_14,situation_14,LLS_14
2649,57058,0.929437,Non soumise atteint taux légal,2608.0
4336,78644,0.724708,Non soumise atteint taux légal,1490.0
4771,91235,0.801870,Non soumise atteint taux légal,1372.0
4895,93030,0.728707,Non soumise atteint taux légal,2772.0
4928,94011,0.768631,Non soumise atteint taux légal,4631.0


In [94]:
# On ajoute les codes des EPCI en 2013 (données INSEE)
epci_2013 = pd.read_csv('../donnees_initiales/unites_urbaines_epci/EPCI_2013.csv', header = 1) # chargement des données reliant chaque commune à son EPCI
com_epci_2013 = epci_2013.merge(df_14, left_on = 'CODGEO', right_on = 'code_commune')
com_epci_2013['code_epci'] = com_epci_2013['EPCI']
com_epci_2013.head()

,CODGEO,LIBGEO,EPCI,LIBEPCI,NATURE_EPCI,code_commune,taux_ls_14,situation_14,LLS_14,code_epci
0,01030,Beauregard,240100735,CC Porte Ouest de la Dombes,CC,01030,0.306180,Non soumise sous seuil pop,109.0,240100735
1,01043,Beynost,240100800,CC de Miribel et du Plateau,CC,01043,0.059057,Soumise prélèvement nul,104.0,240100800
2,01049,La Boisse,240100610,CC du Canton de Montluel,CC,01049,0.019337,Non soumise sous seuil pop,21.0,240100610
3,01053,Bourg-en-Bresse,240100628,CA de Bourg en Bresse,CA,01053,0.410635,Non soumise atteint taux légal,7931.0,240100628
4,01065,Buellas,240100628,CA de Bourg en Bresse,CA,01065,0.131915,Non soumise sous seuil pop,93.0,240100628


In [95]:
# De même pour les codes des agglomérations (données INSEE)
agglos_2010_2019 = pd.read_csv('../donnees_initiales/unites_urbaines_epci/UU2010_au_01-01-2020.csv', header = 5) # chargement des données agglomération
agglos_2010_2019.head()

,CODGEO,LIBGEO,UU2010,LIBUU2010,TYPE_COM,STATUT_2017,DEP,REG
0,01001,L'Abergement-Clémenciat,01000,Communes rurales du département 01,RURAL,R,01,84
1,01002,L'Abergement-de-Varey,01000,Communes rurales du département 01,RURAL,R,01,84
2,01004,Ambérieu-en-Bugey,01302,Ambérieu-en-Bugey,URBAIN,C,01,84
3,01005,Ambérieux-en-Dombes,01000,Communes rurales du département 01,RURAL,R,01,84
4,01006,Ambléon,01000,Communes rurales du département 01,RURAL,R,01,84


Le fichier contenant la liste des communes et les unités urbaine auxquelles elles appartiennent est un fichier de l'INSEE daté de 2020. AInsi, certaines communes existant en 2013 (l'année pour laquelle est calculée le bilan 2014) n'existe plus en 2020, on va donc ajouter ces communes et leur unités urbaines au DataFrame des agglomérations.

In [96]:
# Ajoût des communes manquantes dans les agglomérations
agglos_2010_2019 = agglos_2010_2019.append({'CODGEO' : '19282', 'UU2010' : '00554'}, ignore_index = True)
agglos_2010_2019 = agglos_2010_2019.append({'CODGEO' : '27093', 'UU2010' : '00755'}, ignore_index = True)
agglos_2010_2019 = agglos_2010_2019.append({'CODGEO' : '38312', 'UU2010' : '38701'}, ignore_index = True)
agglos_2010_2019 = agglos_2010_2019.append({'CODGEO' : '50173', 'UU2010' : '50501'}, ignore_index = True)
agglos_2010_2019 = agglos_2010_2019.append({'CODGEO' : '50203', 'UU2010' : '50501'}, ignore_index = True)
agglos_2010_2019 = agglos_2010_2019.append({'CODGEO' : '50416', 'UU2010' : '50501'}, ignore_index = True)
agglos_2010_2019 = agglos_2010_2019.append({'CODGEO' : '50602', 'UU2010' : '50501'}, ignore_index = True)
agglos_2010_2019 = agglos_2010_2019.append({'CODGEO' : '57184', 'UU2010' : '57701'}, ignore_index = True)
agglos_2010_2019 = agglos_2010_2019.append({'CODGEO' : '68070', 'UU2010' : '68701'}, ignore_index = True)
agglos_2010_2019 = agglos_2010_2019.append({'CODGEO' : '69048', 'UU2010' : '00758'}, ignore_index = True)
agglos_2010_2019 = agglos_2010_2019.append({'CODGEO' : '69101', 'UU2010' : '00758'}, ignore_index = True)
agglos_2010_2019 = agglos_2010_2019.append({'CODGEO' : '69114', 'UU2010' : '00758'}, ignore_index = True)
agglos_2010_2019 = agglos_2010_2019.append({'CODGEO' : '71265', 'UU2010' : '71501'}, ignore_index = True)
agglos_2010_2019 = agglos_2010_2019.append({'CODGEO' : '74011', 'UU2010' : '74601'}, ignore_index = True)
agglos_2010_2019 = agglos_2010_2019.append({'CODGEO' : '74093', 'UU2010' : '74601'}, ignore_index = True)
agglos_2010_2019 = agglos_2010_2019.append({'CODGEO' : '74181', 'UU2010' : '74601'}, ignore_index = True)
agglos_2010_2019 = agglos_2010_2019.append({'CODGEO' : '74182', 'UU2010' : '74601'}, ignore_index = True)
agglos_2010_2019 = agglos_2010_2019.append({'CODGEO' : '74217', 'UU2010' : '74601'}, ignore_index = True)
agglos_2010_2019 = agglos_2010_2019.append({'CODGEO' : '74268', 'UU2010' : '74601'}, ignore_index = True)
agglos_2010_2019 = agglos_2010_2019.append({'CODGEO' : '78251', 'UU2010' : '00851'}, ignore_index = True)
agglos_2010_2019 = agglos_2010_2019.append({'CODGEO' : '78524', 'UU2010' : '00851'}, ignore_index = True)
agglos_2010_2019 = agglos_2010_2019.append({'CODGEO' : '88204', 'UU2010' : '88501'}, ignore_index = True)
agglos_2010_2019 = agglos_2010_2019.append({'CODGEO' : '91182', 'UU2010' : '00851'}, ignore_index = True)

In [97]:
com_agglos_2013 = agglos_2010_2019.merge(df_14, right_on = 'code_commune', left_on = 'CODGEO')
com_agglos_2013.head()

,CODGEO,LIBGEO,UU2010,LIBUU2010,TYPE_COM,STATUT_2017,DEP,REG,code_commune,taux_ls_14,situation_14,LLS_14
0,01030,Beauregard,00758,Lyon,URBAIN,B,01,84.0,01030,0.306180,Non soumise sous seuil pop,109.0
1,01043,Beynost,00758,Lyon,URBAIN,B,01,84.0,01043,0.059057,Soumise prélèvement nul,104.0
2,01049,La Boisse,00758,Lyon,URBAIN,B,01,84.0,01049,0.019337,Non soumise sous seuil pop,21.0
3,01053,Bourg-en-Bresse,01501,Bourg-en-Bresse,URBAIN,C,01,84.0,01053,0.410635,Non soumise atteint taux légal,7931.0
4,01065,Buellas,01104,Buellas,URBAIN,C,01,84.0,01065,0.131915,Non soumise sous seuil pop,93.0


Les communes 24026 et 24312 sont classées dans l'unité urbaine 24000, c'est-à-dire dans les communes rurales du département 24, cependant, dans les données du bilan de 2014, ces deux communes sont dans l'unité urbaine de Périgueux. On change donc le code de leur unité urbaine.

In [98]:
com_agglos_2013[com_agglos_2013.CODGEO == '24026']

,CODGEO,LIBGEO,UU2010,LIBUU2010,TYPE_COM,STATUT_2017,DEP,REG,code_commune,taux_ls_14,situation_14,LLS_14
877,24026,Bassillac et Auberoche,24000,Communes rurales du département 24,RURAL,R,24,75.0,24026,0.036936,Non soumise sous seuil pop,27.0


In [99]:
com_agglos_2013[com_agglos_2013.CODGEO == '24312']

,CODGEO,LIBGEO,UU2010,LIBUU2010,TYPE_COM,STATUT_2017,DEP,REG,code_commune,taux_ls_14,situation_14,LLS_14
908,24312,Sanilhac,24000,Communes rurales du département 24,RURAL,R,24,75.0,24312,0.076296,Non soumise sous seuil pop,103.0


In [100]:
com_agglos_2013.replace('24000', '24501', inplace = True)

In [101]:
# les différentes possibilités de valeurs des communes pour comme structure d'application
b_14['struct appli'].unique()

array(['agglo', 'agglo/epci', 'epci', 'isolé'], dtype=object)

In [102]:
data = [] # variable temporaire pour stocker les données

for i in range(len(b_14)) : # pour chaque commune
    data.append(b_14['Code INSEE'][i].zfill(5)) # code commune
    
    # code aaglo et/ou EPCI
    if b_14['struct appli'][i] == 'agglo' :
        #print(com_agglos_2013[com_agglos_2013.CODGEO == b_14['Code INSEE'][i].zfill(5)].UU2010, b_14['Code INSEE'][i].zfill(5))
        data.append(com_agglos_2013[com_agglos_2013.CODGEO == b_14['Code INSEE'][i].zfill(5)].iloc[0].UU2010) # le code de l'UU
        data.append('NA') # Il n'y a pas de code EPCI
    elif b_14['struct appli'][i] == 'epci' :
        data.append('NA') # Il n'y a pas d'UU
        data.append(str(com_epci_2013[com_epci_2013.CODGEO == b_14['Code INSEE'][i].zfill(5)].iloc[0].EPCI)) # Le code de l'EPCI
    elif b_14['struct appli'][i] == 'agglo/epci' :
        #print(com_agglos_2013[com_agglos_2013.CODGEO == b_14['Code INSEE'][i].zfill(5)].UU2010, b_14['Code INSEE'][i].zfill(5))
        data.append(com_agglos_2013[com_agglos_2013.CODGEO == b_14['Code INSEE'][i].zfill(5)].iloc[0].UU2010) # le code de l'UU
        data.append(str(com_epci_2013[com_epci_2013.CODGEO == b_14['Code INSEE'][i].zfill(5)].iloc[0].EPCI)) # Le code de l'EPCI
    else :
        data.append('NA')
        data.append('NA')
    
    # taux logements sociaux
    if float(b_14.total_lls_new[i]) > 0 or b_14['fait le 30/10/18'][i] == 'Non soumise sous seuil pop' :
        data.append(float(b_14.total_lls_new[i])/b_14['RP 2013'][i])
    else :
        data.append(np.nan)
        
    # situation
    data.append(b_14['fait le 30/10/18'][i])
    
    # nombre de logements sociaux
    data.append(b_14["Total LS 2013"][i])
    
data = np.array(data)
colonnes = ['code_commune', 'code_agglo_14', 'code_epci_14', 'taux_ls_14', 'situation_14', 'LLS_14']
data = np.reshape(data, (len(b_14), len(colonnes)))
df_14 = pd.DataFrame(data, columns = colonnes) # création du DataFrame du bilan 2014
df_14.taux_ls_14 = pd.to_numeric(df_14.taux_ls_14, errors = 'coerce')
df_14.head()

,code_commune,code_agglo_14,code_epci_14,taux_ls_14,situation_14,LLS_14
0,01030,00758,NA,0.306180,Non soumise sous seuil pop,109.0
1,01043,00758,NA,0.059057,Soumise prélèvement nul,104.0
2,01049,00758,NA,0.019337,Non soumise sous seuil pop,21.0
3,01053,01501,240100628,0.410635,Non soumise atteint taux légal,7931.0
4,01065,NA,240100628,0.131915,Non soumise sous seuil pop,93.0


In [103]:
df_14['taux_agglo_14'] = np.nan
df_14['taux_epci_14'] = np.nan
df_14['rp_agglo_14'] = np.nan
df_14['rp_epci_14'] = np.nan
df_14['rp_agglo_nc_14'] = np.nan
df_14['rp_epci_nc_14'] = np.nan
df_14.head()

,code_commune,code_agglo_14,code_epci_14,taux_ls_14,situation_14,LLS_14,taux_agglo_14,taux_epci_14,rp_agglo_14,rp_epci_14,rp_agglo_nc_14,rp_epci_nc_14
0,01030,00758,NA,0.306180,Non soumise sous seuil pop,109.0,NaN,NaN,NaN,NaN,NaN,NaN
1,01043,00758,NA,0.059057,Soumise prélèvement nul,104.0,NaN,NaN,NaN,NaN,NaN,NaN
2,01049,00758,NA,0.019337,Non soumise sous seuil pop,21.0,NaN,NaN,NaN,NaN,NaN,NaN
3,01053,01501,240100628,0.410635,Non soumise atteint taux légal,7931.0,NaN,NaN,NaN,NaN,NaN,NaN
4,01065,NA,240100628,0.131915,Non soumise sous seuil pop,93.0,NaN,NaN,NaN,NaN,NaN,NaN


In [104]:
agglos = np.delete(df_14.code_agglo_14.unique(), 2)
# df_14.code_agglo_14.unique() contient la liste des codes des agglomérations présentes dans le bilan de 2014.
# L'élément en position 2 de la liste est occupée par 'nan' qui correspond à la commune ne fait pas partie d'une UU
# concernée par l'article 55 de la loi SRU donc on enlève cet élément de la liste.

# On va calculer les taux de logements sociaux pour chaque agglomération.
# Ainsi que le nombre total de résidences principales de chaque agglomération
for agglo in agglos : # Pour chaque agglomération
    df = b_14[df_14.code_agglo_14 == agglo] # Toutes les données sur les communes de l'agglomération
    
    nb_com_agglo = len(df) # le nombre de communes de l'agglomération
    nb_com_nc = len(df[df.total_lls_new == 0])
                 # le nombre de communes de l'agglomération, pour laquelle il manque le nombre de logements sociaux
    
    nb_rp = int(sum(df['RP 2013'])) # le nombre de résidences principales de l'agglomération
    nb_ls = sum(df.total_lls_new) # le nombre de logements sociaux de l'agglomération
    nb_rp_nc = int(sum(df['RP 2013'][df.total_lls_new == 0])) 
                 # le nombre de résidences principales de l'agglomération qui sont potentiellement des logements sociaux sans 
                 # être classées comme tel (pas d'informations sur les logements sociaux dans la commune) 
    
    com = df.index # l'index des communes de l'agglomération
    
    # On remplit notre base de données avec les valeurs calculées.
    for commune in com :
        if nb_rp != 0 :
            df_14['taux_agglo_14'][commune] = nb_ls/nb_rp # le taux de logement sociaux de l'agglomération
        df_14['rp_agglo_14'][commune] = nb_rp # nombre de résidences principales de l'agglomeration
        df_14['rp_agglo_nc_14'][commune] = nb_rp_nc # nombre de résidences principales non classées dans l'agglomeration
        
df_14.head(10)

,code_commune,code_agglo_14,code_epci_14,taux_ls_14,situation_14,LLS_14,taux_agglo_14,taux_epci_14,rp_agglo_14,rp_epci_14,rp_agglo_nc_14,rp_epci_nc_14
0,01030,00758,NA,0.306180,Non soumise sous seuil pop,109.0,0.226400,NaN,718927.0,NaN,2286.0,NaN
1,01043,00758,NA,0.059057,Soumise prélèvement nul,104.0,0.226400,NaN,718927.0,NaN,2286.0,NaN
2,01049,00758,NA,0.019337,Non soumise sous seuil pop,21.0,0.226400,NaN,718927.0,NaN,2286.0,NaN
3,01053,01501,240100628,0.410635,Non soumise atteint taux légal,7931.0,0.342950,NaN,27141.0,NaN,0.0,NaN
4,01065,NA,240100628,0.131915,Non soumise sous seuil pop,93.0,NaN,NaN,NaN,NaN,NaN,NaN
5,01142,00758,NA,0.122156,Soumise prélevée,204.0,0.226400,NaN,718927.0,NaN,2286.0,NaN
6,01145,NA,240100628,0.132075,Non soumise sous seuil pop,63.0,NaN,NaN,NaN,NaN,NaN,NaN
7,01157,00758,NA,0.043050,Non soumise sous seuil pop,35.0,0.226400,NaN,718927.0,NaN,2286.0,NaN
8,01160,00652,NA,0.226283,Soumise prélèvement nul,966.0,0.163782,NaN,73482.0,NaN,1017.0,NaN
9,01166,00758,NA,0.073265,Non soumise sous seuil pop,57.0,0.226400,NaN,718927.0,NaN,2286.0,NaN


In [105]:
b_14[df_14.code_agglo_14 == '9F501']

,struct appli,Dep,Code INSEE,Communes,Population commune,Population >= seuil,Taux LLS à atteindre en 2013 pour les communes concernées par L302-5,Nom Agglo,Nom EPCI,RP 2013,...,Dépenses déductibles EPCI,Dépenses déductibles,Dépenses indûment déduites.1,Reliquat disponible.1,Bénéficiaires locaux.1,Prélèvement net hors majoration à destination des bénéficiaires locaux,Majoration net à destination du Fonds National,Prélèvement net total,Reliquat reportable,Obs
5139,agglo,976,97611,Mamoudzou,57281,1,20,Mamoudzou,NaN,0,...,NC,NC,NC,NC,NC,NC,NC,NC,NC,NaN


L'agglomération 5F501 composé de la commune Mamoudzou n'a pas de taux de logements sociaux indiqué car il n'y a pas d'information sur les logements de cette commune.

In [106]:
code_epci = df_14.code_epci_14.unique()[1:]
# df_14.code_epci_14.unique() contient la liste des codes des EPCI présents dans la base de données de 2021.
# Le premier élément de la liste est 'nan' (lorsque l'EPCI de la commune n'est pas concernée par la loi SRU).
# On enlève donc cet élément  de la liste pour ne garder que les codes des EPCI.

# On va calculer les taux de logements sociaux pour chaque EPCI
# Ainsi que le nombre total de résidences principales de chaque EPCI
for epci in code_epci : # Pour chaque EPCI
    #print(epci)
    df = b_14[df_14.code_epci_14 == epci] # Toutes les données sur les communes de l'EPCI
    
    nb_com_epci = len(df) # le nombre de communes de l'EPCI
    nb_com_nc = len(df[df.total_lls_new == 0])
                 # le nombre de communes de l'EPCI, pour laquelle il manque le nombre de logements sociaux
    
    nb_rp = int(sum(df['RP 2013'])) # le nombre de résidences principales de l'EPCI
    nb_ls = sum(df.total_lls_new) # le nombre de logements sociaux de l'EPCI
    nb_rp_nc = int(sum(df['RP 2013'][df.total_lls_new == 0])) 
                 # le nombre de résidences principales de l'EPCI qui sont potentiellement des logements sociaux sans 
                 # être comptés comme tel (pas d'informations sur les logements sociaux dans la commune)
    
    com = df.index # l'index des communes de l'epci
    
    # On remplit notre base de données avec les valeurs calculées.
    for commune in com : 
        df_14['taux_epci_14'][commune] = nb_ls/nb_rp # le taux de logement sociaux de l'epci
        df_14['rp_epci_14'][commune] = nb_rp 
        df_14['rp_epci_nc_14'][commune] = nb_rp_nc 
df_14.head(10)

,code_commune,code_agglo_14,code_epci_14,taux_ls_14,situation_14,LLS_14,taux_agglo_14,taux_epci_14,rp_agglo_14,rp_epci_14,rp_agglo_nc_14,rp_epci_nc_14
0,01030,00758,NA,0.306180,Non soumise sous seuil pop,109.0,0.226400,NaN,718927.0,NaN,2286.0,NaN
1,01043,00758,NA,0.059057,Soumise prélèvement nul,104.0,0.226400,NaN,718927.0,NaN,2286.0,NaN
2,01049,00758,NA,0.019337,Non soumise sous seuil pop,21.0,0.226400,NaN,718927.0,NaN,2286.0,NaN
3,01053,01501,240100628,0.410635,Non soumise atteint taux légal,7931.0,0.342950,0.307628,27141.0,32289.0,0.0,0.0
4,01065,NA,240100628,0.131915,Non soumise sous seuil pop,93.0,NaN,0.307628,NaN,32289.0,NaN,0.0
5,01142,00758,NA,0.122156,Soumise prélevée,204.0,0.226400,NaN,718927.0,NaN,2286.0,NaN
6,01145,NA,240100628,0.132075,Non soumise sous seuil pop,63.0,NaN,0.307628,NaN,32289.0,NaN,0.0
7,01157,00758,NA,0.043050,Non soumise sous seuil pop,35.0,0.226400,NaN,718927.0,NaN,2286.0,NaN
8,01160,00652,NA,0.226283,Soumise prélèvement nul,966.0,0.163782,NaN,73482.0,NaN,1017.0,NaN
9,01166,00758,NA,0.073265,Non soumise sous seuil pop,57.0,0.226400,NaN,718927.0,NaN,2286.0,NaN


## 12. Bilan 2015

Il y a **10** communes "non soumise[s] exemptée[s]" et **179** communes "non soumis[s] attei[gnent] taux légal"  sans aucun logement social renseigné.

In [107]:
# chargement des données du bilan de 2015
b_15 = pd.read_csv("../donnees_initiales/bilans_annuels/ART_55_Bilan_SRU_2015_donnees2020.csv",
                   sep = ';')#[:5776]
b_15.head()

,struct applicat°,Code region,Nouvelle région,Region,Dép,Code INSEE,Commune,Population 2014 commune,Pop >= seuil (3500 - 1500),Taux de LLS applicable au 1/1/2014 pour les communes concernées par L.302-5 CCH,...,Dépenses déductibles,Dépenses indûment déduites.1,Reliquat disponible,Bénéficiaires locaux.1,Prélèvement net hors majoration à destination des bénéficiaires locaux,Majoration net à destination du Fonds National,Prélèvement net total,Reliquat reportable,Obs.,Obs PH2
0,epci,82.0,AURA,Rhône-Alpes,1,1011,Apremont,374.0,0.0,20.0,...,NC,NC,NC,NC,NC,NC,NC,NC,NaN,NaN
1,epci,82.0,AURA,Rhône-Alpes,1,1014,Arbent,3458.0,0.0,20.0,...,NC,NC,NC,NC,NC,NC,NC,NC,NaN,NaN
2,agglo,82.0,AURA,Rhône-Alpes,1,1030,Beauregard,867.0,0.0,25.0,...,NC,NC,NC,NC,NC,NC,NC,NC,NaN,NaN
3,epci,82.0,AURA,Rhône-Alpes,1,1031,Bellignat,3590.0,1.0,20.0,...,NC,NC,NC,NC,NC,NC,NC,NC,NaN,NaN
4,epci,82.0,AURA,Rhône-Alpes,1,1035,Belleydoux,323.0,0.0,20.0,...,NC,NC,NC,NC,NC,NC,NC,NC,NaN,NaN


In [108]:
# noms des colonnes
for j in b_15.columns :
    print(j)

struct applicat°
Code region
Nouvelle région
Region
Dép
Code INSEE
Commune
Population 2014 commune
Pop >= seuil (3500 - 1500)
Taux de LLS applicable au 1/1/2014 pour les communes concernées par L.302-5 CCH
Code Unité Urbaine
Nom Agglo
Décrois démo UU
Population 2014 Agglo
Taux de LLS de l'agglo
Code EPCI
Nom EPCI
Dep. EPCI
Décrois démo EPCI
Population 2014 EPCI
Taux de LLS de l'EPCI
RP 2014
PFH 2014
Situation de la commune au regard Situation de la commune par rapport au L.302-5 du CCH
Exemption
Décrois. Démo
DSU et >15%
Communes isolées prélevées à partir de 2017 et Mamoudzou
SERV, PEB ou PPR
>25% (>20%) 2014
Taux LLS 2014
Total LLS 2014
total_lls_new
Inv. notif.
% DRF 5 ou 7,5
5/100 f°
Prélèvt brut redressé
Majorat° redressée
Prélèvt total redressé
Constat de carence
Date de l'A.P.
Taux de maj.
Dép. déduc.
Dépenses indûment déduites
Bénéficiaires locaux
Prélèvt arrêté redressé
reliquat disponible
Prélèvt Fonds National redressé
Prélèvt total net
reliquat reportable
% DRF
Plafond
CARE

In [109]:
data = [] # variable temporaire pour stocker les données

for i in range(len(b_15)) : # pour chaque commune
    # code commune :
    data.append(str(b_15['Code INSEE'][i]).zfill(5))
    
    # code agglo :
    temp = str(b_15['Code Unité Urbaine'][i])
    if temp == 'nan' :
        data.append(np.nan)
    else :
        data.append(str(b_15['Code Unité Urbaine'][i]).zfill(5))
    
    # code epci :
    temp = str(b_15['Code EPCI'][i])
    if temp == 'nan' :
        data.append(np.nan)
    else :
        data.append(str(int(b_15['Code EPCI'][i])))
        
    # taux LS commune :
    if float(b_15.total_lls_new[i]) > 0 or b_15['Situation de la commune au regard Situation de la commune par rapport au L.302-5 du CCH'][i] == 'Non soumise sous seuil pop' :
        data.append(float(b_15.total_lls_new[i])/b_15['RP 2014'][i])
    else :
        data.append(np.nan)
        
    # situation commune :
    data.append(b_15['Situation de la commune au regard Situation de la commune par rapport au L.302-5 du CCH'][i])
    
    # nombre de logements sociaux
    data.append(b_15['Total LLS 2014'][i])
    
data = np.array(data)
colonnes = ['code_commune', 'code_agglo_15', 'code_epci_15', 'taux_ls_15', 'situation_15', 'LLS_15']
data = np.reshape(data, (len(b_15), len(colonnes)))
df_15 = pd.DataFrame(data, columns = colonnes) # création du DataFrame du bilan 2015
df_15.taux_ls_15 = pd.to_numeric(df_15.taux_ls_15, errors = 'coerce')
df_15.head()

,code_commune,code_agglo_15,code_epci_15,taux_ls_15,situation_15,LLS_15
0,01011,nan,200042935,0.075862,Non soumise sous seuil pop,11.0
1,01014,nan,200042935,0.305534,Non soumise sous seuil pop,403.0
2,01030,00758,nan,0.297003,Non soumise sous seuil pop,109.0
3,01031,nan,200042935,0.504094,Non soumise atteint taux légal,862.0
4,01035,nan,200042935,0.000000,Non soumise sous seuil pop,0.0


In [110]:
# création des colonnes en lien avec les agglomérations et EPCI
df_15['taux_agglo_15'] = np.nan
df_15['taux_epci_15'] = np.nan
df_15['rp_agglo_15'] = np.nan
df_15['rp_epci_15'] = np.nan
df_15['rp_agglo_nc_15'] = np.nan
df_15['rp_epci_nc_15'] = np.nan
df_15.head()

,code_commune,code_agglo_15,code_epci_15,taux_ls_15,situation_15,LLS_15,taux_agglo_15,taux_epci_15,rp_agglo_15,rp_epci_15,rp_agglo_nc_15,rp_epci_nc_15
0,01011,nan,200042935,0.075862,Non soumise sous seuil pop,11.0,NaN,NaN,NaN,NaN,NaN,NaN
1,01014,nan,200042935,0.305534,Non soumise sous seuil pop,403.0,NaN,NaN,NaN,NaN,NaN,NaN
2,01030,00758,nan,0.297003,Non soumise sous seuil pop,109.0,NaN,NaN,NaN,NaN,NaN,NaN
3,01031,nan,200042935,0.504094,Non soumise atteint taux légal,862.0,NaN,NaN,NaN,NaN,NaN,NaN
4,01035,nan,200042935,0.000000,Non soumise sous seuil pop,0.0,NaN,NaN,NaN,NaN,NaN,NaN


In [111]:
agglos = b_15['Code Unité Urbaine'].unique()[1:]
# b_15['Code Unité Urbaine'].unique() contient la liste des codes des agglomérations présentes dans le bilan de 2015.
# Le premier élément de la liste est occupée par 'nan' qui correspond à la commune ne fait pas partie d'une UU
# concernée par l'article 55 de la loi SRU donc on ne parcourt la liste des UU que pour les éléments suivants.

# On va calculer les taux de logements sociaux pour chaque agglomération
# Ainsi que le nombre total de résidences principales de chaque agglomération
for agglo in agglos : # Pour chaque agglomération
    df = b_15[b_15['Code Unité Urbaine'] == agglo] # Toutes les données sur les communes de l'agglomération
    
    nb_com_agglo = len(df) # le nombre de communes de l'agglomération
    nb_com_nc = len(df[df.total_lls_new == 0])
                 # le nombre de communes de l'agglomération, pour laquelle il manque le nombre de logements sociaux
    
    nb_rp = int(sum(df['RP 2014'])) # le nombre de résidences principales de l'agglomération
    nb_ls = sum(df.total_lls_new) # le nombre de logements sociaux de l'agglomération
    nb_rp_nc = int(sum(df['RP 2014'][df.total_lls_new == 0])) 
                 # le nombre de résidences principales de l'agglomération qui sont potentiellement des logements sociaux sans 
                 # être comptées comme tel (pas d'informations sur les logements sociaux dans la commune)
    
    com = df.index # l'index des communes de l'agglomération
    
    # On remplit notre base de données avec les valeurs calculées.
    for commune in com : 
        df_15['taux_agglo_15'][commune] = nb_ls/nb_rp # le taux de logement sociaux de l'agglomération
        df_15['rp_agglo_15'][commune] = nb_rp
        df_15['rp_agglo_nc_15'][commune] = nb_rp_nc
df_15.head(10)

,code_commune,code_agglo_15,code_epci_15,taux_ls_15,situation_15,LLS_15,taux_agglo_15,taux_epci_15,rp_agglo_15,rp_epci_15,rp_agglo_nc_15,rp_epci_nc_15
0,01011,nan,200042935,0.075862,Non soumise sous seuil pop,11.0,NaN,NaN,NaN,NaN,NaN,NaN
1,01014,nan,200042935,0.305534,Non soumise sous seuil pop,403.0,NaN,NaN,NaN,NaN,NaN,NaN
2,01030,00758,nan,0.297003,Non soumise sous seuil pop,109.0,0.228233,NaN,727983.0,NaN,2340.0,NaN
3,01031,nan,200042935,0.504094,Non soumise atteint taux légal,862.0,NaN,NaN,NaN,NaN,NaN,NaN
4,01035,nan,200042935,0.000000,Non soumise sous seuil pop,0.0,NaN,NaN,NaN,NaN,NaN,NaN
5,01043,00758,nan,0.059125,Soumise prélèvement nul,104.0,0.228233,NaN,727983.0,NaN,2340.0,NaN
6,01049,00758,nan,0.019196,Non soumise sous seuil pop,21.0,0.228233,NaN,727983.0,NaN,2340.0,NaN
7,01051,nan,200042935,0.085106,Non soumise sous seuil pop,4.0,NaN,NaN,NaN,NaN,NaN,NaN
8,01053,01501,240100628,0.404093,Non soumise atteint taux légal,7879.0,0.337955,NaN,27406.0,NaN,0.0,NaN
9,01060,nan,200042935,0.149606,Non soumise sous seuil pop,38.0,NaN,NaN,NaN,NaN,NaN,NaN


In [112]:
code_epci = np.delete(b_15['Code EPCI'].unique(), 1)
# b_15['Code EPCI'].unique() contient la liste des codes des EPCI présents dans la base de données de 2021.
# La deuxième position est occupée par 'nan' (lorsque l'EPCI de la commune n'est pas concernée par la loi SRU).
# On enlève donc l'élément en deuxième position de la liste pour ne garder que les codes des EPCI.

# On va calculer les taux de logements sociaux pour chaque EPCI
# Ainsi que le nombre total de résidences principales de chaque EPCI
for epci in code_epci : # Pour chaque EPCI
    #print(epci)
    df = b_15[b_15['Code EPCI'] == epci] # Toutes les données sur les communes de l'EPCI
    
    nb_com_epci = len(df) # le nombre de communes de l'EPCI
    nb_com_nc = len(df[df.total_lls_new == 0])
                 # le nombre de communes de l'EPCI, pour laquelle il manque le nombre de logements sociaux
    
    nb_rp = int(sum(df['RP 2014'])) # le nombre de résidences principales de l'EPCI
    nb_ls = sum(df.total_lls_new) # le nombre de logements sociaux de l'EPCI
    nb_rp_nc = int(sum(df['RP 2014'][df.total_lls_new == 0])) 
                 # le nombre de résidences principales de l'EPCI qui sont potentiellement des logements sociaux sans 
                 # être comptés comme tel (pas d'informations sur les logements sociaux dans la commune)
    
    com = df.index # l'index des communes de l'epci
    
    # On remplit notre base de données avec les valeurs calculées.
    for commune in com : 
        df_15['taux_epci_15'][commune] = nb_ls/nb_rp # le taux de logement sociaux de l'epci
        df_15['rp_epci_15'][commune] = nb_rp
        df_15['rp_epci_nc_15'][commune] = nb_rp_nc
df_15.head(10)

,code_commune,code_agglo_15,code_epci_15,taux_ls_15,situation_15,LLS_15,taux_agglo_15,taux_epci_15,rp_agglo_15,rp_epci_15,rp_agglo_nc_15,rp_epci_nc_15
0,01011,nan,200042935,0.075862,Non soumise sous seuil pop,11.0,NaN,0.322841,NaN,24145.0,NaN,626.0
1,01014,nan,200042935,0.305534,Non soumise sous seuil pop,403.0,NaN,0.322841,NaN,24145.0,NaN,626.0
2,01030,00758,nan,0.297003,Non soumise sous seuil pop,109.0,0.228233,NaN,727983.0,NaN,2340.0,NaN
3,01031,nan,200042935,0.504094,Non soumise atteint taux légal,862.0,NaN,0.322841,NaN,24145.0,NaN,626.0
4,01035,nan,200042935,0.000000,Non soumise sous seuil pop,0.0,NaN,0.322841,NaN,24145.0,NaN,626.0
5,01043,00758,nan,0.059125,Soumise prélèvement nul,104.0,0.228233,NaN,727983.0,NaN,2340.0,NaN
6,01049,00758,nan,0.019196,Non soumise sous seuil pop,21.0,0.228233,NaN,727983.0,NaN,2340.0,NaN
7,01051,nan,200042935,0.085106,Non soumise sous seuil pop,4.0,NaN,0.322841,NaN,24145.0,NaN,626.0
8,01053,01501,240100628,0.404093,Non soumise atteint taux légal,7879.0,0.337955,0.304400,27406.0,32661.0,0.0,0.0
9,01060,nan,200042935,0.149606,Non soumise sous seuil pop,38.0,NaN,0.322841,NaN,24145.0,NaN,626.0


In [113]:
# vérification des valeurs aberrantes
df_15[df_15.taux_ls_15 > .7]

,code_commune,code_agglo_15,code_epci_15,taux_ls_15,situation_15,LLS_15,taux_agglo_15,taux_epci_15,rp_agglo_15,rp_epci_15,rp_agglo_nc_15,rp_epci_nc_15
1191,25547,25601,242503886,0.725922,Non soumise atteint taux légal,1417.0,0.348498,0.326834,48204.0,52109.0,0.0,180.0
1405,27701,nan,200035665,0.770771,Non soumise atteint taux légal,3349.0,NaN,0.293193,NaN,27221.0,NaN,7372.0
3059,57058,57501,245700372,0.856675,Non soumise atteint taux légal,2355.0,0.294640,0.234901,37164.0,34074.0,1591.0,4466.0
4553,76322,00755,200023414,0.715514,Non soumise atteint taux légal,8657.0,0.345702,0.337354,206282.0,219227.0,2360.0,1885.0
4878,78644,00851,247800451,0.722249,Non soumise atteint taux légal,1490.0,0.262480,0.403791,4559735.0,55556.0,22990.0,0.0
5376,91235,00851,249100504,0.802570,Non soumise atteint taux légal,1374.0,0.262480,0.255976,4559735.0,52372.0,22990.0,0.0
5500,93030,00851,200006385,0.731612,Non soumise atteint taux légal,2805.0,0.262480,0.351626,4559735.0,36755.0,22990.0,0.0
5533,94011,00851,nan,0.751488,Non soumise atteint taux légal,4545.0,0.262480,NaN,4559735.0,NaN,22990.0,NaN


## 13. Bilan 2016

Les situations des communes sont renseignées seulement en "non soumise" sans autre précision lorsque les communes ne sont pas soumises mais il manque l'information si c'est parce que la commune atteint le taux légal, si c'est parce qu'elle n'a pas atteint le seuil de population ou si elle est exemptée. Il y a un problème lorsque le nombre de logements sociaux est noté à 0.

In [114]:
# chargement des données du bilan de 2016
b_16 = pd.read_csv("../donnees_initiales/bilans_annuels/ART_55_Bilan_SRU_2016_donnees2020.csv",
                   sep = ';')[:5783]
b_16.head()

,Code INSEE,modification,Structure application,Région,métro DOM,Dép,Code INSEE.1,Commune,Population commune 2012 (publiée 2015),Pop>=seuil (1500-3500),...,Nb LLS très social financement Etat (en service depuis le 01/01/2002),Nb LLS social financement Etat (en service depuis le 01/01/2002),Nb LLS intermédiaire financement Etat (en service depuis le 01/01/2002),Nb LLS autres financement Etat (en service depuis le 01/01/2002),Nb LLS très social financement ANAH (en service depuis le 01/01/2002),Nb LLS social financement ANAH (en service depuis le 01/01/2002),Nb LLS autres financement ANAH (en service depuis le 01/01/2002),Visée,Observations,Observations PH2
0,01011,NaN,epci,Auvergne-Rhône-Alpes,métropole,01,01011,Apremont,385.0,0.0,...,0.0,8.0,0.0,0.0,0.0,0.0,0.0,OUI,NaN,NaN
1,01014,NaN,epci,Auvergne-Rhône-Alpes,métropole,01,01014,Arbent,3440.0,0.0,...,0.0,69.0,19.0,0.0,0.0,0.0,0.0,OUI,NaN,NaN
2,01030,NaN,agglo,Auvergne-Rhône-Alpes,métropole,01,01030,Beauregard,866.0,0.0,...,0.0,22.0,0.0,0.0,2.0,0.0,0.0,OUI,NaN,NaN
3,01031,NaN,epci,Auvergne-Rhône-Alpes,métropole,01,01031,Bellignat,3642.0,1.0,...,0.0,16.0,46.0,0.0,0.0,0.0,0.0,OUI,NaN,NaN
4,01035,NaN,epci,Auvergne-Rhône-Alpes,métropole,01,01035,Belleydoux,332.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,OUI,NaN,NaN


In [115]:
# noms des colonnes
for j in b_16.columns :
    print(j)

Code INSEE
modification
Structure application
Région
métro DOM
Dép
Code INSEE.1
Commune
Population commune 2012 (publiée 2015)
Pop>=seuil (1500-3500)
Taux légal de logements locatifs sociaux applicable au 01/01/2015
Code INSEE nouvelle commune 2016
Nom nouvelle commune 2016
Nb communes fusionnées
Popmun_2016_DGCL_INSEE_popmun2012_Mayotte
Code Agglo
Nom Agglo
Dep. Agglo
Population Agglo
Taux LLS agglo applicable au 01/01/2015
Code EPCI
Nom EPCI
Dep. EPCI
Population EPCI
Taux LLS EPCI applicable au 01/01/2015
Différence taux agglo / taux EPCI (1 si oui et 2 si non)
Zonage_ABC_octobre_2014
Parc national
Parc régional
RP 2015
PFH 2015
Montant DSU 2015
Situation de la commune par rapport L.302-5 CCH
Exemption
Décrois. Démo
SERV, PEB, PPR ou autre
Communes exonérées du prélèvement (1) et communes non prélevables (2)
Exonérée de prélèvement (loi Notré)
DSU et >15%
>25% (>20%) 2015
Nombre logements locatifs sociaux 2015
Taux logements locatifs sociaux 2015
Catégorie 1
Catégorie 2
Catégorie 3
C

In [116]:
b_16['Situation de la commune par rapport L.302-5 CCH'].unique()

array(['non soumise', 'soumise non prélevée', 'soumise prélevée',
       'soumise exonérée', 'soumise non prélevable'], dtype=object)

In [117]:
b_16.replace('non soumise', 'Non soumise', inplace = True)
b_16.replace('soumise non prélevée', 'Soumise non prélevée', inplace = True)
b_16.replace('soumise prélevée', 'Soumise prélevée', inplace = True)
b_16.replace('soumise exonérée', 'Soumise exonérée', inplace = True)
b_16.replace('soumise non prélevable', 'Soumise non prélevée', inplace = True)

In [118]:
data = [] # variable temporaire pour stocker les données

for i in range(len(b_16)) : # pour chaque commune
    # code commune :
    data.append(str(b_16['Code INSEE'][i]).zfill(5))
    
    # code agglo et epci:
    if b_16["Structure application"][i] == 'epci' :
        data.append(np.nan)
        data.append(str(int(b_16["Code EPCI"][i])))
    elif b_16["Structure application"][i] == 'agglo' :
        data.append(b_16['Code Agglo'][i])
        data.append(np.nan)
    elif b_16["Structure application"][i] == 'agglo/epci' :
        data.append(b_16['Code Agglo'][i])
        data.append(str(int(b_16["Code EPCI"][i])))
    elif b_16["Structure application"][i] == 'isolé' :
        data.append(np.nan)
        data.append(np.nan)
    else :
        print(b_16.iloc[i])
    
    # taux ls commune :
    if type(b_16['Taux logements locatifs sociaux 2015'][i]) == float :
        data.append(b_16['Taux logements locatifs sociaux 2015'][i])
    else :
        data.append(float(b_16['Taux logements locatifs sociaux 2015'][i].replace(',', '.')[:-1])/100)
        
    # situation
    data.append(b_16['Situation de la commune par rapport L.302-5 CCH'][i])
    
    # nombre de logements sociaux
    data.append(b_16["Nombre logements locatifs sociaux 2015"][i])
    
data = np.array(data)
colonnes = ['code_commune', 'code_agglo_16', 'code_epci_16', 'taux_ls_16', 'situation_16', 'LLS_16']
data = np.reshape(data, (len(b_16), len(colonnes)))
df_16 = pd.DataFrame(data, columns = colonnes) # création du DataFrame du bilan 2016
df_16.taux_ls_16 = pd.to_numeric(df_16.taux_ls_16, errors = 'coerce')
df_16.head()

,code_commune,code_agglo_16,code_epci_16,taux_ls_16,situation_16,LLS_16
0,01011,nan,200042935,0.0738,Non soumise,11.0
1,01014,nan,200042935,0.2981,Non soumise,400.0
2,01030,00758,nan,0.2876,Non soumise,107.0
3,01031,nan,200042935,0.4997,Non soumise,853.0
4,01035,nan,200042935,0.0000,Non soumise,0.0


In [119]:
# ajoût des colonnes en rapport avec les unités urbaines et EPCI
df_16['taux_agglo_16'] = np.nan
df_16['taux_epci_16'] = np.nan
df_16['rp_agglo_16'] = np.nan
df_16['rp_epci_16'] = np.nan
df_16['rp_agglo_nc_16'] = np.nan
df_16['rp_epci_nc_16'] = np.nan
df_16.head()

,code_commune,code_agglo_16,code_epci_16,taux_ls_16,situation_16,LLS_16,taux_agglo_16,taux_epci_16,rp_agglo_16,rp_epci_16,rp_agglo_nc_16,rp_epci_nc_16
0,01011,nan,200042935,0.0738,Non soumise,11.0,NaN,NaN,NaN,NaN,NaN,NaN
1,01014,nan,200042935,0.2981,Non soumise,400.0,NaN,NaN,NaN,NaN,NaN,NaN
2,01030,00758,nan,0.2876,Non soumise,107.0,NaN,NaN,NaN,NaN,NaN,NaN
3,01031,nan,200042935,0.4997,Non soumise,853.0,NaN,NaN,NaN,NaN,NaN,NaN
4,01035,nan,200042935,0.0000,Non soumise,0.0,NaN,NaN,NaN,NaN,NaN,NaN


In [120]:
agglos = b_16['Code Agglo'].unique()[1:]
# b_16['Code agglo'].unique() contient la liste des codes des agglomérations présentes dans le bilan de 2016.
# Le premier élément de la liste est occupée par '00000' qui correspond à la commune ne fait pas partie d'une UU
# concernée par l'article 55 de la loi SRU donc on ne parcourt la liste des UU que pour les éléments suivants.

# On va calculer les taux de logements sociaux pour chaque agglomération
# Ainsi que le nombre total de résidences principales de chaque agglomération
for agglo in agglos : # Pour chaque agglomération
    df = b_16[b_16['Code Agglo'] == agglo] # Toutes les données sur les communes de l'agglomération
    
    nb_com_agglo = len(df) # le nombre de communes de l'agglomération
    nb_com_nc = len(df[df['Nombre logements locatifs sociaux 2015'] == 0])
                 # le nombre de communes de l'EPCI, pour laquelle il manque le nombre de logements sociaux
    
    nb_rp = int(sum(df['RP 2015'])) # le nombre de résidences principales de l'agglomération
    nb_ls = 0 # le nombre de logements sociaux de l'agglomération
    for i in df['Nombre logements locatifs sociaux 2015'] :
        nb_ls = nb_ls + int(i)
    nb_rp_nc = int(sum(df['RP 2015'][df['Nombre logements locatifs sociaux 2015'] == 0])) 
                 # le nombre de résidences principales de l'agglomération qui sont potentiellement des logements sociaux sans 
                 # être comptées comme tel (pas d'informations sur les logements sociaux dans la commune)
    
    com = df.index # l'index des communes de l'agglomération
    
    # On remplit notre base de données avec les valeurs calculées.
    for commune in com : 
        df_16['taux_agglo_16'][commune] = nb_ls/nb_rp # le taux de logement sociaux de l'agglomération
        df_16['rp_agglo_16'][commune] = nb_rp
        df_16['rp_agglo_nc_16'][commune] = nb_rp_nc
df_16.head(10)

,code_commune,code_agglo_16,code_epci_16,taux_ls_16,situation_16,LLS_16,taux_agglo_16,taux_epci_16,rp_agglo_16,rp_epci_16,rp_agglo_nc_16,rp_epci_nc_16
0,01011,nan,200042935,0.0738,Non soumise,11.0,NaN,NaN,NaN,NaN,NaN,NaN
1,01014,nan,200042935,0.2981,Non soumise,400.0,NaN,NaN,NaN,NaN,NaN,NaN
2,01030,00758,nan,0.2876,Non soumise,107.0,0.228420,NaN,739886.0,NaN,2042.0,NaN
3,01031,nan,200042935,0.4997,Non soumise,853.0,NaN,NaN,NaN,NaN,NaN,NaN
4,01035,nan,200042935,0.0000,Non soumise,0.0,NaN,NaN,NaN,NaN,NaN,NaN
5,01043,00758,nan,0.0588,Soumise non prélevée,104.0,0.228420,NaN,739886.0,NaN,2042.0,NaN
6,01049,00758,nan,0.0192,Non soumise,21.0,0.228420,NaN,739886.0,NaN,2042.0,NaN
7,01051,nan,200042935,0.0833,Non soumise,4.0,NaN,NaN,NaN,NaN,NaN,NaN
8,01053,01501,240100628,0.4067,Non soumise,7988.0,0.340432,NaN,27656.0,NaN,0.0,NaN
9,01060,nan,200042935,0.1484,Non soumise,38.0,NaN,NaN,NaN,NaN,NaN,NaN


In [121]:
code_epci = np.delete(b_16['Code EPCI'].unique(), 1)
# b_16['Code EPCI'].unique() contient la liste des codes des EPCI présents dans la base de données de 2016
# La première position est occupée par '0' (lorsque l'EPCI de la commune n'est pas concernée par la loi SRU).
# On enlève donc l'élément en deuxième position de la liste pour ne garder que les codes des EPCI.

# On va calculer les taux de logements sociaux pour chaque EPCI
# Ainsi que le nombre total de résidences principales de chaque EPCI
for epci in code_epci : # Pour chaque EPCI
    df = b_16[b_16['Code EPCI'] == epci] # Toutes les données sur les communes de l'EPCI
    
    nb_com_epci = len(df) # le nombre de communes de l'EPCI
    nb_com_nc = len(df[df['Nombre logements locatifs sociaux 2015'] == 0])
                 # le nombre de communes de l'EPCI, pour laquelle il manque le nombre de logements sociaux
    
    nb_rp = int(sum(df['RP 2015'])) # le nombre de résidences principales de l'EPCI
    nb_ls = 0 # le nombre de logements sociaux de l'EPCI
    for i in df['Nombre logements locatifs sociaux 2015'] :
        nb_ls = nb_ls + int(i)
    nb_rp_nc = int(sum(df['RP 2015'][df['Nombre logements locatifs sociaux 2015'] == 0])) 
                 # le nombre de résidences principales de l'EPCI qui sont potentiellement des logements sociaux sans 
                 # être comptées comme tel (pas d'informations sur les logements sociaux dans la commune)
    
    com = df.index # l'index des communes de l'epci
    
    # On remplit notre base de données avec les valeurs calculées.
    for commune in com : 
        df_16['taux_epci_16'][commune] = nb_ls/nb_rp # le taux de logement sociaux de l'epci
        df_16['rp_epci_16'][commune] = nb_rp
        df_16['rp_epci_nc_16'][commune] = nb_rp_nc
df_16.head(10)

,code_commune,code_agglo_16,code_epci_16,taux_ls_16,situation_16,LLS_16,taux_agglo_16,taux_epci_16,rp_agglo_16,rp_epci_16,rp_agglo_nc_16,rp_epci_nc_16
0,01011,nan,200042935,0.0738,Non soumise,11.0,NaN,0.323922,NaN,24333.0,NaN,514.0
1,01014,nan,200042935,0.2981,Non soumise,400.0,NaN,0.323922,NaN,24333.0,NaN,514.0
2,01030,00758,nan,0.2876,Non soumise,107.0,0.228420,NaN,739886.0,NaN,2042.0,NaN
3,01031,nan,200042935,0.4997,Non soumise,853.0,NaN,0.323922,NaN,24333.0,NaN,514.0
4,01035,nan,200042935,0.0000,Non soumise,0.0,NaN,0.323922,NaN,24333.0,NaN,514.0
5,01043,00758,nan,0.0588,Soumise non prélevée,104.0,0.228420,NaN,739886.0,NaN,2042.0,NaN
6,01049,00758,nan,0.0192,Non soumise,21.0,0.228420,NaN,739886.0,NaN,2042.0,NaN
7,01051,nan,200042935,0.0833,Non soumise,4.0,NaN,0.323922,NaN,24333.0,NaN,514.0
8,01053,01501,240100628,0.4067,Non soumise,7988.0,0.340432,0.305924,27656.0,32969.0,0.0,0.0
9,01060,nan,200042935,0.1484,Non soumise,38.0,NaN,0.323922,NaN,24333.0,NaN,514.0


## 14. Bilan 2017

135 communes atteignant le taux légal mais avec 0 logement social\
1 commune exemptée avec 0 logemement social

In [122]:
# chargement des données du bilan 2017
b_17 = pd.read_csv("../donnees_initiales/bilans_annuels/ART_55_Bilan_SRU_2017_art55.csv",
                   sep = ';')
b_17.head()

,Nom de la commune,Dép com,Code INSEE,Région,Métropole – DOM,Structure application,Population commune,Pop>=seuil (1500-3500),Taux de LLS à atteindre,Code Agglo,...,Reliquat à reporter,Nb LLS très social financement Etat (en service au 01/01/2002),Nb LLS social financement Etat (en service au 01/01/2002),Nb LLS intermédiaire financement Etat (en service au 01/01/2002),Nb LLS autres financement Etat (en service au 01/01/2002),Nb LLS très social financement ANAH (en service au 01/01/2002),Nb LLS social financement ANAH (en service au 01/01/2002),Nb LLS autres financement ANAH (en service au 01/01/2002),Visée,Observations
0,Apremont,1,1011,Auvergne-Rhône Alpes,Métropole,EPCI,394,0,20,0,...,0,0,8,0,0,0,0,0,OUI,NaN
1,Arbent,1,1014,Auvergne-Rhône Alpes,Métropole,EPCI,3410,0,20,0,...,0,0,69,19,0,0,0,0,OUI,NaN
2,Beauregard,1,1030,Auvergne-Rhône Alpes,Métropole,Agglo,876,0,25,758,...,0,0,22,0,0,2,0,0,OUI,NaN
3,Bellignat,1,1031,Auvergne-Rhône Alpes,Métropole,EPCI,3646,1,20,0,...,0,0,16,46,0,0,0,0,OUI,NaN
4,Belleydoux,1,1035,Auvergne-Rhône Alpes,Métropole,EPCI,327,0,20,0,...,0,0,2,0,0,0,0,0,OUI,NaN


In [123]:
# changement du type du code des EPCI
for i in range(len(b_17)) :
    b_17['Code EPCI'][i] = str(b_17['Code EPCI'][i])

In [124]:
# noms des colonnes
for j in b_17.columns :
    print(j)

Nom de la commune
Dép com
Code INSEE
Région
Métropole – DOM
Structure application
Population commune
Pop>=seuil (1500-3500)
Taux de LLS à atteindre
Code Agglo
Nom Agglo
Dep. Agglo
Population Agglo
Taux de LLS de l'agglo
Code EPCI
Nom EPCI
Dep. EPCI
Population EPCI
Taux de LLS de l'EPCI
Nbe de RP
PFH
Situation de la commune par rapport au L.302-5 du CCH
Décrois. Démo
Exonérée de prélèvement (loi Notré et EC)
DSU et >15%
SERV, PEB ou PPR
>25% (>20%)
Total LLS
total_lls_new
Taux LLS 2016
Catégorie 1
Catégorie 2
Catégorie 3
Catégorie 3a
Catégorie 4
Logts déconventionnés
Logts vendus aux locataires
Inventaire réalisé
Notif réalisée
5/100 Dép Fonct°
% DRF
Prélèvement brut
Majoration
Total prélèvement brut
Constat de carence
Date de l'A.P.
Taux de maj.
Conv Préfet org HLM.
Délégataire du droit de préemption
Nb conventions SEM
Nb DIA examiné SEM
Nb DIA aboutit SEM
Nb LLS programmé SEM
Nb conventions bailleur
Nb DIA examiné bailleur
Nb DIA aboutit bailleur
Nb LLS programmé bailleur
Nb conventio

In [125]:
b_17['Situation de la commune par rapport au L.302-5 du CCH'].unique()

array(['Non soumise sous seuil pop', 'Non soumise atteinte tx légal',
       'Soumise prélèvement nul', 'Soumise prélevée',
       'Soumise non prélevée', 'Soumise exonérée',
       'Non soumise exemption'], dtype=object)

In [126]:
b_17.replace('Non soumise atteinte tx légal', 'Non soumise atteint taux légal', inplace = True)
b_17.replace('Non soumise exemption', 'Non soumise exemptée', inplace = True)

In [127]:
data = [] # variable temporaire pour stocker les données
a = 0 # variable de vérification

for i in range(len(b_17)) : # pour chaque commune
    data.append(b_17['Code INSEE'][i].zfill(5)) # code de la commune
    
    # codes unité urbaine et EPCI
    if b_17['Structure application'][i] == 'EPCI' :
        data.append(np.nan)
        data.append(b_17['Code EPCI'][i].replace('\xa0', ''))
    elif b_17['Structure application'][i] == 'Agglo/EPCI' :
        data.append(b_17['Code Agglo'][i].zfill(5))
        data.append(b_17['Code EPCI'][i].replace('\xa0', ''))
    elif b_17['Structure application'][i] == 'Agglo' :
        data.append(b_17['Code Agglo'][i].zfill(5))
        data.append(np.nan)
    else :
        data.append(np.nan)
        data.append(np.nan)
        
    # taux de logements sociaux
    if b_17['Taux LLS 2016'][i][0] == 'N' :
        data.append(np.nan)
    else :
        data.append(float(b_17['Taux LLS 2016'][i].replace(',', '.').replace('%', ''))/100)
    
    # situation
    data.append(b_17['Situation de la commune par rapport au L.302-5 du CCH'][i])
    
    # nombre de logements sociaux
    data.append(b_17["Total LLS"][i])
    
# print(a)
data = np.array(data)
colonnes = ['code_commune', 'code_agglo_17', 'code_epci_17', 'taux_ls_17', 'situation_17', 'LLS_17']
data = np.reshape(data, (len(b_17), len(colonnes)))
df_17 = pd.DataFrame(data, columns = colonnes) # création du DataFrame du bilan 2017
df_17.taux_ls_17 = pd.to_numeric(df_17.taux_ls_17, errors = 'coerce')
df_17.head()

,code_commune,code_agglo_17,code_epci_17,taux_ls_17,situation_17,LLS_17
0,01011,nan,200042935,0.0738,Non soumise sous seuil pop,11
1,01014,nan,200042935,0.2957,Non soumise sous seuil pop,395
2,01030,00758,nan,0.2793,Non soumise sous seuil pop,105
3,01031,nan,200042935,0.4968,Non soumise atteint taux légal,852
4,01035,nan,200042935,0.0128,Non soumise sous seuil pop,2


In [128]:
# création des colonnes des EPCI et agglomérations
df_17['taux_agglo_17'] = np.nan
df_17['taux_epci_17'] = np.nan
df_17['rp_agglo_17'] = np.nan
df_17['rp_epci_17'] = np.nan
df_17['rp_agglo_nc_17'] = np.nan
df_17['rp_epci_nc_17'] = np.nan
df_17.head()

,code_commune,code_agglo_17,code_epci_17,taux_ls_17,situation_17,LLS_17,taux_agglo_17,taux_epci_17,rp_agglo_17,rp_epci_17,rp_agglo_nc_17,rp_epci_nc_17
0,01011,nan,200042935,0.0738,Non soumise sous seuil pop,11,NaN,NaN,NaN,NaN,NaN,NaN
1,01014,nan,200042935,0.2957,Non soumise sous seuil pop,395,NaN,NaN,NaN,NaN,NaN,NaN
2,01030,00758,nan,0.2793,Non soumise sous seuil pop,105,NaN,NaN,NaN,NaN,NaN,NaN
3,01031,nan,200042935,0.4968,Non soumise atteint taux légal,852,NaN,NaN,NaN,NaN,NaN,NaN
4,01035,nan,200042935,0.0128,Non soumise sous seuil pop,2,NaN,NaN,NaN,NaN,NaN,NaN


In [129]:
agglos = b_17['Code Agglo'].unique()[1:]
# b_17['Code agglo'].unique() contient la liste des codes des agglomérations présentes dans le bilan de 2017.
# Le premier élément de la liste est occupée par '0' qui correspond à la commune ne fait pas partie d'une UU
# concernée par l'article 55 de la loi SRU donc on ne parcourt la liste des UU que pour les éléments suivants.

# On va calculer les taux de logements sociaux pour chaque agglomération
# Ainsi que le nombre total de résidences principales de chaque agglomération
for agglo in agglos : # Pour chaque agglomération
    df = b_17[b_17['Code Agglo'] == agglo] # Toutes les données sur les communes de l'agglomération
    
    nb_com_agglo = len(df) # le nombre de communes de l'agglomération
    nb_com_nc = len(df[df.total_lls_new == 0])
                 # le nombre de communes de l'agglomération, pour laquelle il manque le nombre de logements sociaux
    
    nb_rp = sum(df['Nbe de RP']) # le nombre de résidences principales de l'agglomération
    nb_ls = sum(df.total_lls_new) # le nombre de logements sociaux de l'agglomération
    nb_rp_nc = 0 # le nombre de résidences principales de l'agglomération qui sont potentiellement des logements sociaux sans 
                 # être comptés comme tel (pas d'informations sur les logements sociaux dans la commune)
    for i in df['Nbe de RP'][df.total_lls_new == 0] :
        nb_rp_nc = nb_rp_nc + int(i)
    com = df.index
    
    # On remplit notre base de données avec les valeurs calculées.
    for commune in com : 
        df_17['taux_agglo_17'][commune] = nb_ls/nb_rp # le taux de logement sociaux de l'agglomération
        df_17['rp_agglo_17'][commune] = nb_rp
        df_17['rp_agglo_nc_17'][commune] = nb_rp_nc
df_17.head(10)

,code_commune,code_agglo_17,code_epci_17,taux_ls_17,situation_17,LLS_17,taux_agglo_17,taux_epci_17,rp_agglo_17,rp_epci_17,rp_agglo_nc_17,rp_epci_nc_17
0,01011,nan,200042935,0.0738,Non soumise sous seuil pop,11,NaN,NaN,NaN,NaN,NaN,NaN
1,01014,nan,200042935,0.2957,Non soumise sous seuil pop,395,NaN,NaN,NaN,NaN,NaN,NaN
2,01030,00758,nan,0.2793,Non soumise sous seuil pop,105,0.233284,NaN,746879.0,NaN,2065.0,NaN
3,01031,nan,200042935,0.4968,Non soumise atteint taux légal,852,NaN,NaN,NaN,NaN,NaN,NaN
4,01035,nan,200042935,0.0128,Non soumise sous seuil pop,2,NaN,NaN,NaN,NaN,NaN,NaN
5,01043,00758,nan,0.0747,Soumise prélèvement nul,131,0.233284,NaN,746879.0,NaN,2065.0,NaN
6,01049,00758,nan,0.0186,Non soumise sous seuil pop,21,0.233284,NaN,746879.0,NaN,2065.0,NaN
7,01051,nan,200042935,0.0930,Non soumise sous seuil pop,4,NaN,NaN,NaN,NaN,NaN,NaN
8,01053,01501,240100628,0.4058,Non soumise atteint taux légal,8069,0.340601,NaN,28068.0,NaN,0.0,NaN
9,01060,nan,200042935,0.1467,Non soumise sous seuil pop,38,NaN,NaN,NaN,NaN,NaN,NaN


In [130]:
code_epci = np.delete(b_17['Code EPCI'].unique(), 1)
# b_17['Code EPCI'].unique() contient la liste des codes des EPCI présents dans la base de données de 2017.
# La première position est occupée par '0000000' (lorsque l'EPCI de la commune n'est pas concernée par la loi SRU).
# On enlève donc l'élément en deuxième position de la liste pour ne garder que les codes des EPCI.

# On va calculer les taux de logements sociaux pour chaque EPCI
# Ainsi que le nombre total de résidences principales de chaque EPCI
for epci in code_epci : # Pour chaque EPCI
    df = b_17[b_17['Code EPCI'].replace('\xa0', '') == epci.replace('\xa0', '')] # Toutes les données sur les communes de l'EPCI
    
    nb_com_epci = len(df) # le nombre de communes de l'EPCI
    nb_com_nc = len(df[df.total_lls_new == 0])
                 # le nombre de communes de l'EPCI, pour laquelle il manque le nombre de logements sociaux
    
    nb_rp = sum(df['Nbe de RP']) # le nombre de résidences principales de l'EPCI
    nb_ls = sum(df.total_lls_new) # le nombre de logements sociaux de l'EPCI
    nb_rp_nc = 0 # le nombre de résidences principales de l'EPCI qui sont potentiellement des logements sociaux sans 
                 # être comptés comme tel (pas d'informations sur les logements sociaux dans la commune)
    for i in df['Nbe de RP'][df.total_lls_new == 0] :
        nb_rp_nc = nb_rp_nc + int(i)
    
    com = df.index # l'index des communes de l'epci
    
    # On remplit notre base de données avec les valeurs calculées.
    for commune in com : 
        df_17['taux_epci_17'][commune] = nb_ls/nb_rp # le taux de logement sociaux de l'epci
        df_17['rp_epci_17'][commune] = nb_rp
        df_17['rp_epci_nc_17'][commune] = nb_rp_nc
df_17.head(10)

,code_commune,code_agglo_17,code_epci_17,taux_ls_17,situation_17,LLS_17,taux_agglo_17,taux_epci_17,rp_agglo_17,rp_epci_17,rp_agglo_nc_17,rp_epci_nc_17
0,01011,nan,200042935,0.0738,Non soumise sous seuil pop,11,NaN,0.324302,NaN,24397.0,NaN,363.0
1,01014,nan,200042935,0.2957,Non soumise sous seuil pop,395,NaN,0.324302,NaN,24397.0,NaN,363.0
2,01030,00758,nan,0.2793,Non soumise sous seuil pop,105,0.233284,NaN,746879.0,NaN,2065.0,NaN
3,01031,nan,200042935,0.4968,Non soumise atteint taux légal,852,NaN,0.324302,NaN,24397.0,NaN,363.0
4,01035,nan,200042935,0.0128,Non soumise sous seuil pop,2,NaN,0.324302,NaN,24397.0,NaN,363.0
5,01043,00758,nan,0.0747,Soumise prélèvement nul,131,0.233284,NaN,746879.0,NaN,2065.0,NaN
6,01049,00758,nan,0.0186,Non soumise sous seuil pop,21,0.233284,NaN,746879.0,NaN,2065.0,NaN
7,01051,nan,200042935,0.0930,Non soumise sous seuil pop,4,NaN,0.324302,NaN,24397.0,NaN,363.0
8,01053,01501,240100628,0.4058,Non soumise atteint taux légal,8069,0.340601,0.307587,28068.0,33519.0,0.0,0.0
9,01060,nan,200042935,0.1467,Non soumise sous seuil pop,38,NaN,0.324302,NaN,24397.0,NaN,363.0


In [131]:
# vérification des valeurs aberrantes
df_17[df_17.taux_ls_17 > .7]

,code_commune,code_agglo_17,code_epci_17,taux_ls_17,situation_17,LLS_17,taux_agglo_17,taux_epci_17,rp_agglo_17,rp_epci_17,rp_agglo_nc_17,rp_epci_nc_17
1402,27701,nan,200035665,0.7801,Non soumise atteint taux légal,3380,NaN,0.269643,NaN,27414.0,NaN,11835.0
3035,57058,57501,245700372,0.8522,Non soumise atteint taux légal,2208,0.311773,0.242239,36642.0,33789.0,499.0,2271.0
3391,59386,59702,245900410,0.7764,Non soumise atteint taux légal,3188,0.228562,0.221598,449024.0,488060.0,15807.0,27485.0
4836,78644,00851,200058782,0.7181,Non soumise atteint taux légal,1490,0.273503,0.344714,4617846.0,89042.0,9363.0,0.0
5352,91235,00851,200057859,0.8275,Non soumise atteint taux légal,1947,0.273503,0.245588,4617846.0,76388.0,9363.0,243.0
5479,93030,00851,200054781,0.7187,Non soumise atteint taux légal,2859,0.273503,0.276245,4617846.0,3165560.0,9363.0,0.0
5512,94011,00851,200054781,0.7528,Non soumise atteint taux légal,4642,0.273503,0.276245,4617846.0,3165560.0,9363.0,0.0
5654,97120,9A701,200018653,0.7081,Non soumise atteint taux légal,4733,0.241675,0.404234,103718.0,42609.0,0.0,0.0


## 15. Bilan 2018

146 communes atteignant taux légal avec 0 logement social\
6 communes exemptée avec 0 logement social

In [132]:
# chargement des données du bilan de 2018
b_18 = pd.read_csv("../donnees_initiales/bilans_annuels/ART_55_Bilan_SRU_2018_toutes_communes.csv",
                   sep = ';')[:7809]
b_18.tail()

,Nom de la commune,Code INSEE,Département,Région,Métropole – DOM,Structure application,Population commune 2014,Pop>=seuil (1500-3500),Taux de LLS à atteindre,Vérif taux,...,Nb LLS intermédiaire financement Etat (en service au 01/01/2002),Nb LLS autres financement Etat (en service au 01/01/2002),Nb LLS très social financement ANAH (en service au 01/01/2002),Nb LLS social financement ANAH (en service au 01/01/2002),Nb LLS autres financement ANAH (en service au 01/01/2002),Signature d'un CMS,date de signature,Durée du CMS,Visée,Observations
7804,Brando,2B043,2B,Corse,métropole,agglo,1674.0,0.0,25 %,25 %,...,0.0,2.0,1.0,0.0,0.0,0.0,NaN,NaN,OUI,Commune de moins de 3500 habitants donc pas d'...
7805,Furiani,2B120,2B,Corse,métropole,agglo/epci,5782.0,1.0,25 %,25 %,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,OUI,NaN
7806,San-Martino-di-Lota,2B305,2B,Corse,métropole,agglo/epci,2915.0,0.0,25 %,25 %,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,OUI,Commune de moins de 3500 habitants donc pas d'...
7807,Santa-Maria-di-Lota,2B309,2B,Corse,métropole,agglo/epci,1813.0,0.0,25 %,25 %,...,0.0,0.0,0.0,1.0,0.0,0.0,NaN,NaN,OUI,Commune de moins de 3500 habitants donc pas d'...
7808,Ville-di-Pietrabugno,2B353,2B,Corse,métropole,agglo/epci,3465.0,0.0,25 %,25 %,...,0.0,2.0,0.0,0.0,0.0,0.0,NaN,NaN,OUI,Commune retombée à moins de 3500 habitants don...


In [133]:
# noms des colonnes
for j in b_18.columns :
    print(j)

Nom de la commune
Code INSEE
Département
Région
Métropole – DOM
Structure application
Population commune 2014
Pop>=seuil (1500-3500)
Taux de LLS à atteindre
Vérif taux
Unnamed: 10
Code Agglo
Nom Agglo
Dep. Agglo
Population Agglo
Taux de LLS de l'agglo
Code EPCI
Nom EPCI
Dep. EPCI
Population EPCI
Taux de LLS de l'EPCI
Nbe de RP
PFH
Situation de la commune par rapport au L.302-5 du CCH
exemption décret
Exonérée de prélèvement (soumise 1ère fois)
DSU
Verif DSU
>25% (>20%)
Total LLS
total_lls_new
Taux LLS
Catégorie 1
Catégorie 2
Catégorie 3
Catégorie 3a
Catégorie 4
Logts déconventionnés
Logts vendus aux locataires
Inventaire réalisé
Notif réalisée
% DRF
Montant DRF correspondant
Prelevement brut sans majoration
Majoration
Total prélèvement brut
Constat de carence
Date de l'A.P.
Taux de maj.
Dépenses déductibles communes
Dépenses indûment déduites
Reliquat disponible
Bénéficiaires locaux
Prélèvement à destination des bénéficiaires locaux
Majoration à destination du fonds national
Reliquat r

In [134]:
b_18['Situation de la commune par rapport au L.302-5 du CCH'].unique()

array(['Non soumise sous seuil pop', 'Non soumise atteinte taux légal',
       'Soumise prélevée', 'Soumise non prélevée',
       'Soumise prélèvement nul', 'Non soumise exemptée',
       'Soumise exonérée'], dtype=object)

In [135]:
b_18.replace('Non soumise atteinte taux légal', 'Non soumise atteint taux légal', inplace = True)

In [136]:
data = [] # variable temporaire pour stocker les données
a = 0 # variable de vérification

for i in range(len(b_18)) : # pour chaque commune
    # code commune
    data.append(b_18['Code INSEE'][i].zfill(5))
    
    # code agglo et/ou EPCI
    if b_18['Structure application'][i] == 'epci' :
        data.append(np.nan)
        data.append(str(int(b_18['Code EPCI'][i])))
    elif b_18['Structure application'][i] == 'agglo/epci' :
        data.append(str(b_18['Code Agglo'][i]).zfill(5))
        data.append(str(int(b_18['Code EPCI'][i])))
    elif b_18['Structure application'][i] == 'agglo' :
        data.append(str(b_18['Code Agglo'][i]).zfill(5))
        data.append(np.nan)
    else :
        a = a + 1
        
    # taux commune 
    data.append(float(b_18.total_lls_new[i])/b_18['Nbe de RP'][i])
    
    # situation
    data.append(b_18['Situation de la commune par rapport au L.302-5 du CCH'][i])
    
    # nombre de logements sociaux
    data.append(b_18["Total LLS"][i])
    
# print(a)
data = np.array(data)
colonnes = ['code_commune', 'code_agglo_18', 'code_epci_18', 'taux_ls_18', 'situation_18', 'LLS_18']
data = np.reshape(data, (len(b_18), len(colonnes)))
df_18 = pd.DataFrame(data, columns = colonnes) # création du DataFrame du bilan 2018
df_18.taux_ls_18 = pd.to_numeric(df_18.taux_ls_18, errors = 'coerce')
df_18.head()

,code_commune,code_agglo_18,code_epci_18,taux_ls_18,situation_18,LLS_18
0,01011,nan,200042935,0.0,Non soumise sous seuil pop,0
1,01014,nan,200042935,0.0,Non soumise sous seuil pop,0
2,01024,nan,200071751,0.0,Non soumise sous seuil pop,0
3,01029,nan,200071751,0.0,Non soumise sous seuil pop,0
4,01030,00758,nan,0.0,Non soumise sous seuil pop,0


In [137]:
# création des colonnes des agglomérations et EPCI
df_18['taux_agglo_18'] = np.nan
df_18['taux_epci_18'] = np.nan
df_18['rp_agglo_18'] = np.nan
df_18['rp_epci_18'] = np.nan
df_18['rp_agglo_nc_18'] = np.nan
df_18['rp_epci_nc_18'] = np.nan
df_18.head()

,code_commune,code_agglo_18,code_epci_18,taux_ls_18,situation_18,LLS_18,taux_agglo_18,taux_epci_18,rp_agglo_18,rp_epci_18,rp_agglo_nc_18,rp_epci_nc_18
0,01011,nan,200042935,0.0,Non soumise sous seuil pop,0,NaN,NaN,NaN,NaN,NaN,NaN
1,01014,nan,200042935,0.0,Non soumise sous seuil pop,0,NaN,NaN,NaN,NaN,NaN,NaN
2,01024,nan,200071751,0.0,Non soumise sous seuil pop,0,NaN,NaN,NaN,NaN,NaN,NaN
3,01029,nan,200071751,0.0,Non soumise sous seuil pop,0,NaN,NaN,NaN,NaN,NaN,NaN
4,01030,00758,nan,0.0,Non soumise sous seuil pop,0,NaN,NaN,NaN,NaN,NaN,NaN


In [138]:
f = 0 # variable de vérification
agglos = b_18['Code Agglo'].unique()[1:]
# b_18['Code agglo'].unique() contient la liste des codes des agglomérations présentes dans le bilan de 2018.
# Le premier élément de la liste est occupée par 'nan' qui correspond à la commune ne fait pas partie d'une UU
# concernée par l'article 55 de la loi SRU donc on ne parcourt la liste des UU que pour les éléments suivants.

# On va calculer les taux de logements sociaux pour chaque agglomération
# Ainsi que le nombre total de résidences principales de chaque agglomération
for agglo in agglos : # Pour chaque agglomération
    df = b_18[b_18['Code Agglo'] == agglo] # Toutes les données sur les communes de l'agglomération
    #print(df[['Nbe de RP', 'Situation de la commune par rapport au L.302-5 du CCH']][df['Total LLS'] == 'NR'])
    
    nb_com_agglo = len(df) # le nombre de communes de l'agglomération
    nb_com_nc = len(df[df.total_lls_new == 0])
                 # le nombre de communes de l'agglomération, pour laquelle il manque le nombre de logements sociaux
    
    nb_rp = sum(df['Nbe de RP']) # le nombre de résidences principales de l'agglomération
    nb_ls = sum(df.total_lls_new) # le nombre de logements sociaux de l'agglomération
    nb_rp_nc = 0 # le nombre de résidences principales de l'agglomération qui sont potentiellement des logements sociaux sans 
                 # être comptés comme tel (pas d'informations sur les logements sociaux dans la commune)
    for i in df['Nbe de RP'][df.total_lls_new == 0] :
        nb_rp_nc = nb_rp_nc + int(i)
    for i in + df['Nbe de RP'][df['Total LLS'] == 'NR'] : # il y a une commune exemptée pour laquelle le nombre de LLS n'a pas été compté
        nb_rp_nc = nb_rp_nc + int(i)
        f = f + int(i)
    
    com = df.index # l'index des communes de l'agglomeration
    
    # On remplit notre base de données avec les valeurs calculées.
    for commune in com : 
        df_18['taux_agglo_18'][commune] = nb_ls/nb_rp # le taux de logement sociaux de l'agglomeration
        df_18['rp_agglo_18'][commune] = nb_rp
        df_18['rp_agglo_nc_18'][commune] = nb_rp_nc
df_18.head(10)

,code_commune,code_agglo_18,code_epci_18,taux_ls_18,situation_18,LLS_18,taux_agglo_18,taux_epci_18,rp_agglo_18,rp_epci_18,rp_agglo_nc_18,rp_epci_nc_18
0,01011,nan,200042935,0.000000,Non soumise sous seuil pop,0,NaN,NaN,NaN,NaN,NaN,NaN
1,01014,nan,200042935,0.000000,Non soumise sous seuil pop,0,NaN,NaN,NaN,NaN,NaN,NaN
2,01024,nan,200071751,0.000000,Non soumise sous seuil pop,0,NaN,NaN,NaN,NaN,NaN,NaN
3,01029,nan,200071751,0.000000,Non soumise sous seuil pop,0,NaN,NaN,NaN,NaN,NaN,NaN
4,01030,00758,nan,0.000000,Non soumise sous seuil pop,0,0.233513,NaN,755547.0,NaN,10577.0,NaN
5,01031,nan,200042935,0.502358,Non soumise atteint taux légal,852,NaN,NaN,NaN,NaN,NaN,NaN
6,01035,nan,200042935,0.000000,Non soumise sous seuil pop,0,NaN,NaN,NaN,NaN,NaN,NaN
7,01038,nan,200071751,0.000000,Non soumise sous seuil pop,0,NaN,NaN,NaN,NaN,NaN,NaN
8,01040,nan,200071751,0.000000,Non soumise sous seuil pop,0,NaN,NaN,NaN,NaN,NaN,NaN
9,01043,00758,nan,0.072778,Soumise prélevée,131,0.233513,NaN,755547.0,NaN,10577.0,NaN


In [139]:
code_epci = np.delete(b_18['Code EPCI'].unique(), 2)
# b_18['Code EPCI'].unique() contient la liste des codes des EPCI présents dans la base de données de 2018.
# La deuxième position est occupée par 'nan' (lorsque l'EPCI de la commune n'est pas concernée par la loi SRU).
# On enlève donc l'élément en deuxième position de la liste pour ne garder que les codes des EPCI.

# On va calculer les taux de logements sociaux pour chaque EPCI
# Ainsi que le nombre total de résidences principales de chaque EPCI
for epci in code_epci : # Pour chaque EPCI
    #print(epci)
    df = b_18[b_18['Code EPCI'] == epci] # Toutes les données sur les communes de l'EPCI
    
    nb_com_epci = len(df) # le nombre de communes de l'EPCI
    nb_com_nc = len(df[df.total_lls_new == 0])
                 # le nombre de communes de l'EPCI, pour laquelle il manque le nombre de logements sociaux
    
    nb_rp = sum(df['Nbe de RP']) # le nombre de résidences principales de l'EPCI
    nb_ls = sum(df.total_lls_new) # le nombre de logements sociaux de l'EPCI
    nb_rp_nc = 0 # le nombre de résidences principales de l'EPCI qui sont potentiellement des logements sociaux sans 
                 # être comptés comme tel (pas d'informations sur les logements sociaux dans la commune)
    for i in df['Nbe de RP'][df.total_lls_new == 0] :
        nb_rp_nc = nb_rp_nc + int(i)
    for i in df['Nbe de RP'][df['Total LLS'] == 'NR'] :
        nb_rp_nc = nb_rp_nc + int(i)
    
    com = df.index # l'index des communes de l'EPCI
    
    # On remplit notre base de données avec les valeurs calculées.
    for commune in com : 
        df_18['taux_epci_18'][commune] = nb_ls/nb_rp # le taux de logement sociaux de l'EPCI
        df_18['rp_epci_18'][commune] = nb_rp
        df_18['rp_epci_nc_18'][commune] = nb_rp_nc
df_18.head(10)

,code_commune,code_agglo_18,code_epci_18,taux_ls_18,situation_18,LLS_18,taux_agglo_18,taux_epci_18,rp_agglo_18,rp_epci_18,rp_agglo_nc_18,rp_epci_nc_18
0,01011,nan,200042935,0.000000,Non soumise sous seuil pop,0,NaN,0.238207,NaN,24357.0,NaN,11388.0
1,01014,nan,200042935,0.000000,Non soumise sous seuil pop,0,NaN,0.238207,NaN,24357.0,NaN,11388.0
2,01024,nan,200071751,0.000000,Non soumise sous seuil pop,0,NaN,0.168936,NaN,58087.0,NaN,30049.0
3,01029,nan,200071751,0.000000,Non soumise sous seuil pop,0,NaN,0.168936,NaN,58087.0,NaN,30049.0
4,01030,00758,nan,0.000000,Non soumise sous seuil pop,0,0.233513,NaN,755547.0,NaN,10577.0,NaN
5,01031,nan,200042935,0.502358,Non soumise atteint taux légal,852,NaN,0.238207,NaN,24357.0,NaN,11388.0
6,01035,nan,200042935,0.000000,Non soumise sous seuil pop,0,NaN,0.238207,NaN,24357.0,NaN,11388.0
7,01038,nan,200071751,0.000000,Non soumise sous seuil pop,0,NaN,0.168936,NaN,58087.0,NaN,30049.0
8,01040,nan,200071751,0.000000,Non soumise sous seuil pop,0,NaN,0.168936,NaN,58087.0,NaN,30049.0
9,01043,00758,nan,0.072778,Soumise prélevée,131,0.233513,NaN,755547.0,NaN,10577.0,NaN


In [140]:
# vérification des valeurs aberrantes
df_18[df_18.taux_ls_18 > 0.7]

,code_commune,code_agglo_18,code_epci_18,taux_ls_18,situation_18,LLS_18,taux_agglo_18,taux_epci_18,rp_agglo_18,rp_epci_18,rp_agglo_nc_18,rp_epci_nc_18
1890,27701,nan,200035665,0.781987,Non soumise atteint taux légal,3447,NaN,0.274657,NaN,27700.0,NaN,11952.0
2860,39308,nan,200010650,0.910112,Non soumise sous seuil pop,81,NaN,0.198561,NaN,25715.0,NaN,221.0
4120,57058,57501,245700372,0.773666,Non soumise atteint taux légal,1986,0.296805,0.230006,36620.0,33847.0,0.0,1289.0
6207,76178,00755,200023414,0.726897,Non soumise atteint taux légal,1408,0.365839,0.357654,216177.0,225374.0,2944.0,3726.0
6548,78644,00851,200058782,0.790995,Non soumise atteint taux légal,1669,0.277820,0.340919,4649841.0,90341.0,9419.0,0.0
7404,91235,00851,200057859,0.726856,Non soumise atteint taux légal,2222,0.277820,0.249505,4649841.0,77762.0,9419.0,246.0
7531,93030,00851,200054781,0.732437,Non soumise atteint taux légal,2888,0.277820,0.281256,4649841.0,3182366.0,9419.0,0.0
7706,97120,9A701,200018653,0.732209,Non soumise atteint taux légal,4949,0.252728,0.419057,102648.0,42567.0,0.0,0.0


## 16. Bilan 2019

147 communes atteignant taux légal avec 0 logement social\
22 communes exemptées avec 0 logement social

In [141]:
# chargement des données du bilan de 2019
b_19 = pd.read_csv("../donnees_initiales/bilans_annuels/ART_55_Bilan_SRU_2019_toutes_communes.csv",
                   sep = ';')
b_19.tail()

,Nom de la commune,Colonne1,Code INSEE,Dpt,Région,Métropole – DOM,Structure application,Population commune 2015,Pop>=seuil (1500-3500),Taux de LLS à atteindre,...,Nb LLS autres financement Etat (en service au 01/01/2002),Nb LLS très social financement ANAH (en service au 01/01/2002),Nb LLS social financement ANAH (en service au 01/01/2002),Nb LLS autres financement ANAH (en service au 01/01/2002),Signature d'un CMS,date de signature,Durée du CMS,Visée,Observations,u
7902,Brando,2B043,2B043,2B,Corse,Métropole,Agglo,1644,0,25,...,1,1,0,0,NaN,NaN,NaN,OUI,NaN,Corse
7903,Furiani,2B120,2B120,2B,Corse,Métropole,Agglo/EPCI,5736,1,25,...,0,0,0,0,NaN,NaN,NaN,OUI,NaN,Corse
7904,San-Martino-di-Lota,2B305,2B305,2B,Corse,Métropole,Agglo/EPCI,2911,0,25,...,0,0,0,0,NaN,NaN,NaN,OUI,NaN,Corse
7905,Santa-Maria-di-Lota,2B309,2B309,2B,Corse,Métropole,Agglo/EPCI,1717,0,25,...,0,0,1,0,NaN,NaN,NaN,OUI,NaN,Corse
7906,Ville-di-Pietrabugno,2B353,2B353,2B,Corse,Métropole,Agglo/EPCI,3379,0,25,...,2,0,0,0,NaN,NaN,NaN,OUI,Commune retombée sous le seuil des 3500 habitants,Corse


In [142]:
# noms des colonnes
for j in b_19.columns :
    print(j)

Nom de la commune
Colonne1
Code INSEE
Dpt
Région
Métropole – DOM
Structure application
Population commune 2015
Pop>=seuil (1500-3500)
Taux de LLS à atteindre
Code Agglo
Nom Agglo
Dep. Agglo
Population Agglo
Taux de LLS de l'agglo
Code EPCI
Nom EPCI
Dep. EPCI
Population EPCI
Taux de LLS de l'EPCI
Nbe de RP
PFH
Situation de la commune par rapport au L.302-5 du CCH
exemption décret
Exonérée de prélèvement (soumise 1ère fois)
DSU >15% ou 20%
Vérif DSU
>25% (>20%)
Taux LLS
Total LLS
total_lls_new
Catégorie 1
Catégorie 2
Catégorie 3
Catégorie 3a
Catégorie 4
Logts déconventionnés
Logts vendus aux locataires
Inventaire réalisé
Notif réalisée
% DRF
Montant DRF correspondant
Prelevement brut sans majoration
Majoration
Total prélèvement brut avant plafonnement
Total prélèvement brut après plafonnement éventuel
Constat de carence
Date de l'A.P.
Taux de maj.
Dépenses déductibles communes
Dépenses indûment déduites
Reliquat disponible
Bénéficiaires locaux
Prélèvement à destination des bénéficiaires 

In [143]:
b_19['Situation de la commune par rapport au L.302-5 du CCH'].unique()

array(['Non soumise sous seuil pop', 'Non soumise atteinte taux légal',
       'Soumise prélèvement nul', 'Soumise prélevée',
       'Non soumise exemptée', 'Soumise exonérée', 'Soumise non prélevée',
       nan], dtype=object)

In [144]:
b_19.replace('Non soumise atteinte taux légal', 'Non soumise atteint taux légal', inplace = True)

In [145]:
data = [] # variable temporaire pour stocker les données
a = 0 # variable de vérification

for i in range(len(b_19)) : # pour chaque commune
    # code commune
    data.append(b_19['Code INSEE'][i].zfill(5))
    
    # code agglo et/ou EPCI
    if b_19['Structure application'][i] == 'EPCI' :
        data.append(np.nan)
        data.append(str(int(b_19['Code EPCI'][i])))
    elif b_19['Structure application'][i] == 'Agglo/EPCI' :
        data.append(str(b_19['Code Agglo'][i]).zfill(5))
        data.append(str(int(b_19['Code EPCI'][i])))
    elif b_19['Structure application'][i] == 'Agglo' :
        data.append(str(b_19['Code Agglo'][i]).zfill(5))
        data.append(np.nan)
    else :
        a = a + 1
        
    # taux commune 
    data.append(b_19.total_lls_new[i]/b_19['Nbe de RP'][i])
    
    # situation
    data.append(b_19['Situation de la commune par rapport au L.302-5 du CCH'][i])
    
    # nombre de logements sociaux
    data.append(b_19["Total LLS"][i])
    
# print(a)
data = np.array(data)
colonnes = ['code_commune', 'code_agglo_19', 'code_epci_19', 'taux_ls_19', 'situation_19', 'LLS_19']
data = np.reshape(data, (len(b_19), len(colonnes)))
df_19 = pd.DataFrame(data, columns = colonnes) # création du DataFrame du bilan 2019
df_19.taux_ls_19 = pd.to_numeric(df_19.taux_ls_19, errors = 'coerce')
df_19.head()

,code_commune,code_agglo_19,code_epci_19,taux_ls_19,situation_19,LLS_19
0,01011,nan,200042935,0.0,Non soumise sous seuil pop,0
1,01014,nan,200042935,0.0,Non soumise sous seuil pop,0
2,01024,nan,200071751,0.0,Non soumise sous seuil pop,0
3,01029,nan,200071751,0.0,Non soumise sous seuil pop,0
4,01030,00758,nan,0.0,Non soumise sous seuil pop,0


In [146]:
# création des colonnes pour les agglomérations et EPCI
df_19['taux_agglo_19'] = np.nan
df_19['taux_epci_19'] = np.nan
df_19['rp_agglo_19'] = np.nan
df_19['rp_epci_19'] = np.nan
df_19['rp_agglo_nc_19'] = np.nan
df_19['rp_epci_nc_19'] = np.nan
df_19.head()

,code_commune,code_agglo_19,code_epci_19,taux_ls_19,situation_19,LLS_19,taux_agglo_19,taux_epci_19,rp_agglo_19,rp_epci_19,rp_agglo_nc_19,rp_epci_nc_19
0,01011,nan,200042935,0.0,Non soumise sous seuil pop,0,NaN,NaN,NaN,NaN,NaN,NaN
1,01014,nan,200042935,0.0,Non soumise sous seuil pop,0,NaN,NaN,NaN,NaN,NaN,NaN
2,01024,nan,200071751,0.0,Non soumise sous seuil pop,0,NaN,NaN,NaN,NaN,NaN,NaN
3,01029,nan,200071751,0.0,Non soumise sous seuil pop,0,NaN,NaN,NaN,NaN,NaN,NaN
4,01030,00758,nan,0.0,Non soumise sous seuil pop,0,NaN,NaN,NaN,NaN,NaN,NaN


In [147]:
agglos = b_19['Code Agglo'].unique()[1:]
# b_19['Code agglo'].unique() contient la liste des codes des agglomérations présentes dans le bilan de 2019.
# Le premier élément de la liste est occupée par 'nan' qui correspond à la commune ne fait pas partie d'une UU
# concernée par l'article 55 de la loi SRU donc on ne parcourt la liste des UU que pour les éléments suivants.

# On va calculer les taux de logements sociaux pour chaque agglomération
# Ainsi que le nombre total de résidences principales de chaque agglomération
for agglo in agglos : # Pour chaque agglomération
    df = b_19[b_19['Code Agglo'] == agglo] # Toutes les données sur les communes de l'agglomération
    
    nb_com_agglo = len(df) # le nombre de communes de l'agglomération
    nb_com_nc = len(df[df.total_lls_new == 0])
                 # le nombre de communes de l'agglomération, pour laquelle il manque le nombre de logements sociaux
    
    nb_rp = 0 # le nombre de résidences principales de l'agglomération
    for i in df['Nbe de RP'] :
        nb_rp = nb_rp + int(i)
    nb_ls = sum(df.total_lls_new) # le nombre de logements sociaux de l'agglomération
    nb_rp_nc = 0 # le nombre de résidences principales de l'agglomération qui sont potentiellement des logements sociaux sans 
                 # être comptées comme tel (pas d'informations sur les logements sociaux dans la commune)
    for i in df['Nbe de RP'][df.total_lls_new == 0] :
        nb_rp_nc = nb_rp_nc + int(i)
    
    com = df.index # l'index des communes de l'agglomération
    
    # On remplit notre base de données avec les valeurs calculées.
    for commune in com : 
        df_19['taux_agglo_19'][commune] = nb_ls/nb_rp # le taux de logement sociaux de l'agglomération
        df_19['rp_agglo_19'][commune] = nb_rp
        df_19['rp_agglo_nc_19'][commune] = nb_rp_nc
df_19.head(10)

,code_commune,code_agglo_19,code_epci_19,taux_ls_19,situation_19,LLS_19,taux_agglo_19,taux_epci_19,rp_agglo_19,rp_epci_19,rp_agglo_nc_19,rp_epci_nc_19
0,01011,nan,200042935,0.000000,Non soumise sous seuil pop,0,NaN,NaN,NaN,NaN,NaN,NaN
1,01014,nan,200042935,0.000000,Non soumise sous seuil pop,0,NaN,NaN,NaN,NaN,NaN,NaN
2,01024,nan,200071751,0.000000,Non soumise sous seuil pop,0,NaN,NaN,NaN,NaN,NaN,NaN
3,01029,nan,200071751,0.000000,Non soumise sous seuil pop,0,NaN,NaN,NaN,NaN,NaN,NaN
4,01030,00758,nan,0.000000,Non soumise sous seuil pop,0,0.238754,NaN,759029.0,NaN,10825.0,NaN
5,01031,nan,200042935,0.498833,Non soumise atteint taux légal,855,NaN,NaN,NaN,NaN,NaN,NaN
6,01035,nan,200042935,0.000000,Non soumise sous seuil pop,0,NaN,NaN,NaN,NaN,NaN,NaN
7,01038,nan,200071751,0.000000,Non soumise sous seuil pop,0,NaN,NaN,NaN,NaN,NaN,NaN
8,01040,nan,200071751,0.000000,Non soumise sous seuil pop,0,NaN,NaN,NaN,NaN,NaN,NaN
9,01043,00758,nan,0.070773,Soumise prélèvement nul,131,0.238754,NaN,759029.0,NaN,10825.0,NaN


In [148]:
code_epci = np.delete(b_19['Code EPCI'].unique(), 2)
# b_19['Code EPCI'].unique() contient la liste des codes des EPCI présents dans la base de données de 2019.
# La deuxième position est occupée par 'nan' (lorsque l'EPCI de la commune n'est pas concernée par la loi SRU).
# On enlève donc l'élément en deuxième position de la liste pour ne garder que les codes des EPCI.

# On va calculer les taux de logements sociaux pour chaque EPCI
# Ainsi que le nombre total de résidences principales de chaque EPCI
for epci in code_epci : # Pour chaque EPCI
    #print(epci)
    df = b_19[b_19['Code EPCI'] == epci] # Toutes les données sur les communes de l'EPCI
    
    nb_com_epci = len(df) # le nombre de communes de l'EPCI
    nb_com_nc = len(df[df.total_lls_new == 0])
                 # le nombre de communes de l'EPCI, pour laquelle il manque le nombre de logements sociaux
    
    nb_rp = sum(df['Nbe de RP']) # le nombre de résidences principales de l'EPCI
    nb_ls = sum(df.total_lls_new) # le nombre de logements sociaux de l'EPCI
    nb_rp_nc = 0 # le nombre de résidences principales de l'EPCI qui sont potentiellement des logements sociaux sans 
                 # être comptées comme tel (pas d'informations sur les logements sociaux dans la commune)
    for i in df['Nbe de RP'][df.total_lls_new == 0] :
        nb_rp_nc = nb_rp_nc + int(i)
    
    com = df.index # l'index des communes de l'EPCI
    
    # On remplit notre base de données avec les valeurs calculées.
    for commune in com : 
        df_19['taux_epci_19'][commune] = nb_ls/nb_rp # le taux de logement sociaux de l'EPCI
        df_19['rp_epci_19'][commune] = nb_rp
        df_19['rp_epci_nc_19'][commune] = nb_rp_nc
df_19.head(10)

,code_commune,code_agglo_19,code_epci_19,taux_ls_19,situation_19,LLS_19,taux_agglo_19,taux_epci_19,rp_agglo_19,rp_epci_19,rp_agglo_nc_19,rp_epci_nc_19
0,01011,nan,200042935,0.000000,Non soumise sous seuil pop,0,NaN,0.241815,NaN,24618.0,NaN,11539.0
1,01014,nan,200042935,0.000000,Non soumise sous seuil pop,0,NaN,0.241815,NaN,24618.0,NaN,11539.0
2,01024,nan,200071751,0.000000,Non soumise sous seuil pop,0,NaN,0.171412,NaN,58438.0,NaN,30374.0
3,01029,nan,200071751,0.000000,Non soumise sous seuil pop,0,NaN,0.171412,NaN,58438.0,NaN,30374.0
4,01030,00758,nan,0.000000,Non soumise sous seuil pop,0,0.238754,NaN,759029.0,NaN,10825.0,NaN
5,01031,nan,200042935,0.498833,Non soumise atteint taux légal,855,NaN,0.241815,NaN,24618.0,NaN,11539.0
6,01035,nan,200042935,0.000000,Non soumise sous seuil pop,0,NaN,0.241815,NaN,24618.0,NaN,11539.0
7,01038,nan,200071751,0.000000,Non soumise sous seuil pop,0,NaN,0.171412,NaN,58438.0,NaN,30374.0
8,01040,nan,200071751,0.000000,Non soumise sous seuil pop,0,NaN,0.171412,NaN,58438.0,NaN,30374.0
9,01043,00758,nan,0.070773,Soumise prélèvement nul,131,0.238754,NaN,759029.0,NaN,10825.0,NaN


In [149]:
# vérification des valeurs abérrantes
df_19[df_19.taux_ls_19 > .7]

,code_commune,code_agglo_19,code_epci_19,taux_ls_19,situation_19,LLS_19,taux_agglo_19,taux_epci_19,rp_agglo_19,rp_epci_19,rp_agglo_nc_19,rp_epci_nc_19
1928,27701,nan,200035665,0.778341,Non soumise atteint taux légal,3378,NaN,0.261334,NaN,28565.0,NaN,12894.0
4179,57058,57501,245700372,0.786281,Non soumise atteint taux légal,1983,0.300537,0.229540,36498.0,33785.0,0.0,1304.0
6295,76178,00755,200023414,0.731429,Non soumise atteint taux légal,1408,0.359726,0.348713,216901.0,226094.0,13847.0,24097.0
6680,78644,00851,200058782,0.783936,Non soumise atteint taux légal,1669,0.279809,0.342878,4674776.0,91190.0,9212.0,0.0
7625,93030,00851,200054781,0.737676,Non soumise atteint taux légal,2888,0.279809,0.282754,4674776.0,3197237.0,9212.0,0.0
7658,94011,00851,200054781,0.700204,Non soumise atteint taux légal,4468,0.279809,0.282754,4674776.0,3197237.0,9212.0,0.0
7800,97120,9A701,200018653,0.778221,Non soumise atteint taux légal,5067,0.268919,0.443546,100138.0,41317.0,0.0,0.0


## 17. Bilan 2020

140 communes atteignant taux légal avec 0 logement social\
8 communes exemptées avec 0 logement social

In [150]:
# chargement des données du bilan de 2020
b_20 = pd.read_csv("../donnees_initiales/bilans_annuels/ART_55_Bilan_SRU_2020_toutes_communes.csv",
                   sep = ';')[:7928]
b_20.tail()

,Nom commune,Code INSEE,Dpt,Région,Métropole-Dom,Structure d’application,Population commune 2015,Pop>=seuil (1500-3500),Taux à atteindre (décret 2017),Taux à atteindre (décret 2020),...,Nb de LLS programmé,Secteurs PC Préfet,Convention IML parc privé,Nombre de conventions conclues,Nombre de logements concernés,Signature d'un CMS,Date de signature,Durée du CMS,Visée,Observation DDT
7923,Brando,2B043,2B,Corse,Métropole,Agglo,1613.0,0.0,25 %,25 %,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,OUI,Commune à la population inférieure à 3500 habi...
7924,Furiani,2B120,2B,Corse,Métropole,Agglo/EPCI,5682.0,1.0,25 %,25 %,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,OUI,NaN
7925,San-Martino-di-Lota,2B305,2B,Corse,Métropole,Agglo/EPCI,2906.0,0.0,25 %,25 %,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,OUI,Commune à la population inférieure à 3500 habi...
7926,Santa-Maria-di-Lota,2B309,2B,Corse,Métropole,Agglo/EPCI,1718.0,0.0,25 %,25 %,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,OUI,Commune à la population inférieure à 3500 habi...
7927,Ville-di-Pietrabugno,2B353,2B,Corse,Métropole,Agglo/EPCI,3292.0,0.0,25 %,25 %,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,OUI,Commune dont la population est redescendu sous...


In [151]:
# noms des colonnes
for j in b_20.columns :
    print(j)

Nom commune
Code INSEE
Dpt
Région
Métropole-Dom
Structure d’application
Population commune 2015
Pop>=seuil (1500-3500)
Taux à atteindre (décret  2017)
Taux à atteindre (décret 2020)
Code agglo
Nom agglo 2019 
Taux de l’agglo (décret 2017)
Taux moyen tension agglo 2017/2019 inf2
Code EPCI
Nom de l’EPCI
Taux de l’EPCI (décret 2017)
taux moyen trsnion EPCI 20172019
Nb RP
PFH
Situation de la commune par rapport au L.302-5 du CCH
Première année de soumission
Exemption décret
Exonérée de prélèvement (soumise 1ère fois)
DSU >15% ou 20%
DSU vérif
>25% (>20%)
Taux LLS
Total LLS
total_lls_new
Catégorie 1
(logts des bailleurs HLM)
Catégorie 2 (logts conv à l’APL appartenant à des bailleurs non HLM)
Catégorie 3 (SEM des DOM ; logts mineurs)
Catégorie 3a (logts des harkis)
Catégorie 4
(logts foyers, CHRS, CADA)
Catégorie 5 (terrains familiaux)
Catégorie 6 (IML privé)
Catégorie 7
(PSLA)
Catégorie 8
(BRS)
Logts déconventionnés
Logts vendus aux locataires
Inventaire réalisé
Notif réalisée
Nb LLS très 

In [152]:
b_20['Situation de la commune par rapport au L.302-5 du CCH'].unique()

array(['Non soumise sous seuil de pop', 'Non soumise atteinte taux légal',
       'Soumise prélevée', 'Non soumise exemptée',
       'Soumise prélèvement nul', 'Soumise exonérée',
       'Soumise non prélevée'], dtype=object)

In [153]:
b_20.replace('Non soumise sous seuil de pop', 'Non soumise sous seuil pop', inplace = True)
b_20.replace('Non soumise atteinte taux légal', 'Non soumise atteint taux légal', inplace = True)

In [154]:
data = [] # variable temporaire pour stocker les données
a = 0 # variable de vérification

for i in range(len(b_20)) : # pour chaque commune
    # code commune
    data.append(b_20['Code INSEE'][i].zfill(5))
    
    # code agglo et/ou EPCI
    if b_20["Structure d’application"][i] == 'EPCI' :
        data.append(np.nan)
        data.append(str(int(b_20['Code EPCI'][i])))
    elif b_20['Structure d’application'][i] == 'Agglo/EPCI' :
        data.append(str(b_20['Code agglo'][i]).zfill(5))
        data.append(str(int(b_20['Code EPCI'][i])))
    elif b_20['Structure d’application'][i] == 'Agglo' :
        data.append(str(b_20['Code agglo'][i]).zfill(5))
        data.append(np.nan)
    else :
        a = a + 1
        
    # taux commune 
    data.append(float(b_20.total_lls_new[i])/b_20['Nb RP'][i])
    
    # situation
    data.append(b_20['Situation de la commune par rapport au L.302-5 du CCH'][i])
    
    # nombre de logements sociaux
    data.append(b_20["Total LLS"][i])
    
# print(a)
data = np.array(data)
colonnes = ['code_commune', 'code_agglo_20', 'code_epci_20', 'taux_ls_20', 'situation_20', 'LLS_20']
data = np.reshape(data, (len(b_20), len(colonnes)))
df_20 = pd.DataFrame(data, columns = colonnes) # création du DataFrame du bilan 2020
df_20.taux_ls_20 = pd.to_numeric(df_20.taux_ls_20, errors = 'coerce')
df_20.head()

,code_commune,code_agglo_20,code_epci_20,taux_ls_20,situation_20,LLS_20
0,01011,nan,200042935,0.0,Non soumise sous seuil pop,0.0
1,01012,nan,200042935,0.0,Non soumise sous seuil pop,0.0
2,01014,nan,200042935,0.0,Non soumise sous seuil pop,0.0
3,01024,nan,200071751,0.0,Non soumise sous seuil pop,0.0
4,01029,nan,200071751,0.0,Non soumise sous seuil pop,0.0


In [155]:
# création des colonnes des agglomérations et EPCI
df_20['taux_agglo_20'] = np.nan
df_20['taux_epci_20'] = np.nan
df_20['rp_agglo_20'] = np.nan
df_20['rp_epci_20'] = np.nan
df_20['rp_agglo_nc_20'] = np.nan
df_20['rp_epci_nc_20'] = np.nan
df_20.head()

,code_commune,code_agglo_20,code_epci_20,taux_ls_20,situation_20,LLS_20,taux_agglo_20,taux_epci_20,rp_agglo_20,rp_epci_20,rp_agglo_nc_20,rp_epci_nc_20
0,01011,nan,200042935,0.0,Non soumise sous seuil pop,0.0,NaN,NaN,NaN,NaN,NaN,NaN
1,01012,nan,200042935,0.0,Non soumise sous seuil pop,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2,01014,nan,200042935,0.0,Non soumise sous seuil pop,0.0,NaN,NaN,NaN,NaN,NaN,NaN
3,01024,nan,200071751,0.0,Non soumise sous seuil pop,0.0,NaN,NaN,NaN,NaN,NaN,NaN
4,01029,nan,200071751,0.0,Non soumise sous seuil pop,0.0,NaN,NaN,NaN,NaN,NaN,NaN


In [156]:
agglos = b_20['Code agglo'].unique()[1:]
# b_20['Code agglo'].unique() contient la liste des codes des agglomérations présentes dans le bilan de 2020.
# Le premier élément de la liste est occupée par '0' qui correspond à la commune ne fait pas partie d'une UU
# concernée par l'article 55 de la loi SRU donc on ne parcourt la liste des UU que pour les éléments suivants.

# On va calculer les taux de logements sociaux pour chaque agglomération
# Ainsi que le nombre total de résidences principales de chaque agglomération
for agglo in agglos : # Pour chaque agglomération
    df = b_20[b_20['Code agglo'] == agglo] # Toutes les données sur les communes de l'agglomération
    
    nb_com_agglo = len(df) # le nombre de communes de l'agglomération
    nb_com_nc = len(df[df.total_lls_new == 0])
                 # le nombre de communes de l'agglomération, pour laquelle il manque le nombre de logements sociaux
    
    nb_rp = int(sum(df['Nb RP'])) # le nombre de résidences principales de l'agglomération
    nb_ls = sum(df.total_lls_new) # le nombre de logements sociaux de l'agglomération
    nb_rp_nc = int(sum(df['Nb RP'][df.total_lls_new == 0])) 
                 # le nombre de résidences principales de l'agglomération qui sont potentiellement des logements sociaux sans 
                 # être comptées comme tel (pas d'informations sur les logements sociaux dans la commune)
    
    com = df.index # l'index des communes de l'agglomération
    
    # On remplit notre base de données avec les valeurs calculées.
    for commune in com : 
        df_20['taux_agglo_20'][commune] = nb_ls/nb_rp # le taux de logement sociaux de l'agglomération
        df_20['rp_agglo_20'][commune] = nb_rp
        df_20['rp_agglo_nc_20'][commune] = nb_rp_nc
df_20.head(10)

,code_commune,code_agglo_20,code_epci_20,taux_ls_20,situation_20,LLS_20,taux_agglo_20,taux_epci_20,rp_agglo_20,rp_epci_20,rp_agglo_nc_20,rp_epci_nc_20
0,01011,nan,200042935,0.000000,Non soumise sous seuil pop,0.0,NaN,NaN,NaN,NaN,NaN,NaN
1,01012,nan,200042935,0.000000,Non soumise sous seuil pop,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2,01014,nan,200042935,0.000000,Non soumise sous seuil pop,0.0,NaN,NaN,NaN,NaN,NaN,NaN
3,01024,nan,200071751,0.000000,Non soumise sous seuil pop,0.0,NaN,NaN,NaN,NaN,NaN,NaN
4,01029,nan,200071751,0.000000,Non soumise sous seuil pop,0.0,NaN,NaN,NaN,NaN,NaN,NaN
5,01030,00758,nan,0.000000,Non soumise sous seuil pop,0.0,0.240143,NaN,768114.0,NaN,10937.0,NaN
6,01031,nan,200042935,0.497672,Non soumise atteint taux légal,855.0,NaN,NaN,NaN,NaN,NaN,NaN
7,01035,nan,200042935,0.000000,Non soumise sous seuil pop,0.0,NaN,NaN,NaN,NaN,NaN,NaN
8,01038,nan,200071751,0.000000,Non soumise sous seuil pop,0.0,NaN,NaN,NaN,NaN,NaN,NaN
9,01040,nan,200071751,0.000000,Non soumise sous seuil pop,0.0,NaN,NaN,NaN,NaN,NaN,NaN


In [157]:
code_epci = np.delete(b_20['Code EPCI'].unique(), 2)
# b_20['Code EPCI'].unique() contient la liste des codes des EPCI présents dans la base de données de 2020
# La deuxième position est occupée par 'nan' (lorsque l'EPCI de la commune n'est pas concernée par la loi SRU).
# On enlève donc la deuxième position de la liste pour ne garder que les codes des EPCI.

# On va calculer les taux de logements sociaux pour chaque EPCI
# Ainsi que le nombre total de résidences principales de chaque EPCI
for epci in code_epci : # Pour chaque EPCI
    #print(epci)
    df = b_20[b_20['Code EPCI'] == epci] # Toutes les données sur les communes de l'EPCI
    
    nb_com_epci = len(df) # le nombre de communes de l'EPCI
    nb_com_nc = len(df[df.total_lls_new == 0])
                 # le nombre de communes de l'EPCI, pour laquelle il manque le nombre de logements sociaux
    
    nb_rp = int(sum(df['Nb RP'])) # le nombre de résidences principales de l'EPCI
    nb_ls = sum(df.total_lls_new) # le nombre de logements sociaux de l'EPCI
    nb_rp_nc = int(sum(df['Nb RP'][df.total_lls_new == 0])) 
                 # le nombre de résidences principales de l'EPCI qui sont potentiellement des logements sociaux sans 
                 # être comptés comme tel (pas d'informations sur les logements sociaux dans la commune)
    
    com = df.index # l'index des communes de l'epci
    
    # On remplit notre base de données avec les valeurs calculées.
    for commune in com : 
        df_20['taux_epci_20'][commune] = nb_ls/nb_rp # le taux de logement sociaux de l'epci
        df_20['rp_epci_20'][commune] = nb_rp
        df_20['rp_epci_nc_20'][commune] = nb_rp_nc
df_20.head(10)

,code_commune,code_agglo_20,code_epci_20,taux_ls_20,situation_20,LLS_20,taux_agglo_20,taux_epci_20,rp_agglo_20,rp_epci_20,rp_agglo_nc_20,rp_epci_nc_20
0,01011,nan,200042935,0.000000,Non soumise sous seuil pop,0.0,NaN,0.215495,NaN,27532.0,NaN,13738.0
1,01012,nan,200042935,0.000000,Non soumise sous seuil pop,0.0,NaN,0.215495,NaN,27532.0,NaN,13738.0
2,01014,nan,200042935,0.000000,Non soumise sous seuil pop,0.0,NaN,0.215495,NaN,27532.0,NaN,13738.0
3,01024,nan,200071751,0.000000,Non soumise sous seuil pop,0.0,NaN,0.170835,NaN,58858.0,NaN,30593.0
4,01029,nan,200071751,0.000000,Non soumise sous seuil pop,0.0,NaN,0.170835,NaN,58858.0,NaN,30593.0
5,01030,00758,nan,0.000000,Non soumise sous seuil pop,0.0,0.240143,NaN,768114.0,NaN,10937.0,NaN
6,01031,nan,200042935,0.497672,Non soumise atteint taux légal,855.0,NaN,0.215495,NaN,27532.0,NaN,13738.0
7,01035,nan,200042935,0.000000,Non soumise sous seuil pop,0.0,NaN,0.215495,NaN,27532.0,NaN,13738.0
8,01038,nan,200071751,0.000000,Non soumise sous seuil pop,0.0,NaN,0.170835,NaN,58858.0,NaN,30593.0
9,01040,nan,200071751,0.000000,Non soumise sous seuil pop,0.0,NaN,0.170835,NaN,58858.0,NaN,30593.0


In [158]:
df_20[df_20.taux_ls_20 > .7]

,code_commune,code_agglo_20,code_epci_20,taux_ls_20,situation_20,LLS_20,taux_agglo_20,taux_epci_20,rp_agglo_20,rp_epci_20,rp_agglo_nc_20,rp_epci_nc_20
4219,57058,57501,245700372,0.803521,Non soumise atteint taux légal,2008.0,0.300742,0.219417,36380.0,33785.0,6066.0,12205.0
6705,78644,00851,200058782,0.753159,Non soumise atteint taux légal,1669.0,0.280913,0.342806,4725547.0,92163.0,7561.0,0.0
7646,93030,00851,200054781,0.733894,Non soumise atteint taux légal,2882.0,0.280913,0.283816,4725547.0,3229470.0,7561.0,0.0
7821,97120,9A701,200018653,0.848156,Non soumise atteint taux légal,5083.0,0.284714,0.475461,96950.0,39448.0,0.0,0.0


## 18. Bilan 2021

87 communes atteignant taux légal sans logement social\
4 communes exemptées sans logement social

In [159]:
# chargement des données du bilan de 2021
b_21 = pd.read_csv("../donnees_initiales/bilans_annuels/ART_55_Bilan_SRU_2021_toutes_communes.csv",
                   sep = ';')
b_21.tail()

,Nom commune,Unnamed: 1,Code INSEE,dpt,Région,Métropole – DOM,Structure application,Population commune 2017,Pop>=seuil (1500-3500),Taux de LLS à atteindre,...,Convention IML parc privé,Nombre de conventions conclues,Nombre de logements concernés,Taux de rattrapage applicable,Objectif quantitatif,Signature d'un CMS,Date de signature,Durée du CMS,Visée,Observation
8063,Bandraboua,97602,97602,976,Mayotte,DOM,EPCI,13989,1,25,...,0.0,0,0,,NaN,0.0,NaN,NaN,OUI,NaN
8064,Dembeni,97607,97607,976,Mayotte,DOM,EPCI,15848,1,25,...,0.0,0,0,,NaN,0.0,NaN,NaN,OUI,NaN
8065,Koungou,97610,97610,976,Mayotte,DOM,Agglo/EPCI,32156,1,25,...,0.0,0,0,,NaN,0.0,NaN,NaN,OUI,NaN
8066,Mamoudzou,97611,97611,976,Mayotte,DOM,Agglo/EPCI,71437,1,25,...,0.0,0,0,,NaN,0.0,NaN,NaN,OUI,NaN
8067,Mtsamboro,97612,97612,976,Mayotte,DOM,EPCI,7705,1,25,...,0.0,0,0,,NaN,0.0,NaN,NaN,OUI,NaN


In [160]:
# noms des colonnes
for j in b_21.columns :
    print(j)

Nom commune
Unnamed: 1
Code INSEE
dpt
Région
Métropole – DOM
Structure application
Population commune 2017
Pop>=seuil (1500-3500)
Taux de LLS à atteindre
Unnamed: 10
Code Agglo
Nom Agglo
Dept. Agglo
Population Agglo
Taux de LLS de l'agglo
Code EPCI
Nom EPCI
Dept. EPCI
Population EPCI
Taux de LLS de l'EPCI
Nb RP
PFH
Situation de la commune par rapport au L.302-5 du CCH
Première année de soumission
Exemption décret
Exonérée de prélèvement (soumise 1ère fois)
DSU >15% ou 20%
Unnamed: 28
>25% (>20%)
Total LLS
total_lls_new
Taux LLS
Catégorie 1
(logts des bailleurs HLM)
Catégorie 2 (logts conv à l’APL appartenant à des bailleurs non HLM)
Catégorie 3 (SEM des DOM ; logts mineurs)
Catégorie 3a (logts des harkis)
Catégorie 4
(logts foyers, CHRS, CADA)
Catégorie 5 (terrains familiaux)
Catégorie 6 (IML privé)
Catégorie 7
(PSLA)
Catégorie 8
(BRS)
Logts déconventionnés
Logts vendus aux locataires
Inventaire réalisé
Notif réalisée
Très social (PLA d'intégration / PLA LM / PLATS / PLAI / PLAI adapté

In [161]:
b_21['Situation de la commune par rapport au L.302-5 du CCH'].unique()

array(['Non soumise sous seuil de pop', 'Non soumise atteint taux légal',
       'Soumise prélèvement nul', 'Soumise prélevée',
       'Non soumise exemptée', 'Soumise non prélevée', 'Soumise exonérée'],
      dtype=object)

In [162]:
b_21.replace('Non soumise sous seuil de pop', 'Non soumise sous seuil pop', inplace = True)

In [163]:
data = [] # variable temporaire pour stocker les données
a = 0 # variable de vérification

for i in range(len(b_21)) : # pour chaque commune
    # code commune
    data.append(b_21['Code INSEE'][i].zfill(5))
    
    # code agglo et/ou EPCI
    if b_21['Structure application'][i] == 'EPCI' :
        data.append(np.nan)
        data.append(str(int(b_21['Code EPCI'][i])))
    elif b_21['Structure application'][i] == 'Agglo/EPCI' :
        data.append(str(b_21['Code Agglo'][i]).zfill(5))
        data.append(str(int(b_21['Code EPCI'][i])))
    elif b_21['Structure application'][i] == 'Agglo' :
        data.append(str(b_21['Code Agglo'][i]).zfill(5))
        data.append(np.nan)
    elif b_21['Structure application'][i] == 'Isolée' :
        data.append(np.nan)
        data.append(np.nan)
    else :
        print(b_21['Structure application'][i])
        a = a + 1
        
    # taux commune 
    data.append(float(b_21.total_lls_new[i])/b_21['Nb RP'][i])
    
    # situation
    data.append(b_21['Situation de la commune par rapport au L.302-5 du CCH'][i])
    
    # nombre de logements sociaux
    data.append(b_21["Total LLS"][i])
    
# print(a)
data = np.array(data)
colonnes = ['code_commune', 'code_agglo_21', 'code_epci_21', 'taux_ls_21', 'situation_21', 'LLS_21']
data = np.reshape(data, (len(b_21), len(colonnes)))
df_21 = pd.DataFrame(data, columns = colonnes) # création du DataFrame du bilan 2021
df_21.taux_ls_21 = pd.to_numeric(df_21.taux_ls_21, errors = 'coerce')
df_21.head()

,code_commune,code_agglo_21,code_epci_21,taux_ls_21,situation_21,LLS_21
0,01011,nan,200042935,0.0,Non soumise sous seuil pop,0
1,01012,nan,200042935,0.0,Non soumise sous seuil pop,0
2,01014,nan,200042935,0.0,Non soumise sous seuil pop,0
3,01024,nan,200071751,0.0,Non soumise sous seuil pop,0
4,01029,nan,200071751,0.0,Non soumise sous seuil pop,0


In [164]:
# création des colonnes des agglomérations et EPCI
df_21['taux_agglo_21'] = np.nan
df_21['taux_epci_21'] = np.nan
df_21['rp_agglo_21'] = np.nan
df_21['rp_epci_21'] = np.nan
df_21['rp_agglo_nc_21'] = np.nan
df_21['rp_epci_nc_21'] = np.nan
df_21.head()

,code_commune,code_agglo_21,code_epci_21,taux_ls_21,situation_21,LLS_21,taux_agglo_21,taux_epci_21,rp_agglo_21,rp_epci_21,rp_agglo_nc_21,rp_epci_nc_21
0,01011,nan,200042935,0.0,Non soumise sous seuil pop,0,NaN,NaN,NaN,NaN,NaN,NaN
1,01012,nan,200042935,0.0,Non soumise sous seuil pop,0,NaN,NaN,NaN,NaN,NaN,NaN
2,01014,nan,200042935,0.0,Non soumise sous seuil pop,0,NaN,NaN,NaN,NaN,NaN,NaN
3,01024,nan,200071751,0.0,Non soumise sous seuil pop,0,NaN,NaN,NaN,NaN,NaN,NaN
4,01029,nan,200071751,0.0,Non soumise sous seuil pop,0,NaN,NaN,NaN,NaN,NaN,NaN


In [165]:
agglos = b_21['Code Agglo'].unique()[1:]
# b_21['Code agglo'].unique() contient la liste des codes des agglomérations présentes dans le bilan de 2021.
# Le premier élément de la liste est occupée par 'nan' qui correspond à la commune ne fait pas partie d'une UU
# concernée par l'article 55 de la loi SRU donc on ne parcourt la liste des UU que pour les éléments suivants.

# On va calculer les taux de logements sociaux pour chaque agglomération
# Ainsi que le nombre total de résidences principales de chaque agglomération
for agglo in agglos : # Pour chaque agglomération
    df = b_21[b_21['Code Agglo'] == agglo] # Toutes les données sur les communes de l'agglomération
    
    nb_com_agglo = len(df) # le nombre de communes de l'agglomération
    nb_com_nc = len(df[df.total_lls_new == 0])
                 # le nombre de communes de l'Eagglomération, pour laquelle il manque le nombre de logements sociaux
    
    nb_rp = int(sum(df['Nb RP'])) # le nombre de résidences principales de l'agglomération
    nb_ls = sum(df.total_lls_new) # le nombre de logements sociaux de l'agglomération
    nb_rp_nc = int(sum(df['Nb RP'][df.total_lls_new == 0])) 
                 # le nombre de résidences principales de l'agglomération qui sont potentiellement des logements sociaux sans 
                 # être classées comme tel (pas d'informations sur les logements sociaux dans la commune)
    
    com = df.index # l'index des communes de l'agglomération
    
    # On remplit notre base de données avec les valeurs calculées.
    for commune in com : 
        df_21['taux_agglo_21'][commune] = nb_ls/nb_rp # le taux de logement sociaux de l'agglomération
        df_21['rp_agglo_21'][commune] = nb_rp
        df_21['rp_agglo_nc_21'][commune] = nb_rp_nc
df_21.head(10)

,code_commune,code_agglo_21,code_epci_21,taux_ls_21,situation_21,LLS_21,taux_agglo_21,taux_epci_21,rp_agglo_21,rp_epci_21,rp_agglo_nc_21,rp_epci_nc_21
0,01011,nan,200042935,0.000000,Non soumise sous seuil pop,0,NaN,NaN,NaN,NaN,NaN,NaN
1,01012,nan,200042935,0.000000,Non soumise sous seuil pop,0,NaN,NaN,NaN,NaN,NaN,NaN
2,01014,nan,200042935,0.000000,Non soumise sous seuil pop,0,NaN,NaN,NaN,NaN,NaN,NaN
3,01024,nan,200071751,0.000000,Non soumise sous seuil pop,0,NaN,NaN,NaN,NaN,NaN,NaN
4,01029,nan,200071751,0.000000,Non soumise sous seuil pop,0,NaN,NaN,NaN,NaN,NaN,NaN
5,01030,00760,nan,0.000000,Non soumise sous seuil pop,0,0.243303,NaN,772743.0,NaN,10401.0,NaN
6,01031,nan,200042935,0.496516,Non soumise atteint taux légal,855,NaN,NaN,NaN,NaN,NaN,NaN
7,01035,nan,200042935,0.000000,Non soumise sous seuil pop,0,NaN,NaN,NaN,NaN,NaN,NaN
8,01038,nan,200071751,0.000000,Non soumise sous seuil pop,0,NaN,NaN,NaN,NaN,NaN,NaN
9,01040,nan,200071751,0.000000,Non soumise sous seuil pop,0,NaN,NaN,NaN,NaN,NaN,NaN


In [166]:
code_epci = np.delete(b_21['Code EPCI'].unique(), 2)
# b_21['Code EPCI'].unique() contient la liste des codes des EPCI présents dans la base de données de 2021.
# La deuxième position est occupée par 'nan' (lorsque l'EPCI de la commune n'est pas concernée par la loi SRU).
# On enlève donc l'élément en deuxième position de la liste pour ne garder que les codes des EPCI.

# On va calculer les taux de logements sociaux pour chaque EPCI
# Ainsi que le nombre total de résidences principales de chaque EPCI
for epci in code_epci : # Pour chaque EPCI
    #print(epci)
    df = b_21[b_21['Code EPCI'] == epci] # Toutes les données sur les communes de l'EPCI
    
    nb_com_epci = len(df) # le nombre de communes de l'EPCI
    nb_com_nc = len(df[df.total_lls_new == 0])
                 # le nombre de communes de l'EPCI, pour laquelle il manque le nombre de logements sociaux
    
    nb_rp = int(sum(df['Nb RP'])) # le nombre de résidences principales de l'EPCI
    nb_ls = sum(df.total_lls_new) # le nombre de logements sociaux de l'EPCI
    nb_rp_nc = int(sum(df['Nb RP'][df.total_lls_new == 0])) 
                 # le nombre de résidences principales de l'EPCI qui sont potentiellement des logements sociaux sans 
                 # être comptés comme tel (pas d'informations sur les logements sociaux dans la commune)
    
    com = df.index # l'index des communes de l'EPCI
    
    # On remplit notre base de données avec les valeurs calculées.
    for commune in com : 
        df_21['taux_epci_21'][commune] = nb_ls/nb_rp # le taux de logement sociaux de l'EPCI
        df_21['rp_epci_21'][commune] = nb_rp
        df_21['rp_epci_nc_21'][commune] = nb_rp_nc
df_21.head(10)

,code_commune,code_agglo_21,code_epci_21,taux_ls_21,situation_21,LLS_21,taux_agglo_21,taux_epci_21,rp_agglo_21,rp_epci_21,rp_agglo_nc_21,rp_epci_nc_21
0,01011,nan,200042935,0.000000,Non soumise sous seuil pop,0,NaN,0.218770,NaN,27650.0,NaN,13826.0
1,01012,nan,200042935,0.000000,Non soumise sous seuil pop,0,NaN,0.218770,NaN,27650.0,NaN,13826.0
2,01014,nan,200042935,0.000000,Non soumise sous seuil pop,0,NaN,0.218770,NaN,27650.0,NaN,13826.0
3,01024,nan,200071751,0.000000,Non soumise sous seuil pop,0,NaN,0.169795,NaN,59136.0,NaN,30905.0
4,01029,nan,200071751,0.000000,Non soumise sous seuil pop,0,NaN,0.169795,NaN,59136.0,NaN,30905.0
5,01030,00760,nan,0.000000,Non soumise sous seuil pop,0,0.243303,NaN,772743.0,NaN,10401.0,NaN
6,01031,nan,200042935,0.496516,Non soumise atteint taux légal,855,NaN,0.218770,NaN,27650.0,NaN,13826.0
7,01035,nan,200042935,0.000000,Non soumise sous seuil pop,0,NaN,0.218770,NaN,27650.0,NaN,13826.0
8,01038,nan,200071751,0.000000,Non soumise sous seuil pop,0,NaN,0.169795,NaN,59136.0,NaN,30905.0
9,01040,nan,200071751,0.000000,Non soumise sous seuil pop,0,NaN,0.169795,NaN,59136.0,NaN,30905.0


In [167]:
# vérification des valeurs aberrantes
df_21[df_21.taux_ls_21 > .7]

,code_commune,code_agglo_21,code_epci_21,taux_ls_21,situation_21,LLS_21,taux_agglo_21,taux_epci_21,rp_agglo_21,rp_epci_21,rp_agglo_nc_21,rp_epci_nc_21
2062,27701,nan,200089456,0.717693,Non soumise atteint taux légal,3099,NaN,0.217150,NaN,42100.0,NaN,20932.0
4368,57058,57501,245700372,0.825359,Non soumise atteint taux légal,2070,0.288885,0.206709,36319.0,33864.0,7589.0,13771.0
5261,62386,00756,246200364,0.708738,Non soumise atteint taux légal,1898,0.373713,0.479724,210702.0,100637.0,6424.0,904.0
6484,76157,00755,200023414,0.759456,Non soumise atteint taux légal,4859,0.384286,0.371094,219261.0,228573.0,11205.0,21634.0
6488,76178,00755,200023414,0.847944,Non soumise atteint taux légal,1567,0.384286,0.371094,219261.0,228573.0,11205.0,21634.0
6507,76305,76701,200084952,0.745093,Non soumise atteint taux légal,2847,0.364513,0.326547,109741.0,123284.0,7481.0,19087.0
6512,76322,00755,200023414,0.721454,Non soumise atteint taux légal,9389,0.384286,0.371094,219261.0,228573.0,11205.0,21634.0
6861,78644,00851,200058782,0.744093,Non soumise atteint taux légal,1669,0.283109,0.344540,4749662.0,93127.0,9708.0,0.0
7803,93030,00851,200054781,0.720299,Non soumise atteint taux légal,2892,0.283109,0.286365,4749662.0,3241454.0,9708.0,0.0
7978,97120,9A701,200018653,0.919083,Non soumise atteint taux légal,5293,0.291943,0.491017,96529.0,39131.0,0.0,0.0


# Assemblage

In [168]:
codes_communes = df_02_04.merge(df_05_07, how = 'outer').merge(df_08_10, how = 'outer').merge(df_11_13, how = 'outer').merge(df_14_16, how = 'outer').merge(df_17_19, how = 'outer').code_commune.unique()
len(codes_communes)

1314

In [169]:
communes_02_04 = df_02_04.merge(df_04, how = 'outer')
communes_02_05 = communes_02_04.merge(df_05, how = 'outer')
communes_02_06 = communes_02_05.merge(df_06, how = 'outer')
communes_02_07 = communes_02_06.merge(df_07, how = 'outer').merge(df_05_07, how = 'outer')
communes_02_08 = communes_02_07.merge(df_08, how = 'outer')
communes_02_09 = communes_02_08.merge(df_09, how = 'outer')
communes_02_10_1 = communes_02_09.merge(df_10, how = 'outer', copy = False, on = 'code_commune')
communes_02_10 = communes_02_10_1.merge(df_08_10, how = 'outer')
communes_02_11 = communes_02_10.merge(df_11, how = 'outer')
communes_02_12 = communes_02_11.merge(df_12, how = 'outer')
communes_02_13 = communes_02_12.merge(df_13, how = 'outer').merge(df_11_13, how = 'outer')
communes_02_14 = communes_02_13.merge(df_14, how = 'outer', copy = 'False', on = 'code_commune')
communes_02_15 = communes_02_14.merge(df_15, how = 'outer')
communes_02_16 = communes_02_15.merge(df_16, how = 'outer', copy = False).merge(df_14_16, how = 'outer', copy = False).drop_duplicates()
communes_02_17 = communes_02_16.merge(df_17, how = 'outer')
communes_02_18 = communes_02_17.merge(df_18, how = 'outer')
communes_02_19 = communes_02_18.merge(df_19, how = 'outer').merge(df_17_19, how = 'outer').drop_duplicates()
communes_02_20 = communes_02_19.merge(df_20, how = 'outer')
communes_02_21 = communes_02_20.merge(df_21, how = 'outer')

len(communes_02_21)

8526

In [170]:
communes_02_21.to_csv('../donnees_resultat/LLS_communes_quantitatif.csv', index = False)